# 1. <a id='toc1_'></a>[科学技術計算 3](#toc0_)

この演習トピックでは，pythonとnumpyを利用して，線形代数の基本であるベクトルと行列の基本的な演算（加算，減算，積，内積など）の実装と，計算量について学ぶ．さらに，共分散行列の計算などを通して，スカラーとの積やブロードキャスト，ndarrayの変形など，numpy特有の機能も学ぶ．また，時間計算量や空間計算量，大規模なデータを扱うための疎行列，数値計算ライブラリの適切な利用についても学ぶ．

線形代数については，各自の教科書・テキストを参考にすること．


以下で頻繁に使用するnumpyの関数を事前にインポートしておく．また，numpyにはない機能を補完するためにscipyも使用し，データセットを扱うためにscikit-learn（`sklearn`）もインポートする．

In [ ]:
import numpy as np
from numpy.linalg import det, inv, matrix_rank, norm
from numpy import eye, identity, diag
rng = np.random.default_rng()

import scipy
from sklearn.datasets import load_iris

import sys
import psutil


import matplotlib.pyplot as plt
from matplotlib import rcParams
rcParams["savefig.bbox"] = "tight"
plt.gray()


<Figure size 640x480 with 0 Axes>

**目次**<a id='toc0_'></a>    
- 1. [科学技術計算 3](#toc1_)    
  - 1.1. [添字についての注意](#toc1_1_)    
  - 1.2. [ベクトルと行列](#toc1_2_)    
    - 1.2.1. [ベクトル](#toc1_2_1_)    
      - 1.2.1.1. [$n$次元（列）ベクトル](#toc1_2_1_1_)    
      - 1.2.1.2. [$n$次元行ベクトル](#toc1_2_1_2_)    
    - 1.2.2. [$m \times n$行列](#toc1_2_2_)    
      - 1.2.2.1. [行列の転置](#toc1_2_2_1_)    
    - 1.2.3. [和と差](#toc1_2_3_)    
      - 1.2.3.1. [ベクトルの和](#toc1_2_3_1_)    
      - 1.2.3.2. [行列の和](#toc1_2_3_2_)    
    - 1.2.4. [Numpy の ndarray によるベクトルと行列](#toc1_2_4_)    
  - 1.3. [内積](#toc1_3_)    
    - 1.3.1. [ndarray の内積](#toc1_3_1_)    
  - 1.4. [計算量](#toc1_4_)    
    - 1.4.1. [flops](#toc1_4_1_)    
    - 1.4.2. [FLOPS](#toc1_4_2_)    
    - 1.4.3. [計算量の例](#toc1_4_3_)    
  - 1.5. [距離とノルム](#toc1_5_)    
    - 1.5.1. [距離](#toc1_5_1_)    
    - 1.5.2. [ノルム](#toc1_5_2_)    
    - 1.5.3. [ndarray のノルム](#toc1_5_3_)    
    - 1.5.4. [計算量](#toc1_5_4_)    
  - 1.6. [積](#toc1_6_)    
    - 1.6.1. [行列ベクトル積](#toc1_6_1_)    
      - 1.6.1.1. [行列と列ベクトルの積](#toc1_6_1_1_)    
      - 1.6.1.2. [行ベクトルと行列の積](#toc1_6_1_2_)    
    - 1.6.2. [行列同士の積](#toc1_6_2_)    
      - 1.6.2.1. [内積](#toc1_6_2_1_)    
      - 1.6.2.2. [直積](#toc1_6_2_2_)    
      - 1.6.2.3. [スカラーとの積](#toc1_6_2_3_)    
    - 1.6.3. [行列積の計算量](#toc1_6_3_)    
      - 1.6.3.1. [スカラーとの積](#toc1_6_3_1_)    
      - 1.6.3.2. [2つの行列の積](#toc1_6_3_2_)    
      - 1.6.3.3. [3つの行列の積](#toc1_6_3_3_)    
      - 1.6.3.4. [3つのベクトルの外積と内積](#toc1_6_3_4_)    
        - 1.6.3.4.1. [例1：$\boldsymbol{x} \boldsymbol{y}^T \boldsymbol{z}$](#toc1_6_3_4_1_)    
        - 1.6.3.4.2. [例2：$(I + \boldsymbol{x} \boldsymbol{y}^T) \boldsymbol{z}$](#toc1_6_3_4_2_)    
    - 1.6.4. [ndarrayの行列積](#toc1_6_4_)    
      - 1.6.4.1. [np.outer](#toc1_6_4_1_)    
      - 1.6.4.2. [reshape](#toc1_6_4_2_)    
        - 1.6.4.2.1. [reshapeの注意点](#toc1_6_4_2_1_)    
  - 1.7. [ndarray 特有の演算](#toc1_7_)    
    - 1.7.1. [ndarrayとスカラーとの積](#toc1_7_1_)    
    - 1.7.2. [ndarrayとスカラーとの和](#toc1_7_2_)    
    - 1.7.3. [要素毎の積](#toc1_7_3_)    
  - 1.8. [ブロードキャスト](#toc1_8_)    
    - 1.8.1. [行列の各行とベクトルの和](#toc1_8_1_)    
    - 1.8.2. [ブロードキャストの例：中心化](#toc1_8_2_)    
      - 1.8.2.1. [数式](#toc1_8_2_1_)    
      - 1.8.2.2. [ブロードキャスト](#toc1_8_2_2_)    
      - 1.8.2.3. [非効率な実装例](#toc1_8_2_3_)    
    - 1.8.3. [ブロードキャストの実際の例：共分散行列](#toc1_8_3_)    
      - 1.8.3.1. [行と列の方向](#toc1_8_3_1_)    
      - 1.8.3.2. [共分散行列の定義](#toc1_8_3_2_)    
      - 1.8.3.3. [アヤメのデータセット](#toc1_8_3_3_)    
  - 1.9. [線形代数の関数](#toc1_9_)    
    - 1.9.1. [単位行列](#toc1_9_1_)    
    - 1.9.2. [対角行列](#toc1_9_2_)    
      - 1.9.2.1. [対角行列の逆行列](#toc1_9_2_1_)    
      - 1.9.2.2. [対角行列の平方根](#toc1_9_2_2_)    
    - 1.9.3. [行列式](#toc1_9_3_)    
    - 1.9.4. [逆行列](#toc1_9_4_)    
    - 1.9.5. [ランク，階数](#toc1_9_5_)    
      - 1.9.5.1. [ランク計算の例](#toc1_9_5_1_)    
      - 1.9.5.2. [ランクの性質](#toc1_9_5_2_)    
      - 1.9.5.3. [ランク落ちの例](#toc1_9_5_3_)    
    - 1.9.6. [正定値対称行列](#toc1_9_6_)    
      - 1.9.6.1. [対称行列](#toc1_9_6_1_)    
      - 1.9.6.2. [正定値，負定置，半正定値](#toc1_9_6_2_)    
    - 1.9.7. [特殊な行列](#toc1_9_7_)    
      - 1.9.7.1. [対角行列](#toc1_9_7_1_)    
      - 1.9.7.2. [三重対角行列](#toc1_9_7_2_)    
      - 1.9.7.3. [帯行列](#toc1_9_7_3_)    
      - 1.9.7.4. [上三角行列](#toc1_9_7_4_)    
        - 1.9.7.4.1. [単位上三角行列](#toc1_9_7_4_1_)    
        - 1.9.7.4.2. [狭義上三角行列](#toc1_9_7_4_2_)    
        - 1.9.7.4.3. [上ヘッセンベルグ行列](#toc1_9_7_4_3_)    
        - 1.9.7.4.4. [numpy を用いた上三角行列の生成](#toc1_9_7_4_4_)    
      - 1.9.7.5. [下三角行列](#toc1_9_7_5_)    
        - 1.9.7.5.1. [単位下三角行列](#toc1_9_7_5_1_)    
        - 1.9.7.5.2. [狭義下三角行列](#toc1_9_7_5_2_)    
        - 1.9.7.5.3. [下ヘッセンベルグ行列](#toc1_9_7_5_3_)    
        - 1.9.7.5.4. [numpyを用いた下三角行列の生成](#toc1_9_7_5_4_)    
      - 1.9.7.6. [三角行列の行列式](#toc1_9_7_6_)    
        - 1.9.7.6.1. [特に対角成分が1である三角行列の行列式](#toc1_9_7_6_1_)    
  - 1.10. [行列の変換](#toc1_10_)    
    - 1.10.1. [直交行列](#toc1_10_1_)    
      - 1.10.1.1. [列直交行列](#toc1_10_1_1_)    
    - 1.10.2. [行列の基本変形](#toc1_10_2_)    
      - 1.10.2.1. [列・行の定数倍](#toc1_10_2_1_)    
      - 1.10.2.2. [列・行の置換](#toc1_10_2_2_)    
      - 1.10.2.3. [列・行の線形和](#toc1_10_2_3_)    
      - 1.10.2.4. [ndarayによる実装](#toc1_10_2_4_)    
        - 1.10.2.4.1. [定数倍](#toc1_10_2_4_1_)    
        - 1.10.2.4.2. [置換](#toc1_10_2_4_2_)    
        - 1.10.2.4.3. [線形和](#toc1_10_2_4_3_)    
  - 1.11. [時間計算量と空間計算量](#toc1_11_)    
    - 1.11.1. [notebook における計算時間の計測](#toc1_11_1_)    
    - 1.11.2. [計算時間の簡単な計測方法](#toc1_11_2_)    
    - 1.11.3. [実行時間の詳細な計測方法](#toc1_11_3_)    
    - 1.11.4. [メモリ使用量](#toc1_11_4_)    
  - 1.12. [密行列と疎行列](#toc1_12_)    
    - 1.12.1. [密行列とは](#toc1_12_1_)    
    - 1.12.2. [疎行列とは](#toc1_12_2_)    
    - 1.12.3. [疎行列の格納方法](#toc1_12_3_)    
    - 1.12.4. [密行列のメモリ使用量の概算](#toc1_12_4_)    
    - 1.12.5. [scipyにおける疎行列](#toc1_12_5_)    
      - 1.12.5.1. [CSR形式](#toc1_12_5_1_)    
        - 1.12.5.1.1. [CSR疎行列の情報：indptr, indices, data](#toc1_12_5_1_1_)    
      - 1.12.5.2. [COO形式](#toc1_12_5_2_)    
        - 1.12.5.2.1. [COO疎行列の情報](#toc1_12_5_2_1_)    
      - 1.12.5.3. [大規模疎行列の例](#toc1_12_5_3_)    
        - 1.12.5.3.1. [ファイルの読み込みと疎行列への変換](#toc1_12_5_3_1_)    
        - 1.12.5.3.2. [メモリ使用量](#toc1_12_5_3_2_)    
  - 1.13. [数値計算ライブラリとの関係](#toc1_13_)    
    - 1.13.1. [BLAS](#toc1_13_1_)    
      - 1.13.1.1. [BLASの高速化](#toc1_13_1_1_)    
      - 1.13.1.2. [オープンソース実装](#toc1_13_1_2_)    
      - 1.13.1.3. [BLASの代表的なAPI](#toc1_13_1_3_)    
      - 1.13.1.4. [numpyとBLAS](#toc1_13_1_4_)    
    - 1.13.2. [LAPACK](#toc1_13_2_)    
      - 1.13.2.1. [LAPACKのAPI](#toc1_13_2_1_)    
    - 1.13.3. [numpyのBLAS・LAPACK](#toc1_13_3_)    
    - 1.13.4. [scipyのBLAS・LAPACK](#toc1_13_4_)    
    - 1.13.5. [疎行列の実装とライブラリ](#toc1_13_5_)    

<!-- vscode-jupyter-toc-config
	numbering=true
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->


## 1.1. <a id='toc1_1_'></a>[添字についての注意](#toc0_)

数式ではベクトルや行列の添字は $x_1, \ldots, x_n$ のように1から始めることが多いが，Pythonのリストや`numpy`の`ndarray`では，添字が `v[0], ..., v[n-1]` のように0から始まる．言語によって異なるため，これには注意が必要である．

- 例えば，C，C++，Java，PHP，Pythonなどは添字が0から始まる **0オリジン（0-based）** である．
- 一方で，Fortran，MATLAB，R，Lua，Juliaなどは添字が1から始まる **1オリジン（1-based）** である．

そのため，特に数式をコードに落とし込む際には，この添字の違いに気をつける必要がある．

- https://en.wikipedia.org/wiki/Comparison_of_programming_languages_(array)
    > Some languages index from zero. Some index from one. Some carry no such restriction, or even allow indexing by any enumerated type, not only integers.



## 1.2. <a id='toc1_2_'></a>[ベクトルと行列](#toc0_)

### 1.2.1. <a id='toc1_2_1_'></a>[ベクトル](#toc0_)

#### 1.2.1.1. <a id='toc1_2_1_1_'></a>[$n$次元（列）ベクトル](#toc0_)

数値を$n$個並べたものを数ベクトル，単にベクトル（vector）と呼び，その数値を要素とする．以下のベクトル$\boldsymbol{x}$は，実数$x_i \in R$を要素に持つ$n$次元ベクトル（$n$-vector）であり，通常は縦方向に並べた列ベクトルとして表される．

$$
\boldsymbol{x}
= \begin{pmatrix}
x_1 \\
x_2 \\
\vdots \\
x_n
\end{pmatrix}
\in R^n
$$


数学的には，ある体$C$上の線形空間$V$を考え，その元$x$をベクトルと呼ぶ（詳細は線形代数のテキストを参照）．



#### 1.2.1.2. <a id='toc1_2_1_2_'></a>[$n$次元行ベクトル](#toc0_)

要素を横方向に並べたベクトルを行ベクトルと呼ぶ．例えば，行ベクトル$\boldsymbol{y}$は次のように表される．

$$
\boldsymbol{y} = (y_1, y_2, \ldots, y_n) \in R^n
$$

列ベクトルの転置を行ベクトルとして表す場合には，転置記号$T$を用いる．例えば，列ベクトル$\boldsymbol{x}$の転置は以下のように書ける．

$$
\boldsymbol{x}^T = (x_1, x_2, \ldots, x_n) \in R^n
$$

また，列ベクトルを定義するときに，余白を節約するために行ベクトルの形式で表す場合も多い．

$$
\boldsymbol{x} = (x_1, x_2, \ldots, x_n)^T \in R^n
$$

ベクトル$\boldsymbol{x}$の$i$番目の要素が$x_i$であるとき，その要素を$[\boldsymbol{x}]_{i}$で表すことがある．

$$
[\boldsymbol{x}]_{i} = x_i
$$


### 1.2.2. <a id='toc1_2_2_'></a>[$m \times n$行列](#toc0_)

要素$x_{ij} \in R$を行（縦）方向$i=1,2,\ldots,m$と列（横）方向$j=1,2,\ldots,n$に並べたものを行列（matrix）と呼ぶ．

$$
X =
\begin{pmatrix}
x_{11} & x_{12} & \cdots & x_{1n} \\
x_{21} & x_{22} & \cdots & x_{2n} \\
\vdots & \vdots & \ddots & \vdots \\
x_{m1} & x_{m2} & \cdots & x_{mn}
\end{pmatrix}
    \in R^{m \times n}
$$

この行列の次元は$m \times n$であり，$m \times n$行列であるともいう．
特に$m=n$の場合には$n$次正方行列であるという．

行列$X$の$i$行$j$列目の要素が$x_{ij}$のとき，
その要素を$[X]_{ij}$で表す場合がある．


#### 1.2.2.1. <a id='toc1_2_2_1_'></a>[行列の転置](#toc0_)

行列の縦と横を入れ替える操作を転置と呼び，
記号$T$を用いて$X$の転置を$X^T$と表す．

$$
X^T =
\begin{pmatrix}
x_{11} & x_{21} & \cdots & x_{m1} \\
x_{12} & x_{22} & \cdots & x_{m2} \\
\vdots & \vdots & \ddots & \vdots \\
x_{1n} & x_{2n} & \cdots & x_{mn}
\end{pmatrix}
    \in R^{n \times m}
$$

つまり$[X]_{ij} = x_{ij}$のとき，$[X^T]_{ij} = x_{ji}$である．


### 1.2.3. <a id='toc1_2_3_'></a>[和と差](#toc0_)

ベクトルと行列の加算と減算は，それぞれの要素の和と差である．

#### 1.2.3.1. <a id='toc1_2_3_1_'></a>[ベクトルの和](#toc0_)

$\boldsymbol{a}, \boldsymbol{b}, \boldsymbol{c} \in R^n$について
$$
\boldsymbol{a} = (a_1, a_2, \ldots, a_n)^T, \quad
\boldsymbol{b} = (b_1, b_2, \ldots, b_n)^T
$$
とすると，ベクトルの和と差は以下のように定義される．

$$
\boldsymbol{a} \pm \boldsymbol{b} = \boldsymbol{c} = (a_1 \pm b_1, a_2 \pm b_2, \ldots, a_n \pm b_n)^T
$$

各要素$i$に対して，

$$
[\boldsymbol{a} \pm \boldsymbol{b}]_i = [\boldsymbol{c}]_i
$$

#### 1.2.3.2. <a id='toc1_2_3_2_'></a>[行列の和](#toc0_)

行列$A, B, C \in R^{m \times n}$について，

$$
[A]_{ij} = a_{ij}, \quad [B]_{ij} = b_{ij}
$$

とすると，行列の和と差は以下のように定義される．

$$
A \pm B = C
$$

各要素$ij$に対して，

$$
c_{ij} = a_{ij} \pm b_{ij}
$$


### 1.2.4. <a id='toc1_2_4_'></a>[Numpy の ndarray によるベクトルと行列](#toc0_)

Pythonでベクトルや行列を扱うためには，numpyの`ndarray`を用いる．具体的には，1次元の`ndarray`がベクトル，2次元の`ndarray`が行列に相当する．


なお，numpyには`numpy.matrix`というデータ型も存在するが，これは非推奨（deprecated）であり，現在は`ndarray`を使用することが推奨されている．今後，`numpy.matrix`が削除される可能性もあるため，使用は避けるべきである．

- https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
    > It is no longer recommended to use this class, even for linear algebra. Instead use regular arrays. The class may be removed in the future.


以下に，5次元のベクトルを生成し，それらの和を計算する例を示す．
この例では，1次元の`ndarray`に対して列ベクトル（縦ベクトル）と行ベクトル（横ベクトル）の区別はなく，同じ構造を持つ．


In [12]:
n = 5
a = rng.random(n)
b = rng.random(n)
c = a + b
print("a", a, a.ndim, a.shape)
print("b", b, b.ndim, b.shape)
print("c", c, c.ndim, c.shape)


a [0.67632586 0.73177317 0.93528674 0.64340165 0.94949066] 1 (5,)
b [0.253642   0.09233153 0.60410291 0.94917012 0.44695365] 1 (5,)
c [0.92996786 0.8241047  1.53938965 1.59257177 1.39644432] 1 (5,)


以下は 3x5 行列を生成し，和を計算している．


In [13]:
m = 3
n = 5
A = rng.random(size=(m, n))
B = rng.random(size=(m, n))
C = A + B
print("A\n", A, A.ndim, A.shape)
print("B\n", B, B.ndim, B.shape)
print("C\n", C, C.ndim, C.shape)


A
 [[0.50379377 0.04330207 0.12554653 0.94730822 0.05753813]
 [0.57142443 0.00239915 0.68625256 0.50939338 0.65871324]
 [0.72687696 0.61534083 0.28750942 0.47952469 0.58906521]] 2 (3, 5)
B
 [[0.86360522 0.06914288 0.31351434 0.09979439 0.23643496]
 [0.13402651 0.03742835 0.26522738 0.94329281 0.52007656]
 [0.48108735 0.10316315 0.31456872 0.27421954 0.98264912]] 2 (3, 5)
C
 [[1.36739899 0.11244495 0.43906087 1.04710261 0.29397309]
 [0.70545094 0.03982749 0.95147994 1.45268619 1.1787898 ]
 [1.20796431 0.71850398 0.60207814 0.75374423 1.57171433]] 2 (3, 5)




行列を転置するためには，ndarrayのメソッドである`.T`を使用する．以下では，行列の和の転置，すなわち

$$
A + B = C \quad \Rightarrow \quad (A^T + B^T)^T = C
$$

を確認している．

In [14]:
print("A\n", A)
print("A^T\n", A.T)

C = A + B
print("C\n", C)
print("(A^T + B^T)^T\n", (A.T + B.T).T)
print("(A^T + B^T)^T - C\n", (A.T + B.T).T - C)
print("(A^T + B^T)^T == C ?", np.allclose((A.T + B.T).T, C))


A
 [[0.50379377 0.04330207 0.12554653 0.94730822 0.05753813]
 [0.57142443 0.00239915 0.68625256 0.50939338 0.65871324]
 [0.72687696 0.61534083 0.28750942 0.47952469 0.58906521]]
A^T
 [[0.50379377 0.57142443 0.72687696]
 [0.04330207 0.00239915 0.61534083]
 [0.12554653 0.68625256 0.28750942]
 [0.94730822 0.50939338 0.47952469]
 [0.05753813 0.65871324 0.58906521]]
C
 [[1.36739899 0.11244495 0.43906087 1.04710261 0.29397309]
 [0.70545094 0.03982749 0.95147994 1.45268619 1.1787898 ]
 [1.20796431 0.71850398 0.60207814 0.75374423 1.57171433]]
(A^T + B^T)^T
 [[1.36739899 0.11244495 0.43906087 1.04710261 0.29397309]
 [0.70545094 0.03982749 0.95147994 1.45268619 1.1787898 ]
 [1.20796431 0.71850398 0.60207814 0.75374423 1.57171433]]
(A^T + B^T)^T - C
 [[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]
(A^T + B^T)^T == C ? True


なお次元が異なる場合には和は，当然エラーになる．


In [15]:
a = rng.random(3)
b = rng.random(5)
c = a + c


ValueError: operands could not be broadcast together with shapes (3,) (5,) 

In [ ]:
m = 3
n = 5
A = rng.random(size=(m, n))

A + A.T


ValueError: operands could not be broadcast together with shapes (3,5) (5,3) 

## 1.3. <a id='toc1_3_'></a>[内積](#toc0_)

ベクトル $\boldsymbol{x}, \boldsymbol{y} \in R^n$ の内積 $\boldsymbol{x}^T \boldsymbol{y} \in R$ は，次のように定義される．

\begin{align*}
\boldsymbol{x}^T \boldsymbol{y}
&=
(x_1, x_2, \ldots, x_n)
\begin{pmatrix}
y_1 \\ y_2 \\ \vdots \\ y_n
\end{pmatrix} \\
&=
x_1 y_1 + x_2 y_2 + \cdots + x_n y_n \\
&=
\sum_i x_i y_i
\end{align*}


数学的には，線形空間の元（ベクトル）に対する双線形関数として定義され，$(\boldsymbol{x}, \boldsymbol{y})$, $\langle \boldsymbol{x}, \boldsymbol{y} \rangle$, $\boldsymbol{x} \cdot \boldsymbol{y}$ などと表記されることがある．この定義は $R^n$ 上の標準内積であり，ベクトルの転置による行列積として $\boldsymbol{x}^T \boldsymbol{y}$ という記法が使われている．内積の結果はスカラーであるため，スカラー積とも呼ばれ，また $\boldsymbol{x} \cdot \boldsymbol{y}$ の記法からドット積とも呼ばれることがある．


### 1.3.1. <a id='toc1_3_1_'></a>[ndarray の内積](#toc0_)


ndarray のベクトル同士の内積は，`@` 演算子を用いて`x @ y` で計算することができる．内積は対称的であるため，$\boldsymbol{x}^T \boldsymbol{y}$ と $\boldsymbol{y}^T \boldsymbol{x}$ は同じ結果になる．

なお，1次元 ndarray に対して `.T` を使用しても，形状は変わらないため，以下のように`x @ y` と `x.T @ y` の結果は同じになる．

In [ ]:
n = 5
x = rng.random(n)
y = rng.random(n)
print("x", x)
print("y", y)
print("(x, y)", x @ y)
print("(y, x)", y @ x)
print("x^T", x.T)
print("y^T", y.T)
print("x^T y", x.T @ y)
print("y^T x", y.T @ x)


x [0.24081133 0.49102954 0.88037271 0.6103513  0.56956428]
y [0.31805414 0.39210807 0.33422405 0.21704466 0.92817235]
(x, y) 1.2244967244192317
(y, x) 1.2244967244192317
x^T [0.24081133 0.49102954 0.88037271 0.6103513  0.56956428]
y^T [0.31805414 0.39210807 0.33422405 0.21704466 0.92817235]
x^T y 1.2244967244192317
y^T x 1.2244967244192317


`.dot()`メソッドでも内積を計算することはできるが，`@`演算子の方が便利であり，以降は`@`を使用する．

In [ ]:
print(x @ y)
print(x.dot(y))
print(y.dot(x))
print(np.dot(x, y))
print(np.dot(y, x))


1.2244967244192317
1.2244967244192317
1.2244967244192317
1.2244967244192317
1.2244967244192317


内積には以下の性質がある．

- $\langle \boldsymbol{a}, \boldsymbol{a} \rangle \ge 0 , \ \text{and} \ \langle \boldsymbol{a}, \boldsymbol{a} \rangle = 0 \iff \boldsymbol{a} = \boldsymbol{0}$
- $\langle \boldsymbol{a}, \boldsymbol{b} \rangle = \langle \boldsymbol{b}, \boldsymbol{a} \rangle$
- $\alpha, \beta \in R$について$\langle \alpha \boldsymbol{a} + \beta \boldsymbol{b}, \boldsymbol{c} \rangle = \alpha \langle \boldsymbol{a}, \boldsymbol{c} \rangle + \beta \langle \boldsymbol{b}, \boldsymbol{c} \rangle$

以下はこれを確認する例である．このコードでは，内積が非負であること，内積が対称性を持つこと，そして双線形性を持つことを確かめている．


In [ ]:
n = 5
a = rng.random(n)
b = rng.random(n)
c = rng.random(n)
alpha = rng.random()
beta = rng.random()

print("<a,a> >= 0:", a @ a)
print("<a,b> == <b,a>:", np.isclose(a @ b, b @ a))

print(
    "<alpha a + beta b, c> == alpha <a,c> + beta <b,c>:",
    np.isclose(
        (alpha * a + beta * b) @ c,
        alpha * (a @ c) + beta * (b @ c)
        )
    )


<a,a> >= 0: 1.284998192785487
<a,b> == <b,a>: True
<alpha a + beta b, c> == alpha <a,c> + beta <b,c>: True


## 1.4. <a id='toc1_4_'></a>[計算量](#toc0_)

### 1.4.1. <a id='toc1_4_1_'></a>[flops](#toc0_)

内積計算では，要素ごとの積$x_i y_i$と，その総和$\sum_{i=1}^n$を行う．これらの計算は浮動小数点数の操作であり，計算量は浮動小数点演算の回数で測定される．**浮動小数点演算**の1回の加減乗除を「flop」（floating point operation，浮動小数点演算）と呼び，その回数を「**flops**」と表記する（複数形のsは「floating point operation**s**」を指す）．

$n$次元ベクトルの内積は，$n$回の乗算と$n-1$回の加算で構成されるが，通常は`y = y + a*x`という**積和計算**（multiply-accumulation; MAC, multiply-addition; MAD）の形で計算されるため，実際の計算量は$2n$ flopsとなる．実際のCPUなどでは**融合積和演算**（fused multiply-addition; FMA）実装されており，MACは1命令で実行される．

- https://en.wikipedia.org/wiki/Multiply%E2%80%93accumulate_operation
    > In computing, especially digital signal processing, the multiply–accumulate (MAC) or multiply-add (MAD) operation is a common step that computes the product of two numbers and adds that product to an accumulator.
- https://en.wikipedia.org/wiki/Multiply%E2%80%93accumulate_operation#Fused_multiply%E2%80%93add
    > A fused multiply–add (FMA or fmadd)[7] is a floating-point multiply–add operation performed in one step (fused operation), with a single rounding.

MACの実装は，単精度浮動小数点数の計算ではSAXPY（single-precision ax plus y），倍精度ではDAXPY（double-precision ax plus y）という名称が使われている．

- https://developer.nvidia.com/blog/six-ways-saxpy/
    > SAXPY stands for "Single-Precision A·X Plus Y".  It is a function in the standard Basic Linear Algebra Subroutines (BLAS)library.
- https://www.ibm.com/docs/en/essl/6.2?topic=vss-saxpy-daxpy-caxpy-zaxpy-multiply-vector-by-scalar-add-vector-store-in-vector
    > SAXPY, DAXPY, CAXPY, and ZAXPY (Multiply a Vector X by a Scalar, Add to a Vector Y, and Store in the Vector Y)

内積の計算量はビッグオー記法で$O(n)$となる．なお一般的なアルゴリズム論の議論とは異なり，
数値計算では定数倍の違いも性能に大きな影響を与えるため，
オーダー記法だけでなく，flopsの具体的な数値を使って計算量を議論することが多い．


### 1.4.2. <a id='toc1_4_2_'></a>[FLOPS](#toc0_)

「flops」と類似した指標に，計算機の性能指標であるFLOPS（floating point operations **per second**; FLOPS, FLOPs, flops）がある．これは**1秒間に**処理できる浮動小数点数演算の回数を示し，CPUの計算能力を表す．

例えば，Intel Core i9の性能は1.18TFLOPSとされている<sup>1</sup>．$n=100$次元ベクトルの内積計算には200 flopsかかるため，単純計算で0.169ナノ秒かかることがわかる（ただし，実際の計算時間は様々な要因に依存する）．

- <sup>1</sup>https://ja.wikipedia.org/wiki/FLOPS


In [ ]:
n = 100
flop_counts = 2 * n
core_i9_flops = 1.18e12
time_in_second = flop_counts / core_i9_flops
time_in_nano_second = time_in_second / 1e-9
print(time_in_nano_second, "ns")


0.1694915254237288 ns


今後の議論では，ハードウェアに依存しない計算量を扱うため，「flops」はflop counts（浮動小数点演算の回数）を指すものとする（flops per secondを意味しない）．

### 1.4.3. <a id='toc1_4_3_'></a>[計算量の例](#toc0_)

計算量の例として，ベクトルや行列の和にかかるflopsを見てみる．
ベクトルや行列の和の計算量は，それぞれの要素数に比例する．

- **$n$次元ベクトルの和**: 要素ごとに1回の加算が必要なので，計算量は$n$ flops．
  - 具体的には，$\boldsymbol{a} = (a_1, a_2, \ldots, a_n)^T$と$\boldsymbol{b} = (b_1, b_2, \ldots, b_n)^T$に対して，各要素で1回の加算を行うため，$n$回の加算が必要．

- **$m \times n$行列の和**: 要素ごとに1回の加算が必要なので，計算量は$mn$ flops．
  - 具体的には，$A = [a_{ij}]$と$B = [b_{ij}]$の各要素について1回の加算が行われ，$m \times n$個の要素があるため，$mn$回の加算が必要．

上述したベクトルの内積の計算量についても，再度示しておく．

- **$n$次元ベクトルの内積**: 要素ごとに1回の乗算と1回の加算が必要なので，計算量は$2n$ flops．
  - 具体的には，$\boldsymbol{a} = (a_1, a_2, \ldots, a_n)^T$と$\boldsymbol{b} = (b_1, b_2, \ldots, b_n)^T$に対して，各要素で1回の乗算とその後の加算の積和計算が行われるため，$n$回の乗算と$n$回の加算が必要で，合計$2n$ flopsとなる．


## 1.5. <a id='toc1_5_'></a>[距離とノルム](#toc0_)

### 1.5.1. <a id='toc1_5_1_'></a>[距離](#toc0_)

ベクトル$\boldsymbol{a}, \boldsymbol{b} \in R^n$との距離を$d(\boldsymbol{a},\boldsymbol{b}) \in R$と書く．これはベクトル間が「どれくらい離れているか」を示す値である．

- https://ja.wikipedia.org/wiki/%E8%B7%9D%E9%9B%A2%E7%A9%BA%E9%96%93
    > dが以下の3つの条件（距離の公理という）を全て満たすとき、dはX上の距離関数、もしくは単にX上の距離（英: metric）という

距離関数$d$は，次の3つの性質（距離の公理）を満たす必要がある．

- $d(\boldsymbol{a},\boldsymbol{b}) \ge 0, \ \text{and} \ d(\boldsymbol{a},\boldsymbol{b}) = 0 \iff \boldsymbol{a} = \boldsymbol{b}$
- $d(\boldsymbol{a},\boldsymbol{b}) = d(\boldsymbol{b},\boldsymbol{a})$
- $d(\boldsymbol{a},\boldsymbol{b}) \le d(\boldsymbol{a},\boldsymbol{c}) + d(\boldsymbol{c},\boldsymbol{b})$


距離は，ベクトルの差をノルム（後述）で次のように表すことによっても定義できる．このようにして定められた距離を「ノルムが誘導する距離」と呼ぶ．


- $d(\boldsymbol{a},\boldsymbol{b}) = \| \boldsymbol{a} - \boldsymbol{b} \|$

この式では，ベクトル$\boldsymbol{a}$と$\boldsymbol{b}$の差を取り，それをノルムで測ることで，二つのベクトルの間の距離を計算する．


### 1.5.2. <a id='toc1_5_2_'></a>[ノルム](#toc0_)

ベクトル$\boldsymbol{a}$の「大きさ」や「長さ」を表すものがノルム（norm）であり，これを$\|\boldsymbol{a}\|$と書く．

- https://ja.wikipedia.org/wiki/%E3%83%8E%E3%83%AB%E3%83%A0%E7%B7%9A%E5%9E%8B%E7%A9%BA%E9%96%93
  > ノルム体 K 上のノルム線型空間とは、K-線型空間 V と V 上のノルム ‖ • ‖ の組 (V, ‖ • ‖) を言う。

ノルムは以下の3つの性質を満たす．

- $\|\boldsymbol{a}\| \ge 0, \ \text{and} \ \|\boldsymbol{a}\| = 0 \iff \boldsymbol{a} = \boldsymbol{0}$
- $\alpha \in R, \| \alpha \boldsymbol{a} \| = |\alpha| \ \|\boldsymbol{a}\|$
- $\| \boldsymbol{a} + \boldsymbol{b} \| \le \|\boldsymbol{a}\| + \|\boldsymbol{b}\|$


ノルムにはいくつかの種類があり，どのノルムを使うのかを明示するために，$\|\boldsymbol{a}\|_2$や$\|\boldsymbol{a}\|_p$といったように，下に添字をつけて表記する．よく使うノルムには次のようなものがある．


- $L_p$ノルム：一般的なノルム．
    - $\|\boldsymbol{x}\|_p = \left(\sum_i x_i^p\right)^{\frac{1}{p}}$
    > https://ja.wikipedia.org/wiki/Lp%E7%A9%BA%E9%96%93
- $L_2$ノルム（ユークリッドノルム）：ベクトルの長さを測る際によく使われる．
    - $\|\boldsymbol{x}\|_2  = \sqrt{\boldsymbol{x}^T \boldsymbol{x}} = \sqrt{\sum_i x_i^2}$
    - なお$L_2$ ノルムの 2 乗
    $\|\boldsymbol{x}\|_2^2 = \boldsymbol{x}^T \boldsymbol{x} = \sum_i x_i^2$は平方根をとらないため，計算が簡単で応用によく用いられることが多い．
- $L_1$ノルム（絶対値ノルム）：各要素の絶対値を合計したもの．
    - $\|\boldsymbol{x}\|_1 = \sum_i |x_i|$
- $L_0$ノルム：ベクトルの中で非ゼロの要素の個数を数えるもの．
    - $\|\boldsymbol{x}\|_0 = \#\{ i \ | \ x_i = 0 \}$
    > https://en.wikipedia.org/wiki/Lp_space#When_p_=_0
- $L_\infty$ ノルム（最大値ノルム）：ベクトルの中で最も大きな絶対値を持つ要素を選ぶ．
    - $\|\boldsymbol{x}\|_\infty = \max_i \{ |x_i| \}$
    > https://en.wikipedia.org/wiki/L-infinity

なお，$L_0$ノルムはノルムの定義を満たしていないが，「$L_0$ノルム」と呼ばれている．


### 1.5.3. <a id='toc1_5_3_'></a>[ndarray のノルム](#toc0_)

$L_2$ノルムは，その定義から，ベクトル自身の内積の平方根を取ることで求めることができる．


In [ ]:
n = 5
x = rng.random(n)

print("x", x)
print("||x||_2", np.sqrt(x @ x))


x [0.35177106 0.37718318 0.83243923 0.93875109 0.02199291]
||x||_2 1.3567248767885909


しかしnumpyにはノルムを計算する関数として`np.linalg.norm()`が用意されており，scipyにも同様に`scipy.linalg.norm()`がある．これらはベクトルや行列のノルムを効率的に計算する．なお，`linalg`とは「線形代数（linear algebra）」の略である．



- https://numpy.org/doc/stable/reference/generated/numpy.linalg.norm.html
    > Matrix or vector norm.
- https://docs.scipy.org/doc/scipy/reference/generated/scipy.linalg.norm.html
    > Matrix or vector norm.


デフォルトでは，`np.linalg.norm()`は$L_2$ノルムを返す．例えば，以下のコードでベクトルの$L_2$ノルムを計算できる．


In [ ]:
print("||x||_2", norm(x))


||x||_2 1.3567248767885909



また，他のノルム（$L_1$ノルム，$L_0$ノルム，$L_\infty$ノルム）を計算する場合は，`ord`という引数にそれぞれ対応する値を渡す．具体的には，$L_1$ノルムなら`ord=1`，$L_0$ノルムなら`ord=0`，$L_\infty$ノルムなら`ord=np.inf`を指定する．


In [ ]:
print("||x||_2", norm(x, ord=2))  # L2
print("||x||_1", norm(x, ord=1))  # L1
print("||x||_0", norm(x, ord=0))  # L0
print("||x||_inf", norm(x, ord=np.inf))  # L_inf


||x||_2 1.3567248767885909
||x||_1 2.5221374614799523
||x||_0 5.0
||x||_inf 0.938751089847886


### 1.5.4. <a id='toc1_5_4_'></a>[計算量](#toc0_)

ノルムの計算の計算量は，$L_1$ノルムはは$2n$ flops，$L_2$ノルムでは$2n$ flopsと平方根計算1回となる．$L_2$ノルムの計算では平方根を取るが，平方根の計算には通常の加減乗除よりも実際には時間がかかる．しかし，flopsのカウントではこの平方根の計算は別扱いされることが多い．

また，ユークリッド距離（$L_2$ノルム）$|| \boldsymbol{x} - \boldsymbol{y} ||_2$を計算する場合，まずベクトルの引き算を行い，その後ノルムを計算する．この場合，計算量は$n + 2n = 3n$ flopsとなる．



## 1.6. <a id='toc1_6_'></a>[積](#toc0_)

ここでは，ベクトルと行列の積の計算方法について説明する．

### 1.6.1. <a id='toc1_6_1_'></a>[行列ベクトル積](#toc0_)

#### 1.6.1.1. <a id='toc1_6_1_1_'></a>[行列と列ベクトルの積](#toc0_)

行列$A \in R^{m \times n}$と列ベクトル$\boldsymbol{x} \in R^n$の
積$A\boldsymbol{x} \in R^m$は次のように計算される．

\begin{align*}
A \boldsymbol{x} &=
\begin{pmatrix}
a_{11} & a_{12} & \cdots & a_{1n} \\
a_{21} & a_{22} & \cdots & a_{2n} \\
\vdots & \vdots & \ddots & \vdots \\
a_{m1} & a_{m2} & \cdots & a_{mn}
\end{pmatrix}
\begin{pmatrix}
x_{1} \\
x_{2} \\
\\
\vdots \\
\\
x_{n}
\end{pmatrix}
=
\begin{pmatrix}
\sum_i a_{1i} x_i \\
\sum_i a_{2i} x_i \\
\vdots \\
\sum_i a_{mi} x_i \\
\end{pmatrix}
\end{align*}

つまり，行列の行ごとに，対応するベクトルの成分を掛けて足し合わせることで，$m$次元の列ベクトルを得る．

#### 1.6.1.2. <a id='toc1_6_1_2_'></a>[行ベクトルと行列の積](#toc0_)

行ベクトル$\boldsymbol{x} \in R^m$と行列$A \in R^{m \times n}$の積
$\boldsymbol{x}^T A \in R^n$は次のように計算される．

\begin{align*}
\boldsymbol{x}^T A &=
\begin{pmatrix}
x_{1} &
x_{2} &
\cdots &
x_{m}
\end{pmatrix}
\begin{pmatrix}
a_{11} & a_{12} & \cdots & a_{1n} \\
a_{21} & a_{22} & \cdots & a_{2n} \\
\vdots & \vdots & \ddots & \vdots \\
a_{m1} & a_{m2} & \cdots & a_{mn}
\end{pmatrix}
\\
&=
\begin{pmatrix}
\sum_i a_{i1} x_i &
\sum_i a_{i2} x_i &
\cdots &
\sum_i a_{in} x_i
\end{pmatrix}
\end{align*}

つまり，行ベクトルの各成分と行列の各列を掛けて足し合わせ，$n$次元の行ベクトルを得る．



### 1.6.2. <a id='toc1_6_2_'></a>[行列同士の積](#toc0_)

行列$A \in R^{m \times n}$と行列$B \in R^{n \times p}$の積
$AB = C \in R^{m \times p}$は次のように計算される．

\begin{align*}
A B &=
\begin{pmatrix}
a_{11} & a_{12} & \cdots & a_{1n} \\
a_{21} & a_{22} & \cdots & a_{2n} \\
\vdots & \vdots & \ddots & \vdots \\
a_{m1} & a_{m2} & \cdots & a_{mn}
\end{pmatrix}
\begin{pmatrix}
b_{11} & b_{12} & \cdots & b_{1p} \\
b_{21} & b_{22} & \cdots & b_{2p} \\
\vdots & \vdots & \ddots & \vdots \\
b_{n1} & b_{n2} & \cdots & b_{np}
\end{pmatrix}
=
\begin{pmatrix}
c_{11} & c_{12} & \cdots & c_{1p} \\
c_{21} & c_{22} & \cdots & c_{2p} \\
\vdots & \vdots & \ddots & \vdots \\
c_{m1} & c_{m2} & \cdots & c_{mp}
\end{pmatrix}
\end{align*}

行列積の各要素$[C]_{j\ell}$は，次の式で表される．

\begin{align*}
[AB]_{j\ell}= [C]_{j\ell} = \sum_i a_{ji} b_{i\ell}
\end{align*}

つまり$C$の$j\ell$要素は，$A$の$j$行目と$B$の$\ell$列目の要素を掛け合わせて足し合わせることで求める．

また，行列の積に関しては次のような転置の性質が成り立つ．

$$
(AB)^T = B^T A^T
$$




#### 1.6.2.1. <a id='toc1_6_2_1_'></a>[内積](#toc0_)


ベクトル$\boldsymbol{x} \in R^n$とベクトル$\boldsymbol{y} \in R^n$の内積（inner product）は，次のように表される．

\begin{align*}
\boldsymbol{x}^T \boldsymbol{y}
&=
(x_1, x_2, \ldots, x_n)
\begin{pmatrix}
y_1 \\ y_2 \\ \vdots \\ y_n
\end{pmatrix}
=
\sum_i x_i y_i \in R
\end{align*}

内積は1つのスカラー値を返す計算であり，つまり$\boldsymbol{x}^T \boldsymbol{y} \in R$である．$1 \times n$行列と$n \times 1$行列の積としては，この結果は$1 \times 1$の行列とみなすことができるが（$\boldsymbol{x}^T \boldsymbol{y} \in R^{1 \times 1}$），通常はスカラーとして扱う．


さらに，$n$次正方行列$A$を用いた内積の定義もよく使われる．この場合，次のように表される．

\begin{align*}
\boldsymbol{x}^T A \boldsymbol{y}
&=
\sum_{i,j} a_{ij} x_i y_j
\end{align*}



#### 1.6.2.2. <a id='toc1_6_2_2_'></a>[直積](#toc0_)

ベクトル$\boldsymbol{x} \in R^m$と$\boldsymbol{y} \in R^n$の直積（cross product）とは，次のように定義される．

\begin{align*}
\boldsymbol{x} \boldsymbol{y}^T
=
\begin{pmatrix}
x_1 \\ x_2 \\ \vdots \\ x_n
\end{pmatrix}
(y_1, y_2, \ldots, y_n)
&=
\begin{pmatrix}
x_1 y_1 & x_1 y_2 & \cdots & x_1 y_n \\
x_2 y_1 & x_2 y_2 & \cdots & x_2 y_n \\
\vdots & \vdots & \ddots & \vdots \\
x_m y_1 & x_m y_2 & \cdots & x_m y_n
\end{pmatrix}
\in R^{m \times n}
\end{align*}

直積は，2つのベクトルから行列を作り出す計算で，$\boldsymbol{x}$の各要素と$\boldsymbol{y}$の各要素を掛け合わせて行列を作る．直積は外積（outer product）やテンソル積とも呼ばれることがあるが，3次元ベクトル解析における外積（cross product）とは異なるものである．3次元ベクトルの外積は，2つのベクトルに垂直なベクトルを返す操作である．



#### 1.6.2.3. <a id='toc1_6_2_3_'></a>[スカラーとの積](#toc0_)

行列$A \in R^{m \times n}$やベクトル$\boldsymbol{x} \in R^n$ととスカラー$c$の積，次のように計算される．

\begin{align*}
c A &=
\begin{pmatrix}
c a_{11} & c a_{12} & \cdots & c a_{1n} \\
c a_{21} & c a_{22} & \cdots & c a_{2n} \\
\vdots & \vdots & \ddots & \vdots \\
c a_{m1} & c a_{m2} & \cdots & c a_{mn}
\end{pmatrix}
\end{align*}

これは，行列$A$の各要素にスカラー$c$を掛けることで得られる行列であり，要素は次のように表せる．

\begin{align*}
[c A] &= [c a_{ij}]_{ij}
\end{align*}

ベクトルの場合，$c \boldsymbol{x}$の計算は以下の通りである．

\begin{align*}
c \boldsymbol{x}
&=
\begin{pmatrix}
c x_{1} \\
c x_{2} \\
\vdots \\
c x_{n}
\end{pmatrix}
\end{align*}

これは，ベクトル$\boldsymbol{x}$の各要素にスカラー$c$を掛けることで得られるベクトルであり，要素は次のように表される．

\begin{align*}
[c \boldsymbol{x}]_i = c x_i
\end{align*}




### 1.6.3. <a id='toc1_6_3_'></a>[行列積の計算量](#toc0_)

#### 1.6.3.1. <a id='toc1_6_3_1_'></a>[スカラーとの積](#toc0_)

スカラーとの積にかかる計算量は，その要素数に比例する．

- **$n$次元ベクトルとスカラーとの積**: ベクトルの$n$個の要素にスカラーをそれぞれ掛けるので，計算量は$n$ flops．
- **$m \times n$行列とスカラーとの積**: 行列の$mn$個の要素にスカラーをそれぞれ掛けるので，計算量は$mn$ flops．


#### 1.6.3.2. <a id='toc1_6_3_2_'></a>[2つの行列の積](#toc0_)

行列$A \in R^{m \times n}$と$B \in R^{n \times p}$の積$AB$を計算する際の計算量は$2mnp$ flopsである．

- $AB$の結果として得られる行列の各要素（合計で$mp$個）について，$n$次元ベクトルの内積を計算する必要がある．
- そのため，要素ごとに積和計算を行い，合計で$2n \times mp = 2mnp$ flopsになる．



#### 1.6.3.3. <a id='toc1_6_3_3_'></a>[3つの行列の積](#toc0_)

行列$A$と$B$に加えて行列$C \in R^{p \times q}$を掛け合わせて$ABC$を計算する場合，計算順序によって計算量が変わる．

- $(AB)C$の場合：
    - まず$AB$を計算する．この計算の結果$AB$は$m \times p$次元の行列になり，その計算量は$2mnp$ flopsである．
    - 次に，$AB$と$C$の積を計算する．この計算量は$2mpq$ flopsである．
    - 合計で$2mnp + 2mpq = 2mp(n + q)$ flopsになる．

- $A(BC)$の場合：
    - まず$BC$を計算する．この結果，$BC$は$n \times q$次元の行列となり，計算量は$2npq$ flopsである．
    - 次に，$A$と$BC$の積を計算する．この計算量は$2mnq$ flopsである．
    - 合計で$2npq + 2mnq = 2nq(p + m)$ flopsとなる．

このように，行列の積をどの順序で計算するかによって，計算量が異なる場合がある．


#### 1.6.3.4. <a id='toc1_6_3_4_'></a>[3つのベクトルの外積と内積](#toc0_)

この違いは，3つのベクトルの積の場合に顕著になる．

##### 1.6.3.4.1. <a id='toc1_6_3_4_1_'></a>[例1：$\boldsymbol{x} \boldsymbol{y}^T \boldsymbol{z}$](#toc0_)

ベクトル$\boldsymbol{x} \in R^m$，$\boldsymbol{y} \in R^n$，$\boldsymbol{z} \in R^n$に対して，次の積を考える．

\begin{align*}
\boldsymbol{x} \boldsymbol{y}^T \boldsymbol{z}
\end{align*}

計算順序によって計算量は次のように変わる．

- $(\boldsymbol{x} \boldsymbol{y}^T) \boldsymbol{z}$の場合：
    - 外積$\boldsymbol{x} \boldsymbol{y}^T$は$m \times n$次元の行列になるため，その計算量は$2mn$ flopsである．
    - 次に，行列ベクトル積$(\boldsymbol{x} \boldsymbol{y}^T) \boldsymbol{z}$の計算量も$2mn$ flopsである．
    - 合計で$4mn$ flopsとなる．

- $\boldsymbol{x} (\boldsymbol{y}^T \boldsymbol{z})$の場合：
    - まず$\boldsymbol{y}^T \boldsymbol{z}$は内積であり，計算量は$2n$ flopsである．
    - 次に，ベクトル$\boldsymbol{x}$と内積の結果（スカラー）との積の計算量は$m$ flopsである．
    - 合計で$2n + m$ flopsとなる．


##### 1.6.3.4.2. <a id='toc1_6_3_4_2_'></a>[例2：$(I + \boldsymbol{x} \boldsymbol{y}^T) \boldsymbol{z}$](#toc0_)


このような数式は，例えば$(I + \boldsymbol{x} \boldsymbol{y}^T) \boldsymbol{z}$のような形で頻繁に登場することがあるが，この式のまま実装することは計算量の観点から見て不利であり，特に$n$が大きい場合には行列がメモリに乗らない場合もある．そのため，この式を等価な形に変形することで効率的な計算が可能になる．

例えば，以下のように変形する．

$$
\boldsymbol{z} + \boldsymbol{x} (\boldsymbol{y}^T \boldsymbol{z})
$$

こうすると，計算は「ベクトルとスカラーの積」と「ベクトルの和」になるため，メモリの使用量を大幅に削減できる．


この数式変形によって，どの程度計算量が変わるのかを示す．

- 元の式$(I + \boldsymbol{x} \boldsymbol{y}^T) \boldsymbol{z}$の場合：
    - 外積$\boldsymbol{x} \boldsymbol{y}^T \in R^{m \times n}$の計算量は$2mn$ flopsである．
    - さらに，この結果とベクトル$\boldsymbol{z} \in R^n$との行列ベクトル積の計算量は$2mn$ flopsである．
    - したがって，合計で$4mn$ flopsとなる．

- 変形後の式$\boldsymbol{z} + \boldsymbol{x} (\boldsymbol{y}^T \boldsymbol{z})$の場合：
    - まず，$\boldsymbol{y}^T \boldsymbol{z}$は内積であり，計算量は$2n$ flopsである．
    - 次に，内積の結果はスカラー値なので，$\boldsymbol{x}$との積はベクトルとスカラーの積になり，その計算量は$m$ flopsで計算である．
    - 最後に，$\boldsymbol{z}$とのベクトル同士の和を計算する計算量は$m$ flopsで済む．
    - したがって，合計で$2n + m + m = 2(n + m)$ flopsとなる．

元の式の計算量と比較すると，式変形後は，$n$や$m$が大きい場合には計算量が大幅に削減されることがわかる．


したがって，数式が与えられた場合でも，そのまま実装するのではなく，効率を考えて変形して実装することが重要である．


### 1.6.4. <a id='toc1_6_4_'></a>[ndarrayの行列積](#toc0_)

`ndarray`のベクトルや行列の積を計算するためには，演算子`@`を使用する．以下のコードでは，行列ベクトル積や行列積を計算する例を示しており，行列積と転置の性質の確認も行っている．



In [ ]:
m = 5
n = 3
p = 2
A = rng.random(size=(m, n))
B = rng.random(size=(n, p))
x = rng.random(n)
y = rng.random(m)

print("A x\n", A @ x)
print("y^T A\n", y @ A)
print("y^T A\n", y.T @ A)
print("y^T A x\n", y @ A @ x)
print("AB\n", A @ B)
print("B^T A^T == (AB)^T\n", np.allclose(B.T @ A.T, (A @ B).T))


A x
 [0.8909598  0.95329544 1.17354078 1.08232817 1.59453318]
y^T A
 [1.05383811 0.84221739 1.61202859]
y^T A
 [1.05383811 0.84221739 1.61202859]
y^T A x
 2.634425457499619
AB
 [[0.59464872 0.83168972]
 [0.61115279 0.87394582]
 [0.91987572 1.15338397]
 [0.54227619 0.96097355]
 [0.95386188 1.42857525]]
B^T A^T == (AB)^T
 True


#### 1.6.4.1. <a id='toc1_6_4_1_'></a>[np.outer](#toc0_)


numpyには、1次元のndarray同士の直積を計算して2次元の`ndarray`を出力する`outer()`関数が実装されている．


- https://numpy.org/doc/stable/reference/generated/numpy.outer.html
    > Compute the outer product of two vectors.



In [ ]:
print("x y^T\n", np.outer(x, y))


x y^T
 [[0.02678632 0.19865464 0.86883537 0.96314632 0.17747762]
 [0.02742177 0.20336729 0.88944658 0.98599485 0.18168789]
 [0.01323991 0.09819074 0.42944676 0.47606265 0.0877234 ]]


#### 1.6.4.2. <a id='toc1_6_4_2_'></a>[reshape](#toc0_)

numpyの1次元ndarrayには，列ベクトルや行ベクトルの明確な区別がないため，通常の計算には問題は生じない．しかし，特殊な状況では，`reshape()`を用いて1次元の`ndarray`を$1 \times n$や$n \times 1$のように2次元`ndarray`に変換することで役に立つ場合がある．

- https://numpy.org/doc/stable/reference/generated/numpy.reshape.html
    > Gives a new shape to an array without changing its data.

`reshape()`の引数は`.reshape(new_rows, new_cols)`の順で指定し，それぞれ新しい行数と列数を示す．特に，片方の値を`-1`と指定すると，もう一方の値から自動的に適切な値が計算されるため便利である．

以下のコードは，1次元の`ndarray`を変形して2次元にする例である．

In [ ]:
n = 5
x = rng.random(n)
print("x", x, x.shape)

x = x.reshape(n, 1)
print("x\n", x, x.ndim, x.shape)

x = x.reshape(-1, 1)
print("x\n", x, x.ndim, x.shape)

x = x.reshape(1, -1)
print("x\n", x, x.ndim, x.shape)


x [0.51552153 0.46874983 0.16041454 0.63997201 0.78891116] (5,)
x
 [[0.51552153]
 [0.46874983]
 [0.16041454]
 [0.63997201]
 [0.78891116]] 2 (5, 1)
x
 [[0.51552153]
 [0.46874983]
 [0.16041454]
 [0.63997201]
 [0.78891116]] 2 (5, 1)
x
 [[0.51552153 0.46874983 0.16041454 0.63997201 0.78891116]] 2 (1, 5)


この`reshape()`を使えば，ベクトルの直積は以下のように実装できるが，1次元ndarray同士の直積を計算する場合には`np.outer()`を使う方がより効率的である．


In [ ]:
m = 5
n = 3
x = rng.random(m)
y = rng.random(n)
print("x", x, x.ndim, x.shape)
print("y", y, x.ndim, y.shape)

xp = x.reshape(-1, 1)
yp = y.reshape(-1, 1)
print("x\n", xp, xp.ndim, xp.shape)
print("y\n", yp, yp.ndim, yp.shape)

print("x y^T\n", xp @ yp.T)
print("x y^T\n", np.outer(x, y))


x [0.94658405 0.34168499 0.23898573 0.88791286 0.20647014] 1 (5,)
y [0.10267682 0.32488044 0.12769157] 1 (3,)
x
 [[0.94658405]
 [0.34168499]
 [0.23898573]
 [0.88791286]
 [0.20647014]] 2 (5, 1)
y
 [[0.10267682]
 [0.32488044]
 [0.12769157]] 2 (3, 1)
x y^T
 [[0.09719224 0.30752664 0.1208708 ]
 [0.03508313 0.11100677 0.04363029]
 [0.0245383  0.07764179 0.03051646]
 [0.09116807 0.28846552 0.11337898]
 [0.0211997  0.06707811 0.0263645 ]]
x y^T
 [[0.09719224 0.30752664 0.1208708 ]
 [0.03508313 0.11100677 0.04363029]
 [0.0245383  0.07764179 0.03051646]
 [0.09116807 0.28846552 0.11337898]
 [0.0211997  0.06707811 0.0263645 ]]


内積の計算も次のように`reshape()`を用いてから行列積を計算することでも実装できる．しかしこの場合，結果がスカラーではなく$1 \times 1$の2次元ndarrayとして得られるため，その要素をfloat型として取り出すには`.item()`を使う必要がある．
内積を計算する場合には，結果がスカラーとして取得できる`@`演算子を用いるほうがよい．

In [ ]:
n = 3
x = rng.random(n)
y = rng.random(n)
print("x", x, x.ndim, x.shape)
print("y", y, y.ndim, y.shape)

xp = x.reshape(-1, 1)
yp = y.reshape(-1, 1)
print("x\n", xp, xp.ndim, xp.shape)
print("y\n", yp, yp.ndim, yp.shape)

xy1 = xp.T @ yp
print("x^T y\n", xy1, type(xy1), xy1.ndim, xy1.shape)
xy1 = xp.T @ yp
print("-->", xy1.item())

xy2 = x @ y
print("x^T y\n", xy2, type(xy2))


x [0.57758109 0.19407806 0.87332079] 1 (3,)
y [0.79211902 0.1075264  0.0434244 ] 1 (3,)
x
 [[0.57758109]
 [0.19407806]
 [0.87332079]] 2 (3, 1)
y
 [[0.79211902]
 [0.1075264 ]
 [0.0434244 ]] 2 (3, 1)
x^T y
 [[0.51630492]] <class 'numpy.ndarray'> 2 (1, 1)
--> 0.516304916390891
x^T y
 0.516304916390891 <class 'numpy.float64'>


##### 1.6.4.2.1. <a id='toc1_6_4_2_1_'></a>[reshapeの注意点](#toc0_)

`reshape()`を使っても，実際にはオブジェクトのコピーは作成されず，元の配列と同じメモリ領域を共有する．そのため，`reshape()`によって「作成」された新しい変数の要素を変更すると，元の変数の要素も書き換わってしまう．この現象は意図しない動作を引き起こす可能性があり，バグの原因となりやすいため，注意が必要である．対策としては，`.copy()`を使って新しいメモリ領域を確保する方法がある．

以下のコード例は，`reshape()`を使用した際のこの動作を示している．


In [ ]:
m = 5
n = 3
x = rng.random(m)
y = rng.random(n)
print("x", x, x.shape)
print("y", y, y.shape)

xp = x.reshape(-1, 1)
yp = y.reshape(-1, 1)
print("xp\n", xp, xp.shape)
print("yp\n", yp, yp.shape)

# reshape後に要素を変更
xp[0] = 0.1
yp[-1] = 0.25
print("x", x, x.shape)
print("y", y, y.shape)
print("xp", xp, xp.shape)
print("yp", yp, yp.shape)


x [0.77601574 0.87697947 0.89021263 0.17811055 0.52285944] (5,)
y [0.90296599 0.95117876 0.3743184 ] (3,)
xp
 [[0.77601574]
 [0.87697947]
 [0.89021263]
 [0.17811055]
 [0.52285944]] (5, 1)
yp
 [[0.90296599]
 [0.95117876]
 [0.3743184 ]] (3, 1)
x [0.1        0.87697947 0.89021263 0.17811055 0.52285944] (5,)
y [0.90296599 0.95117876 0.25      ] (3,)
xp [[0.1       ]
 [0.87697947]
 [0.89021263]
 [0.17811055]
 [0.52285944]] (5, 1)
yp [[0.90296599]
 [0.95117876]
 [0.25      ]] (3, 1)


このコードでは，`reshape()`後に`xp`や`yp`の要素を変更すると，元の変数`x`や`y`の要素も変更されてしまうことがわかる．

この問題を避けるには，`copy()`を使って新しい配列を作成する．以下の例では，`copy()`を使って独立なメモリ領域を確保し，同様の問題が発生しないようにしている．


In [ ]:
m = 5
n = 3
x = rng.random(m)
y = rng.random(n)
print("x", x, x.shape)
print("y", y, y.shape)

xp = x.copy().reshape(-1, 1)
yp = y.copy().reshape(-1, 1)
print("xp\n", xp, xp.shape)
print("yp\n", yp, yp.shape)

# copy後に要素を変更
xp[0] = 0.1
yp[-1] = 0.25
print("x", x, x.shape)
print("y", y, y.shape)
print("xp", xp, xp.shape)
print("yp", yp, yp.shape)


x [0.00491705 0.59381732 0.1118756  0.44103483 0.80477648] (5,)
y [0.69532296 0.8755819  0.2304022 ] (3,)
xp
 [[0.00491705]
 [0.59381732]
 [0.1118756 ]
 [0.44103483]
 [0.80477648]] (5, 1)
yp
 [[0.69532296]
 [0.8755819 ]
 [0.2304022 ]] (3, 1)
x [0.00491705 0.59381732 0.1118756  0.44103483 0.80477648] (5,)
y [0.69532296 0.8755819  0.2304022 ] (3,)
xp [[0.1       ]
 [0.59381732]
 [0.1118756 ]
 [0.44103483]
 [0.80477648]] (5, 1)
yp [[0.69532296]
 [0.8755819 ]
 [0.25      ]] (3, 1)


このコードでは，`xp`や`yp`の要素を変更しても，元の`x`や`y`には影響がないことが確認できる．このように，`reshape()`を使用する際には，元のデータを変更しないように`.copy()`を併用することが必要になる場合がある．

## 1.7. <a id='toc1_7_'></a>[ndarray 特有の演算](#toc0_)

線形代数では定義されていないが，プログラミング上で便利なためにndarrayに対して用意されている演算がnumpyにはいくつか存在する．これらの演算は，特にデータ処理や数値計算を効率的に行う際に役立つ．

### 1.7.1. <a id='toc1_7_1_'></a>[ndarrayとスカラーとの積](#toc0_)

numpyでは，ベクトルや行列（つまりndarray）とスカラー`c`の積は，演算子`*`を使用する．これは数学で定義されている通り，スカラーをベクトルや行列の各要素に掛ける操作であり，次の例に示すようにどちらの順序で計算しても結果は同じになる。


In [ ]:
m = 5
n = 3
A = rng.random(size=(m, n))
x = rng.random(n)
c = rng.random()

print("c", c)
print("A\n", A)
print("cA\n", c * A)  # 行列とスカラーの積
print("cA\n", A * c)  # スカラーと行列の積（順序は関係ない）
print("x", x)
print("cx", c * x)  # ベクトルとスカラーの積
print("cx", x * c)  # スカラーとベクトルの積（順序は関係ない）


c 0.4657075559795171
A
 [[0.00225253 0.6401992  0.51033613]
 [0.08638189 0.39407623 0.67460978]
 [0.90004952 0.12221702 0.19406495]
 [0.87281469 0.12372757 0.75811663]
 [0.48713855 0.50247878 0.37143827]]
cA
 [[0.00104902 0.29814561 0.23766739]
 [0.0402287  0.18352428 0.31417087]
 [0.41915986 0.05691739 0.09037751]
 [0.4064764  0.05762087 0.35306064]
 [0.2268641  0.23400816 0.17298161]]
cA
 [[0.00104902 0.29814561 0.23766739]
 [0.0402287  0.18352428 0.31417087]
 [0.41915986 0.05691739 0.09037751]
 [0.4064764  0.05762087 0.35306064]
 [0.2268641  0.23400816 0.17298161]]
x [0.09860315 0.68045409 0.00195306]
cx [0.04592023 0.31689261 0.00090956]
cx [0.04592023 0.31689261 0.00090956]


### 1.7.2. <a id='toc1_7_2_'></a>[ndarrayとスカラーとの和](#toc0_)

numpyでは，ベクトルや行列（ndarray）とスカラーの和を計算するために演算子`+`が使用できる．これは各要素にスカラーを加算する操作であり，数学的には定義されていないが．プログラミングにおいて非常に便利である．


In [ ]:
m = 5
n = 3
A = rng.random(size=(m, n))
x = rng.random(n)
c = 10.0

print("c", c)
print("A\n", A)
print("cA\n", c * A)
print("c + A\n", c + A)

print("c", c)
print("x", x)
print("c * x", c * x)
print("c + x", c + x)


c 10.0
A
 [[0.92026964 0.44680074 0.06729407]
 [0.35163254 0.8233379  0.61826725]
 [0.76489691 0.84352232 0.88535756]
 [0.55971112 0.38402812 0.32176763]
 [0.92862434 0.41057329 0.05907371]]
cA
 [[9.20269638 4.4680074  0.67294067]
 [3.51632542 8.23337899 6.18267255]
 [7.6489691  8.43522323 8.8535756 ]
 [5.59711116 3.84028122 3.21767626]
 [9.28624341 4.10573291 0.5907371 ]]
c + A
 [[10.92026964 10.44680074 10.06729407]
 [10.35163254 10.8233379  10.61826725]
 [10.76489691 10.84352232 10.88535756]
 [10.55971112 10.38402812 10.32176763]
 [10.92862434 10.41057329 10.05907371]]
c 10.0
x [0.09645637 0.06974087 0.79674704]
c * x [0.96456365 0.69740865 7.96747036]
c + x [10.09645637 10.06974087 10.79674704]



### 1.7.3. <a id='toc1_7_3_'></a>[要素毎の積](#toc0_)

これまで説明してきた通り，numpyにおいてベクトルや行列の積を行うための演算子は`@`であるが，演算子`*`は要素毎の積に使用できる．これは，対応する要素ごとに乗算を行う操作であり，初級レベルの線形代数では登場しないが，数学的にはアダマール積（Hadamard product）や要素積と呼ばれている．この演算は，同じサイズの行列やベクトルに対して定義されている．

アダマール積の記号としては，$\boldsymbol{x} \circ \boldsymbol{y}$や$\boldsymbol{x} \otimes \boldsymbol{y}$，$\boldsymbol{x} \odot \boldsymbol{y}$などが使用される．

- https://ja.wikipedia.org/wiki/%E3%82%A2%E3%83%80%E3%83%9E%E3%83%BC%E3%83%AB%E7%A9%8D
    > 数学におけるアダマール積は、同じサイズの行列に対して成分ごとに積を取ることによって定まる行列の積である。要素ごとの積、シューア積、点ごとの積、成分ごとの積などとも呼ばれる。

In [ ]:
m = 5
n = 3
A = rng.random(size=(m, n))
B = rng.random(size=(m, n))
x = rng.random(n)
y = rng.random(n)

print("A\n", A)
print("B\n", B)
print("A + B\n", A + B)
print("A * B\n", A * B)


print("x", x)
print("y", y)
print("x + y", x + y)
print("x * y", x * y)


A
 [[0.00368606 0.72249152 0.72087576]
 [0.41716329 0.62195992 0.85659639]
 [0.55710123 0.75544362 0.79062193]
 [0.47217107 0.64978494 0.63627239]
 [0.0650262  0.39694188 0.62359485]]
B
 [[0.07062805 0.27594398 0.1678174 ]
 [0.57790384 0.45750015 0.4162645 ]
 [0.05435944 0.00827952 0.41174207]
 [0.26506607 0.54241313 0.16759928]
 [0.87649816 0.41499697 0.9674786 ]]
A + B
 [[0.07431411 0.9984355  0.88869316]
 [0.99506713 1.07946007 1.27286089]
 [0.61146067 0.76372315 1.202364  ]
 [0.73723714 1.19219807 0.80387168]
 [0.94152436 0.81193885 1.59107345]]
A * B
 [[2.60339345e-04 1.99367184e-01 1.20975498e-01]
 [2.41080267e-01 2.84546755e-01 3.56570669e-01]
 [3.02837100e-02 6.25471396e-03 3.25532310e-01]
 [1.25156530e-01 3.52451882e-01 1.06638796e-01]
 [5.69953475e-02 1.64729677e-01 6.03314672e-01]]
x [0.63416148 0.26877963 0.83406374]
y [0.78034245 0.87111957 0.35165699]
x + y [1.41450393 1.1398992  1.18572073]
x * y [0.49486312 0.23413919 0.29330434]


## 1.8. <a id='toc1_8_'></a>[ブロードキャスト](#toc0_)

数学的には，行列とベクトルの和や，次元の合わない行列同士の積は定義されていない．しかし，プログラミング的にはこれらの操作が非常に便利な場合がある．numpyにおいては，このような異なる次元の配列同士の演算を可能にする機能は「ブロードキャスト（broadcasting）」と呼ばれている．前述のスカラーとの和も，ブロードキャストの一種とみなすことができる．

### 1.8.1. <a id='toc1_8_1_'></a>[行列の各行とベクトルの和](#toc0_)

$m \times n$の行列`A`に対して，$n$次元ベクトル`x`を「右から」加算する場合，numpyのブロードキャスト機能を使うことで，`A + x`という形式で和を計算できる．この操作の動作は次の通りである．

- 行列`A`の最後の次元（列数$n$）とベクトル`x`の次元（長さ$n$）が一致している必要がある．
- ベクトル`x`は，`A`の行数$m$に対応して引き伸ばされ，`A`の各行に対して同じベクトル`x`が足される．
- その結果，$m \times n$の行列に対して，ベクトル`x`が各行に加算される．

ブロードキャストは，右側から左側へと次元が引き伸ばされて実行されるため，演算は可換ではない．つまり，単純に左右を入れ替えて計算することはできない点に注意が必要である．

その他の詳細な説明については，numpyの公式マニュアルを参照のこと．

- https://numpy.org/doc/stable/user/basics.broadcasting.html
    > The term broadcasting describes how NumPy treats arrays with different shapes during arithmetic operations.


次の例は，$A \in R^{m \times n}$の各行にベクトル$x \in R^n$を加算し，また行列$A$の転置$A^T \in R^{n \times m}$の各行にベクトル$y \in R^m$を加算する（したがって$A$の各列に加算する）例を示している．


In [ ]:
m = 5
n = 3
A = rng.random(size=(m, n))
x = np.full(n, 2.0)  # 全ての要素が2.0のベクトルx
y = np.full(m, 3.0)  # 全ての要素が3.0のベクトルy

print("x", x)
print("y", y)

print("A\n", A)
print("A + x\n", A + x)  # Aの各行にベクトルxを加算
print("A.T + y\n", A.T + y)  # Aの各列にベクトルyを加算


x [2. 2. 2.]
y [3. 3. 3. 3. 3.]
A
 [[0.07411572 0.05560082 0.3878194 ]
 [0.25469043 0.98788697 0.13142613]
 [0.97967224 0.0642711  0.40841284]
 [0.0850801  0.09780983 0.05761218]
 [0.32184085 0.45871914 0.82880807]]
A + x
 [[2.07411572 2.05560082 2.3878194 ]
 [2.25469043 2.98788697 2.13142613]
 [2.97967224 2.0642711  2.40841284]
 [2.0850801  2.09780983 2.05761218]
 [2.32184085 2.45871914 2.82880807]]
A.T + y
 [[3.07411572 3.25469043 3.97967224 3.0850801  3.32184085]
 [3.05560082 3.98788697 3.0642711  3.09780983 3.45871914]
 [3.3878194  3.13142613 3.40841284 3.05761218 3.82880807]]


### 1.8.2. <a id='toc1_8_2_'></a>[ブロードキャストの例：中心化](#toc0_)

行列の行や列の平均ベクトルを計算し，それぞれの行や列から平均を引く中心化処理（centering）は，データの前処理や主成分分析（PCA）などで頻繁に用いられる．

#### 1.8.2.1. <a id='toc1_8_2_1_'></a>[数式](#toc0_)

行列$A \in R^{m \times n}$の平均行ベクトル$\boldsymbol{m}_{\mathrm{row}} \in R^n$と平均列ベクトル$\boldsymbol{m}_{\mathrm{col}} \in R^m$は，次のように定義される．

\begin{align*}
\boldsymbol{m}_{\mathrm{row}} &= \frac{1}{m} \boldsymbol{1}_m^T A \in R^{1 \times n} \\
\boldsymbol{m}_{\mathrm{col}} &= \frac{1}{n} A \boldsymbol{1}_n \in R^{m \times 1}
\end{align*}

ここで$\boldsymbol{1}_n$は要素が全て1の$n$次元ベクトルである．
行列$A$の各行から平均行ベクトルを引いた結果$A'_{\mathrm{row}}$と，各列から平均列ベクトルを引いた結果$A'_{\mathrm{col}}$は，次のように表される．

\begin{align*}
A'_{\mathrm{row}}
&= A - \boldsymbol{1}_m \boldsymbol{m}_{\mathrm{row}}
= A - \frac{1}{m} \boldsymbol{1}_m \boldsymbol{1}_m^T A
= (I - \frac{1}{m} \boldsymbol{1}_m \boldsymbol{1}_m^T) A
\\
A'_{\mathrm{cols}}
&= A - \boldsymbol{m}_{\mathrm{col}} \boldsymbol{1}_n^T
= A - \frac{1}{n} A \boldsymbol{1}_n \boldsymbol{1}_n^T
= A (I - \frac{1}{n} \boldsymbol{1}_n \boldsymbol{1}_n^T)
\end{align*}
ここで$\boldsymbol{1}_n \boldsymbol{1}_n^T \in R^{n \times n}$は要素が全て1の$n \times n$行列，$I$は$n \times n$単位行列である．

理論的な解析ではこれらの数式を使うが，これをそのまま実装すると計算量が大きくなり，効率的ではない．計算量の観点から，数式を簡素化することが重要である．

例えば，$A'_{\mathrm{cols}}= A (I - \frac{1}{n} \boldsymbol{1}_n \boldsymbol{1}_n^T)$の計算を行うと，次のような計算量が必要になる．

- $\boldsymbol{1}_n \boldsymbol{1}_n^T$（全て1の行列）の積に$2n^2$ flops
- $\frac{1}{n}$とのスカラー積に$n^2$ flops
- 単位行列$I$との減算に$n^2$ flops
- 行列$A$との積に$2mn^2$ flops
- 合計で$4 n^2 + 2 mn^2$ flops

この計算量はオーダーが$O(n^2 + m n^2)$という非常に大きなものであり，特に次元$n$が大きい場合には非効率的になる．

一方，数式として計算量の少ない$A'_{\mathrm{cols}}= A - \boldsymbol{m}_{\mathrm{col}} \boldsymbol{1}_n^T$の方を用いると，以下のような計算量に削減できる．

- 平均列ベクトル$\boldsymbol{m}_{\mathrm{col}}$の計算
  - 行方向の総和$A \boldsymbol{1}_n$の計算に$2mn$ flops
  - その結果と$\frac{1}{n}$とのスカラー積の計算に$m$ flops
- $\boldsymbol{m}_{\mathrm{col}} \boldsymbol{1}_n^T$の計算に$2mn$ flops
- $A$から減算するために$mn$ flops
- 合計で$5mn + m$ flops

このように，計算量は大幅に削減されるが，まだ改善の余地がある．この式においても，$\boldsymbol{1}_n$は全ての要素が1であり，本来1との積は計算不要であるにもかかわらず，定義どおりの積計算を行ってしまっている．このため，無駄な計算が残っている．

したがって，数式をそのまま実装するのではなく，特にブロードキャストなどの効率的な手法を使って不要な計算を省くことが重要である．


#### 1.8.2.2. <a id='toc1_8_2_2_'></a>[ブロードキャスト](#toc0_)

ブロードキャストを使えば，計算量を大幅に削減でき，簡潔なコードを書くことができる．次の例では，`.mean()`を使って行列の平均ベクトルを計算し，ブロードキャストを用いて各行や各列から平均を引いている．

- https://numpy.org/doc/stable/reference/generated/numpy.ndarray.mean.html
    > Returns the average of the array elements along given axis.


In [ ]:
m = 5
n = 3
A = rng.random(size=(m, n))
print("A\n", A)
print()

# 平均行ベクトルを求める
mean_row = A.mean(axis=0)
A_row = A - mean_row
print("mean row vector of A", mean_row)
print("A_row\n", A_row)
print("mean row vector of A_row", A_row.mean(axis=0))

print()

# 平均列ベクトルを求める
mean_col = A.mean(axis=1)
A_col = (A.T - mean_col).T  # 転置して計算し，元に戻す
print("mean column vector of A", mean_col)
print("A_col\n", A_col)
print("mean column vector of A_col", A_col.mean(axis=1))


A
 [[0.67615612 0.8943671  0.94075369]
 [0.77832441 0.20348756 0.97946012]
 [0.04363003 0.14065932 0.92419631]
 [0.9894937  0.54479969 0.59890299]
 [0.24442602 0.81889839 0.42540142]]

mean row vector of A [0.54640606 0.52044241 0.77374291]
A_row
 [[ 0.12975006  0.37392469  0.16701078]
 [ 0.23191835 -0.31695485  0.20571721]
 [-0.50277602 -0.37978309  0.1504534 ]
 [ 0.44308765  0.02435727 -0.17483991]
 [-0.30198004  0.29845598 -0.34834149]]
mean row vector of A_row [ 4.44089210e-17 -2.22044605e-17 -2.22044605e-17]

mean column vector of A [0.8370923  0.65375736 0.36949522 0.71106546 0.49624194]
A_col
 [[-0.16093619  0.0572748   0.10366139]
 [ 0.12456704 -0.4502698   0.32570276]
 [-0.32586519 -0.2288359   0.55470109]
 [ 0.27842824 -0.16626578 -0.11216247]
 [-0.25181592  0.32265645 -0.07084053]]
mean column vector of A_col [ 3.70074342e-17  3.70074342e-17 -7.40148683e-17  3.70074342e-17
 -3.70074342e-17]


このブロードキャストを用いた場合，$A'_{\mathrm{cols}}$の計算量は以下のようになる．

- 平均列ベクトル$\boldsymbol{m}_{\mathrm{col}}$の計算
    - `.mean()`による行方向の積和計算に$2mn$ flops
- $A$の各列とのブロードキャストによる減算に$mn$ flops
- 合計で$3mn$ flops

このように，効率的かつ簡潔なコードが実現できる．


#### 1.8.2.3. <a id='toc1_8_2_3_'></a>[非効率な実装例](#toc0_)

以下は，定義式に従った実装の例だが，コードが読みにくく，計算量も多いため推奨されない．
なおこの例では，`np.outer(np.ones(m), np.ones(m))`の部分を`np.ones((m, m))`で代替することも可能だが，ブロードキャストを使う方が簡潔で計算量も少ない．


In [ ]:
A_row = (np.eye(m) - np.outer(np.ones(m), np.ones(m)) / m) @ A
print("A_row\n", A_row)

A_col = A @ (np.eye(n) - np.outer(np.ones(n), np.ones(n)) / n)
print("A_col\n", A_col)


A_row
 [[ 0.12975006  0.37392469  0.16701078]
 [ 0.23191835 -0.31695485  0.20571721]
 [-0.50277602 -0.37978309  0.1504534 ]
 [ 0.44308765  0.02435727 -0.17483991]
 [-0.30198004  0.29845598 -0.34834149]]
A_col
 [[-0.16093619  0.0572748   0.10366139]
 [ 0.12456704 -0.4502698   0.32570276]
 [-0.32586519 -0.2288359   0.55470109]
 [ 0.27842824 -0.16626578 -0.11216247]
 [-0.25181592  0.32265645 -0.07084053]]


### 1.8.3. <a id='toc1_8_3_'></a>[ブロードキャストの実際の例：共分散行列](#toc0_)

共分散行列の計算は，ブロードキャストによる中心化処理の実際の例である．ここでは，よく使われるアヤメ（iris）データセットを用いたデータ処理の流れを見ていく．まず，データの格納方法について整理する．


#### 1.8.3.1. <a id='toc1_8_3_1_'></a>[行と列の方向](#toc0_)

行列$A \in R^{m \times n}$とベクトル$\boldsymbol{x} \in R^n$の積$A \boldsymbol{x} \in R^m$を計算する場合，$\boldsymbol{x}$の各要素を重みとして，$A$の各列$\boldsymbol{a}_i \in R^m$の線形和を計算していることに相当する．


\begin{align*}
A \boldsymbol{x}
&= (\boldsymbol{a}_1, \boldsymbol{a}_2, \ldots, \boldsymbol{a}_n) \boldsymbol{x} \\
&= x_1 \boldsymbol{a}_1 + x_2 \boldsymbol{a}_2 + \ldots + x_n \boldsymbol{a}_n
\end{align*}

機械学習の分野では，$A$の各列$\boldsymbol{a}_i$が1つのサンプル（データ）に対応することが多い．この場合，$A$は$m$次元ベクトルのデータが$n$個あるデータセットを表す．つまり，$A \boldsymbol{x}$は各データに対する重み付け和を計算していることになる．このような行列$A$をデータ行列やデザイン行列と呼ぶことがある．

一方，ndarrayとしての行列`A`は，各行が1次元ndarrayであり，その1次元ndarrayが縦方向に積み重なっている．これはPythonの2次元リストと同様の構造になっている．


In [ ]:
m = 5
n = 3
A = rng.random(size=(m, n))

print(A)


[[0.0633005  0.22798933 0.24869848]
 [0.50957291 0.02076688 0.50648309]
 [0.64629033 0.63943285 0.73834302]
 [0.66963203 0.07316242 0.00859063]
 [0.86309379 0.89789045 0.75283382]]


これは行方向の要素がメモリ上で連続（contiguous）に配置されているためである．この方式は「row-major」と呼ばれ，C言語などの一般的なプログラミング言語で採用されている．そのため，`np.array()`のメモリ配列方向を表す引数`order`のデフォルトは`"C"`という1文字が使われており，これはrow-majorを表している．

一方，列方向に連続して配置された方式を「column-major」と呼び，Fortranなどで採用されていた（現在ではあまり一般的ではない）．Fortran以外では，MATLABやOctaveが採用している．`np.array()`の引数`order`でcolumn-majorを指定するには，Fortranの1文字`"F"`を使う．

- https://numpy.org/doc/stable/glossary.html#term-contiguous
    > An array is contiguous if: it occupies an unbroken block of memory, and array elements with higher indexes occupy higher addresses (that is, no stride is negative).
- https://numpy.org/doc/stable/glossary.html#term-row-major
    > NumPy creates arrays in row-major order by default.
- https://numpy.org/doc/stable/glossary.html#term-column-major
- https://en.wikipedia.org/wiki/Row-_and_column-major_order
    > In computing, row-major order and column-major order are methods for storing multidimensional arrays in linear storage such as random access memory.

Pythonでは，実際の内部メモリ配置に関係なく，`print()`で表示される内容や要素へのアクセスはこれまで説明した通りである．`A[0]`と書けば，メモリ配置に関係なく，先頭の0行目が得られる．

コードとしては，データセット中のサンプル$m$次元ベクトルの扱いに関係する．数式上は**各サンプルを列ベクトルとして扱う行列$A$** を構成し，$A \boldsymbol{x}$を計算するが，実際のコードでは，**各サンプルを行ベクトルに持つndarray `A`** を使って`x @ A`とすることもある．この場合，`A`に転置は不要である．数式的には等価であるが，実装コードの見た目が数式と反対になるため，バグが発生しやすいので注意が必要である．

以下は，アヤメのデータセットを`A`に格納した例である．このデータセットには$n=4$次元のデータが$m=150$個ある．なお，`A`を転置したものを`B`としている．


In [ ]:
iris_data = load_iris()

A = iris_data["data"]
m, n = A.shape
print(f"m={m}, n={n}")
print("first 10 samples\n", A[:10])
print("A^T A\n", A.T @ A)

B = A.T
m, n = B.shape
print(f"m={m}, n={n}")
print("first 10 samples\n", B[:, :10])
print("B B^T\n", B @ B.T)


m=150, n=4
first 10 samples
 [[5.1 3.5 1.4 0.2]
 [4.9 3.  1.4 0.2]
 [4.7 3.2 1.3 0.2]
 [4.6 3.1 1.5 0.2]
 [5.  3.6 1.4 0.2]
 [5.4 3.9 1.7 0.4]
 [4.6 3.4 1.4 0.3]
 [5.  3.4 1.5 0.2]
 [4.4 2.9 1.4 0.2]
 [4.9 3.1 1.5 0.1]]
A^T A
 [[5223.85 2673.43 3483.76 1128.14]
 [2673.43 1430.4  1674.3   531.89]
 [3483.76 1674.3  2582.71  869.11]
 [1128.14  531.89  869.11  302.33]]
m=4, n=150
first 10 samples
 [[5.1 4.9 4.7 4.6 5.  5.4 4.6 5.  4.4 4.9]
 [3.5 3.  3.2 3.1 3.6 3.9 3.4 3.4 2.9 3.1]
 [1.4 1.4 1.3 1.5 1.4 1.7 1.4 1.5 1.4 1.5]
 [0.2 0.2 0.2 0.2 0.2 0.4 0.3 0.2 0.2 0.1]]
B B^T
 [[5223.85 2673.43 3483.76 1128.14]
 [2673.43 1430.4  1674.3   531.89]
 [3483.76 1674.3  2582.71  869.11]
 [1128.14  531.89  869.11  302.33]]


データ行列として`A`を用いるか`B`を用いるかによって，数式上は転置が異なることに注意する必要がある．ここでは$A^T = B$であるので，`A`を用いた場合の式$A^T \boldsymbol{x}$と，`B`を用いた場合の式$B \boldsymbol{x}$は同じ意味であり，$A^T A$と$B B^T$は同じものである．

#### 1.8.3.2. <a id='toc1_8_3_2_'></a>[共分散行列の定義](#toc0_)

上記の話を踏まえて，データから共分散行列を計算する方法を見ていく．

各行にサンプルを持つndarrayと同様に，
$m$個の$n$次元ベクトル$\boldsymbol{x}_i \in R^n$を**各行**に持つ
次のデータ行列$X$を考える．

$$
X
= \begin{pmatrix}\boldsymbol{x}_1^T \\ \boldsymbol{x}_2^T \\ \vdots \\ \boldsymbol{x}_m^T \end{pmatrix}
= (\boldsymbol{x}_1, \boldsymbol{x}_2, \ldots, \boldsymbol{x}_m)^T
\in R^{m \times n}
$$

このデータの平均ベクトル$\bar{\boldsymbol{x}} \in R^n$は次のように表される．

$$
\bar{\boldsymbol{x}} = \frac{1}{m} \sum_{i=1}^m \boldsymbol{x}_i = (\bar{x}_1, \bar{x}_2, \ldots, \bar{x}_n)^T
$$


このデータの共分散行列（covariance matrix）$S$は，次のような成分を持つ$n \times n$行列で表される．

- 第$i$対角成分：第$i$成分の分散$\sigma_{i}^2$
- 非対角成分：第$i$成分と第$j$成分の共分散$\sigma_{ij}$

共分散行列の成分は次式で定義される．

\begin{align*}
\sigma_i^2
&= \frac{1}{m} \sum_{i=1}^m (x_i - \bar{x}_i)^2 \\
\sigma_{ij}
&= \frac{1}{m} \sum_{i=1}^m (x_i - \bar{x}_i) (x_j - \bar{x}_j)
\end{align*}

これらの要素を行列の形式で表すと，次のようになる．

\begin{align*}
S =
\begin{pmatrix}
\sigma_1^2 & \sigma_{21} & \cdots & \sigma_{n1} \\
\sigma_{12} & \sigma_2^2 & \cdots & \sigma_{n2} \\
\vdots & & \ddots \\
\sigma_{1n} & \sigma_{2n} & \cdots & \sigma_n^2 \\
\end{pmatrix}
\end{align*}

この共分散行列は，平均ベクトル$\bar{\boldsymbol{x}}$と各データ$\boldsymbol{x}_i$を用いて，次のように表すことができる．

$$
S = \frac{1}{m} \sum_{i=1}^m (\boldsymbol{x}_i - \bar{\boldsymbol{x}})(\boldsymbol{x}_i - \bar{\boldsymbol{x}})^T
$$

なお，平均ベクトル$\bar{\boldsymbol{x}}$を$X$の各行から除去した行列$X'_{\mathrm{row}}
= X - \boldsymbol{1}_m \bar{\boldsymbol{x}}^T
$を用いると，共分散行列は次のように簡潔に書ける．

$$
S = \frac{1}{m} (X'_{\mathrm{row}})^T X'_{\mathrm{row}}
$$

numpyでは，共分散行列を計算するために`np.cov()`という関数が用意されている．この関数の引数に与えるデータ行列は，**各列**が1つサンプルを表し，行はデータの次元（変数）を表している，と仮定している．この点に注意してデータを与える必要がある．

- https://numpy.org/doc/stable/reference/generated/numpy.cov.html
    > Estimate a covariance matrix, given data and weights.


#### 1.8.3.3. <a id='toc1_8_3_3_'></a>[アヤメのデータセット](#toc0_)

実際のデータの共分散行列を求めるために，広く知られているアヤメのデータセット（the iris dataset）を使用する．

- https://en.wikipedia.org/wiki/Iris_flower_data_set

このデータセットは，3種類のアヤメ（Iris setosa，Iris versicolor，Iris virginica）の4つの特徴量（変数）で構成されている．
特徴量は，sepal（がく片）とpetal（花弁）のそれぞれの長さと幅である．以下の写真に示すように，アヤメにはsepalとpetalがそれぞれ3枚ずつあり，データセットはその長さと幅を計測したものである．



| class   | Iris setosa             | Iris versicolor                   | Iris virginica |
|---------|-------------------------|-----------------------------------|----------------|
| image   | <img src="https://upload.wikimedia.org/wikipedia/commons/5/56/Kosaciec_szczecinkowaty_Iris_setosa.jpg" width=150>| <img src="https://upload.wikimedia.org/wikipedia/commons/4/41/Iris_versicolor_3.jpg" width=150>| <img src="https://upload.wikimedia.org/wikipedia/commons/thumb/9/9f/Iris_virginica.jpg/736px-Iris_virginica.jpg" width=150>|
| license | [CC BY-SA 3.0 by Radomil](https://commons.wikimedia.org/wiki/File:Kosaciec_szczecinkowaty_Iris_setosa.jpg) | [CC BY-SA 3.0 by Danielle Langlois](https://commons.wikimedia.org/wiki/File:Iris_versicolor_3.jpg) | [CC BY-SA 2.0 by Frank Mayfield](https://commons.wikimedia.org/wiki/File:Iris_virginica.jpg)   |

次に，アヤメのデータセットをscikit-learnから取得し，データ行列`X`に格納するコードを示す．
このデータは$n=4$次元のデータが$m=150$個で構成されており，データ行列の各行が1つのサンプルデータを表している．したがって，データ行列を転置してから，共分散行列を求める関数`np.cov()`に与えている．

- https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_iris.html
    > The iris dataset is a classic and very easy multi-class classification dataset.


In [ ]:
iris_data = load_iris()

X = iris_data["data"]
m, n = X.shape
print(f"m={m}, n={n}")
print("first 10 samples\n", X[:10])

S = np.cov(X.T)
print("S\n", S)


m=150, n=4
first 10 samples
 [[5.1 3.5 1.4 0.2]
 [4.9 3.  1.4 0.2]
 [4.7 3.2 1.3 0.2]
 [4.6 3.1 1.5 0.2]
 [5.  3.6 1.4 0.2]
 [5.4 3.9 1.7 0.4]
 [4.6 3.4 1.4 0.3]
 [5.  3.4 1.5 0.2]
 [4.4 2.9 1.4 0.2]
 [4.9 3.1 1.5 0.1]]
S
 [[ 0.68569351 -0.042434    1.27431544  0.51627069]
 [-0.042434    0.18997942 -0.32965638 -0.12163937]
 [ 1.27431544 -0.32965638  3.11627785  1.2956094 ]
 [ 0.51627069 -0.12163937  1.2956094   0.58100626]]


なお`np.cov()`の引数`rowvar`を`False`に設定すると，データ行列の各列が変数で各行がサンプルであるという解釈になる．
このオプションを使用することで，`X`を転置することなく共分散行列を計算できる．


In [ ]:
S = np.cov(X, rowvar=False)
print("S\n", S)


S
 [[ 0.68569351 -0.042434    1.27431544  0.51627069]
 [-0.042434    0.18997942 -0.32965638 -0.12163937]
 [ 1.27431544 -0.32965638  3.11627785  1.2956094 ]
 [ 0.51627069 -0.12163937  1.2956094   0.58100626]]


次に，ブロードキャストを用いて，各行から平均を引いた行列`X_row`を作成し，その積を用いて共分散行列を計算する．このとき，$m - 1$で割ることで不偏分散を求めているが，これは`np.cov()`のデフォルトの挙動と一致させるためである．

In [ ]:
mean_row = X.mean(axis=0)
X_row = X - mean_row

S = X_row.T @ X_row / (m - 1)
print("S (unbiased)\n", S)


S (unbiased)
 [[ 0.68569351 -0.042434    1.27431544  0.51627069]
 [-0.042434    0.18997942 -0.32965638 -0.12163937]
 [ 1.27431544 -0.32965638  3.11627785  1.2956094 ]
 [ 0.51627069 -0.12163937  1.2956094   0.58100626]]


なお，$m$で割る標本分散（biased variance）を求める場合は，`np.cov()`の引数`bias`に`True`を指定する．


In [ ]:
S = np.cov(X, rowvar=False, bias=True)
print("S (biased)\n", S)

S = X_row.T @ X_row / m
print("S (biased)\n", S)


S (biased)
 [[ 0.68112222 -0.04215111  1.26582     0.51282889]
 [-0.04215111  0.18871289 -0.32745867 -0.12082844]
 [ 1.26582    -0.32745867  3.09550267  1.286972  ]
 [ 0.51282889 -0.12082844  1.286972    0.57713289]]
S (biased)
 [[ 0.68112222 -0.04215111  1.26582     0.51282889]
 [-0.04215111  0.18871289 -0.32745867 -0.12082844]
 [ 1.26582    -0.32745867  3.09550267  1.286972  ]
 [ 0.51282889 -0.12082844  1.286972    0.57713289]]


## 1.9. <a id='toc1_9_'></a>[線形代数の関数](#toc0_)

numpyに実装されている線形代数の関数の中で，頻繁に使用されるものについて説明する．


### 1.9.1. <a id='toc1_9_1_'></a>[単位行列](#toc0_)

$n$次の単位行列
\begin{align*}
I &=
\begin{pmatrix}
1 & 0 & \cdots & 0 \\
0 & 1 & \cdots & 0 \\
\vdots & \vdots & \ddots & \vdots \\
0 & 0 & \cdots & 1
\end{pmatrix}
\end{align*}
は
`np.eye(n)`または`np.identity(n)`で生成できる．

- https://numpy.org/doc/stable/reference/generated/numpy.identity.html
    > Return the identity array.
- https://numpy.org/doc/stable/reference/generated/numpy.eye.html
    > Return a 2-D array with ones on the diagonal and zeros elsewhere.

以下の例では，$n = 4$の単位行列を生成する．


In [ ]:
n = 4
I = np.eye(n)
print(I)

I = np.identity(n)
print(I)


[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]


### 1.9.2. <a id='toc1_9_2_'></a>[対角行列](#toc0_)

対角成分（対角要素）が$d_1, d_2, \ldots, d_n$である$n$次正方対角行列$D$

\begin{align*}
D &=
\begin{pmatrix}
d_1 & 0 & \cdots & 0 \\
0 & d_2 & \cdots & 0 \\
\vdots & \vdots & \ddots & \vdots \\
0 & 0 & \cdots & d_n
\end{pmatrix}
\end{align*}
は
`np.diag()`で生成できる．
引数として，対角成分を表す1次元のndarrayを与える．

- https://numpy.org/doc/stable/reference/generated/numpy.diag.html
    > Extract a diagonal or construct a diagonal array.


In [ ]:
n = 4
d = rng.random(n)
print("d", d)

print("D\n", diag(d))


d [0.20970149 0.7187192  0.12701908 0.86262852]
D
 [[0.20970149 0.         0.         0.        ]
 [0.         0.7187192  0.         0.        ]
 [0.         0.         0.12701908 0.        ]
 [0.         0.         0.         0.86262852]]


また，`np.diag()`の引数に$n$次正方行列$D$を与えると，その対角成分が$n$次元ベクトルとして返される．

$$
d = \mathrm{diag}(D) = (d_1, d_2, \ldots, d_n)^T \in R^n
$$


In [ ]:
n = 4
A = rng.random(size=(n, n))
print("A\n", A)

d = diag(A)
print("diagonal elements of A\n", d)


A
 [[0.08588511 0.9412896  0.38481803 0.39441287]
 [0.37556936 0.55554927 0.21308529 0.41204646]
 [0.50883601 0.45328706 0.09659703 0.66557341]
 [0.55196141 0.72895619 0.6347653  0.96796647]]
diagonal elements of A
 [0.08588511 0.55554927 0.09659703 0.96796647]


なお以下のように`diag(diag(A))`とすると，行列$A$の対角成分のみを持つ対角行列が得られる．

In [ ]:
n = 4
A = rng.random(size=(n, n))
print("A\n", A)

print("diag(diag(A))\n", diag(diag(A)))


A
 [[0.97209673 0.12249621 0.60568894 0.29231644]
 [0.10542243 0.63920996 0.10851146 0.01402059]
 [0.83658461 0.73469565 0.31763084 0.29672083]
 [0.68389145 0.05849627 0.51807476 0.58611169]]
diag(diag(A))
 [[0.97209673 0.         0.         0.        ]
 [0.         0.63920996 0.         0.        ]
 [0.         0.         0.31763084 0.        ]
 [0.         0.         0.         0.58611169]]


#### 1.9.2.1. <a id='toc1_9_2_1_'></a>[対角行列の逆行列](#toc0_)

対角行列の逆行列の計算は非常に簡単であり，対角成分の逆数を並べた対角行列である．つまり
$n \times n$ の対角成分 $d_{11}, d_{22}, \dots, d_{nn}$ を持つ対角行列 $D$

対角行列の逆行列の計算は非常に簡単である．各対角成分の逆数を取って，それを対角成分に持つ対角行列を作成すればよい．例えば，$n \times n$の対角行列$D$の逆行列は

\begin{align*}
D^{-1} = \begin{pmatrix}
\frac{1}{d_{11}} & 0 & \cdots & 0 \\
0 & \frac{1}{d_{22}} & \cdots & 0 \\
\vdots & \vdots & \ddots & \vdots \\
0 & 0 & \cdots & \frac{1}{d_{nn}}
\end{pmatrix}
\end{align*}

である．


In [ ]:
n = 5
D = np.diag(rng.random(size=n))
D_inv = np.diag(1 / np.diag(D))
print("D\n", D)
print("D^-1\n", D_inv)
print("D D^-1\n", D @ D_inv)


D
 [[0.7911741  0.         0.         0.         0.        ]
 [0.         0.46848201 0.         0.         0.        ]
 [0.         0.         0.37696303 0.         0.        ]
 [0.         0.         0.         0.35347622 0.        ]
 [0.         0.         0.         0.         0.89130003]]
D^-1
 [[1.2639443  0.         0.         0.         0.        ]
 [0.         2.1345537  0.         0.         0.        ]
 [0.         0.         2.65278007 0.         0.        ]
 [0.         0.         0.         2.82904465 0.        ]
 [0.         0.         0.         0.         1.12195665]]
D D^-1
 [[1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1.]]


#### 1.9.2.2. <a id='toc1_9_2_2_'></a>[対角行列の平方根](#toc0_)

ある行列$D$の行列平方根$D^{\frac{1}{2}}$とは，$D^{\frac{1}{2}} D^{\frac{1}{2}} = D$を満たす行列のこと，つまり$D$の$\frac{1}{2}$ 乗である．対角行列場合，その平方根は，各対角成分の平方根を取ってそれを対角成分に持つ対角行列を作成することで簡単に計算できる．

\begin{align*}
D^{\frac{1}{2}} = \begin{pmatrix}
\sqrt{d_{11}} & 0 & \cdots & 0 \\
0 & \sqrt{d_{22}} & \cdots & 0 \\
\vdots & \vdots & \ddots & \vdots \\
0 & 0 & \cdots & \sqrt{d_{nn}}
\end{pmatrix}
\end{align*}




In [ ]:
D_sqrt = np.diag(np.sqrt(np.diag(D)))
print("D\n", D)
print("D^1/2\n", D_sqrt)
print("D^1_2 D^1/2\n", D_sqrt @ D_sqrt)
print("D^1_2 D^1/2 == D", np.allclose(D_sqrt @ D_sqrt, D))


D
 [[0.7911741  0.         0.         0.         0.        ]
 [0.         0.46848201 0.         0.         0.        ]
 [0.         0.         0.37696303 0.         0.        ]
 [0.         0.         0.         0.35347622 0.        ]
 [0.         0.         0.         0.         0.89130003]]
D^1/2
 [[0.88947968 0.         0.         0.         0.        ]
 [0.         0.68445745 0.         0.         0.        ]
 [0.         0.         0.61397315 0.         0.        ]
 [0.         0.         0.         0.59453866 0.        ]
 [0.         0.         0.         0.         0.94408688]]
D^1_2 D^1/2
 [[0.7911741  0.         0.         0.         0.        ]
 [0.         0.46848201 0.         0.         0.        ]
 [0.         0.         0.37696303 0.         0.        ]
 [0.         0.         0.         0.35347622 0.        ]
 [0.         0.         0.         0.         0.89130003]]
D^1_2 D^1/2 == D True


### 1.9.3. <a id='toc1_9_3_'></a>[行列式](#toc0_)

$n$次正方行列$A \in R^{n \times n}$の行列式$|A|$の計算には`np.linalg.det()`を用いる．

- https://numpy.org/doc/stable/reference/generated/numpy.linalg.det.html
    > Compute the determinant of an array.

行列式には以下の性質がある．

- 任意の正方行列$A, B \in R^{n \times n}$について，$|AB| = |A| |B|$
- 単位行列$I$の行列式は$|I| = 1$
- 対角行列$D = \mathrm{diag}(d_1, d_2, \ldots, d_n)$の行列式は，対角成分の積：$|D| = \prod_i d_i$

以下に，これらの性質を確認する例を示す．

まず，$|AB| = |A| |B|$が成り立つことを確認する．

In [ ]:
n = 4
A = rng.random(size=(n, n))
B = rng.random(size=(n, n))

print("det(A)", det(A))
print("det(B)", det(B))
print(
    "det(AB) == det(A) det(B)",
    np.isclose(det(A @ B), det(A) * det(B))
)


det(A) -0.24790687011062176
det(B) -0.137901825166635
det(AB) == det(A) det(B) True


次に，単位行列の行列式が$1$であることを確認する．


In [ ]:
I = np.identity(n)
print("I:\n", I)
print("det(I):", np.linalg.det(I))

I:
 [[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]
det(I): 1.0


最後に，対角行列の行列式がその対角成分の積であることを確認する．

- https://numpy.org/doc/stable/reference/generated/numpy.prod.html
    > Return the product of array elements over a given axis.

In [ ]:
n = 4
A = diag(rng.random(n))
print("A\n", A)

print("det(A)", det(A))
print("det(A)", np.prod(diag(A)))

print(
    "det(A) == prod(diag(A))?",
    np.isclose(det(A), np.prod(diag(A)))
)


A
 [[0.08421128 0.         0.         0.        ]
 [0.         0.57247494 0.         0.        ]
 [0.         0.         0.679842   0.        ]
 [0.         0.         0.         0.56270571]]
det(A) 0.018442341884902163
det(A) 0.018442341884902163
det(A) == prod(diag(A))? True


### 1.9.4. <a id='toc1_9_4_'></a>[逆行列](#toc0_)

正則な$n$次正方行列$A \in R^{n \times n}$の逆行列$A^{-1} \in R^{n \times n}$を求めるには，`np.linalg.inv()`を用いる．

- https://numpy.org/doc/stable/reference/generated/numpy.linalg.inv.html
    > Compute the (multiplicative) inverse of a matrix.

逆行列には次のような性質がある．

- $A A^{-1} = A^{-1} A = I$ （$I$は単位行列）
- $(A^{-1})^{-1} = A$
- $(AB)^{-1} = B^{-1} A^{-1}$
- 逆行列の行列式は，行列式の逆数：$|A^{-1}| = |A|^{-1}$

以下に，これらの性質を確認する例を示す．

まず，$A A^{-1} = A^{-1} A = I$を確認する．


In [ ]:
n = 4
A = rng.random(size=(n, n))
I = np.identity(n)

print("A\n", A)
print("A^-1\n", inv(A))

print("A A^-1 == I\n", np.allclose(A @ inv(A), I))
print("A^-1 A == I\n", np.allclose(inv(A) @ A, I))


A
 [[0.64729444 0.33979645 0.24581116 0.01899478]
 [0.59956047 0.49037188 0.70998306 0.41478877]
 [0.50890498 0.56727411 0.83224611 0.87320321]
 [0.75238462 0.39236953 0.77789013 0.6410131 ]]
A^-1
 [[ 1.66269373 -2.2498126  -0.41546119  1.97249721]
 [ 2.22391744  0.74653704  2.85669349 -4.44042708]
 [-3.45216107  5.21251079 -3.04280717  0.87435198]
 [ 0.87645544 -4.14181036  2.43158341  0.90179424]]
A A^-1 == I
 True
A^-1 A == I
 True



次に，$|A^{-1}| = |A|^{-1}$および$|A| = |A^{-1}|^{-1}$を確認する．


In [ ]:
n = 4
A = rng.random(size=(n, n))

print("A\n", A)
print("det(A)\n", det(A))
print("det(A^-1)\n", det(inv(A)))
print(
    "det(A^-1) == 1/det(A)\n",
    np.isclose(det(inv(A)), 1 / det(A))
)
print(
    "det(A) == 1/det(A^-1)\n",
    np.isclose(det(A), 1 / det(inv(A)))
)


A
 [[0.45069417 0.20651128 0.20819403 0.22321354]
 [0.61572197 0.3702619  0.04680977 0.18926735]
 [0.00660098 0.57817451 0.52633084 0.96642223]
 [0.81382726 0.76809945 0.94303953 0.53915613]]
det(A)
 -0.05747113548805603
det(A^-1)
 -17.400039019723668
det(A^-1) == 1/det(A)
 True
det(A) == 1/det(A^-1)
 True


最後に，$(AB)^{-1} = B^{-1} A^{-1}$を確認する．


In [ ]:
n = 4
A = rng.random(size=(n, n))
B = rng.random(size=(n, n))

print("A\n", A)
print("B\n", B)

print(
    "(AB)^{-1} == A^{-1} B^{-1}\n",
    np.allclose(inv(A @ B), inv(B) @ inv(A))
)


A
 [[0.40350587 0.3319372  0.55479995 0.61815531]
 [0.36558269 0.33687498 0.55452873 0.33534962]
 [0.69953933 0.41619301 0.54880891 0.55583781]
 [0.58261287 0.45467601 0.12123181 0.1078831 ]]
B
 [[0.1589417  0.64320046 0.04155683 0.83269662]
 [0.1979595  0.44525383 0.39475906 0.44629517]
 [0.35322006 0.00638041 0.3794654  0.55941834]
 [0.23049172 0.6201009  0.46367933 0.06424258]]
(AB)^{-1} == A^{-1} B^{-1}
 True


### 1.9.5. <a id='toc1_9_5_'></a>[ランク，階数](#toc0_)

行列の階数（ランク，rank）とは，$n$次正方行列$A$の場合，その列ベクトル（または行ベクトル）の中で1次独立な列ベクトルの最大個数を指す．$\mathrm{rank}(A) = n$のとき，$A$はフルランクであるといい，$\mathrm{rank}(A) < n$であれば$A$はランク落ちしていると表現する．

$m \times n$行列の場合には，行ベクトルのうち1次独立な行ベクトルの最大個数と，列ベクトルのうち1次独立な列ベクトルの最大個数のうち，小さい方がその行列のランクである．

#### 1.9.5.1. <a id='toc1_9_5_1_'></a>[ランク計算の例](#toc0_)

以下に，ランク計算の例を示す．まず，フルランクの行列`A`を使い，その列をコピーしてランクが落ちた行列`B`を生成する．
この場合，`B`の線形独立な列の個数は$n-1$になり，`B`はランク落ちする．

- https://numpy.org/doc/stable/reference/generated/numpy.linalg.matrix_rank.html
     > Return matrix rank of array using SVD method


In [ ]:
I = np.identity(n)
print("I\n", I)
print("rank(I):", matrix_rank(I))

n = 5
A = rng.random(size=(n, n))
print("A\n", A)
print("rank(A):", matrix_rank(A))

B = A.copy()
B[:, 0] = B[:, 1].copy()  # A の列をコピーして B を作成
print("B\n", B)
print("rank(B):", matrix_rank(B))


I
 [[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]
rank(I): 4
A
 [[0.05674278 0.86590828 0.31727006 0.95954435 0.8539019 ]
 [0.46445127 0.4597117  0.19585435 0.5309363  0.17821876]
 [0.41097087 0.15033682 0.88898739 0.46460492 0.50406505]
 [0.55010123 0.77640157 0.87301038 0.42182197 0.32253043]
 [0.39607871 0.27025463 0.6613017  0.22747887 0.21963158]]
rank(A): 5
B
 [[0.86590828 0.86590828 0.31727006 0.95954435 0.8539019 ]
 [0.4597117  0.4597117  0.19585435 0.5309363  0.17821876]
 [0.15033682 0.15033682 0.88898739 0.46460492 0.50406505]
 [0.77640157 0.77640157 0.87301038 0.42182197 0.32253043]
 [0.27025463 0.27025463 0.6613017  0.22747887 0.21963158]]
rank(B): 4


#### 1.9.5.2. <a id='toc1_9_5_2_'></a>[ランクの性質](#toc0_)

次に，$\mathrm{rank}(AB) \leq \min( \mathrm{rank}(A), \mathrm{rank}(B) )$が成り立つことを確認する．


In [ ]:
m = 5
n = 3
p = 4
A = rng.random(size=(m, n))
B = rng.random(size=(n, p))
print("A\n", A)
print("rank(A)", matrix_rank(A))
print("B\n", B)
print("rank(B)", matrix_rank(B))
print("AB\n", A @ B)
print("rank(AB)", matrix_rank(A @ B))


A
 [[0.35838604 0.0765089  0.14760454]
 [0.83881077 0.21252215 0.41001559]
 [0.83010696 0.72439305 0.74868424]
 [0.81183298 0.62101876 0.68549582]
 [0.39919058 0.64653563 0.56317679]]
rank(A) 3
B
 [[0.06004925 0.07717724 0.32345295 0.37598547]
 [0.90803927 0.4348126  0.81943311 0.16911607]
 [0.23739922 0.86975938 0.89335658 0.85970117]]
rank(B) 3
AB
 [[0.1260351  0.18930671 0.31047843 0.27458262]
 [0.3406858  0.51375931 0.81175363 0.70381246]
 [0.88536169 1.03021572 1.53093418 1.07825938]
 [0.77539556 0.92889822 1.38386531 0.99958321]
 [0.74474857 0.80175856 1.16202977 0.74359317]]
rank(AB) 3


#### 1.9.5.3. <a id='toc1_9_5_3_'></a>[ランク落ちの例](#toc0_)

最後に，浮動小数点の丸め誤差により，ランク落ちする例を示す．この現象は，行列が大きくなり，多数の計算が繰り返されて，誤差が蓄積すると発生する可能性がある．

以下では，$A^T A$のランクが数値誤差（情報落ち）により低下していることを確認する．


In [ ]:

A = np.array([
    [1, 1],
    [1e-8, 0],
    [0, 1e-8]
])
print("A\n", A)
print("rank(A)", matrix_rank(A))
print("A^T\n", A.T)
print("rank(A^T)", matrix_rank(A.T))
print("A^T A\n", A.T @ A)
print("rank(A^T A)", matrix_rank(A.T @ A))


A
 [[1.e+00 1.e+00]
 [1.e-08 0.e+00]
 [0.e+00 1.e-08]]
rank(A) 2
A^T
 [[1.e+00 1.e-08 0.e+00]
 [1.e+00 0.e+00 1.e-08]]
rank(A^T) 2
A^T A
 [[1. 1.]
 [1. 1.]]
rank(A^T A) 1


### 1.9.6. <a id='toc1_9_6_'></a>[正定値対称行列](#toc0_)

行列の正定値性は様々なアルゴリズムで重要なや役割を果たす．
特に正定値対称行列は，固有値計算や最適化問題において重要な役割を果たす．

#### 1.9.6.1. <a id='toc1_9_6_1_'></a>[対称行列](#toc0_)

$n$次正方行列$A \in R^{n \times n}$が対称であるとは
$A = A^T$が成り立つことである．

#### 1.9.6.2. <a id='toc1_9_6_2_'></a>[正定値，負定置，半正定値](#toc0_)

$n$次実対称行列$A$が正定値（positive definite）であるとは，任意のベクトル$\boldsymbol{x}$に対して以下が成り立つことを意味する．
\begin{align*}
\boldsymbol{x}^T A \boldsymbol{x} > 0
\end{align*}
実対称行列の場合には$A$の固有値が全て正であることと同値である．
なお
$\boldsymbol{x}^T A \boldsymbol{x} \ge 0$
が成り立つ場合には半正定値（semi-positive definite）と呼ぶ．


また負定値（negative definite）とは
\begin{align*}
\boldsymbol{x}^T A \boldsymbol{x} < 0
\end{align*}
が成り立つことであり，正定値でも負定値でもない場合を不定値（indefinite）とよぶ．


### 1.9.7. <a id='toc1_9_7_'></a>[特殊な行列](#toc0_)

数値計算でよく登場する行列には，正方行列や対角行列の他にも，三角行列，三重対角行列，ヘッセンベルグ行列などがある．これらの行列は数学的に様々な性質を持ち，数値計算のアルゴリズムでその性質を活用することが多い．

これらの行列は要素の多くが0であり，適切なメモリ格納方法を用いることでメモリ使用量を削減でき，0要素との積和を除外することで計算量も削減可能である．

- https://docs.oracle.com/cd/E71939_01/html/E71963/z400049729536.html
    > パック格納スキームでは、行列のゼロでない要素が、列ごとに 1 次元配列にパック化されます。



#### 1.9.7.1. <a id='toc1_9_7_1_'></a>[対角行列](#toc0_)

$n$次正方対角行列$A \in R^{n \times n}$の非対角成分$a_{ij} \ (i \neq j)$がすべて0である行列を対角行列と呼ぶ．

\begin{align*}
A &=
\begin{pmatrix}
a_{11} & 0 & 0 & \cdots & 0 & 0 \\
0 & a_{22} & 0 & \cdots & 0 & 0 \\
0 & 0 & a_{33} & \cdots & 0 & 0 \\
\vdots & \vdots & \vdots & \ddots & \vdots & \vdots \\
0 & 0 & 0 & \cdots & a_{n-1, n-1} & 0 \\
0 & 0 & 0 & \cdots & 0 & a_{nn} \\
\end{pmatrix}
=
\begin{pmatrix}
* \\
  & * \\
  &   & * \\
  &   &   & \ddots \\
  &   &   &         & *
\end{pmatrix}
\end{align*}

ここで，$a_{ii}$を対角成分または主対角成分（main diagonals）と呼ぶ．主対角成分の1つ上の成分$a_{ij} \ (j = i + 1)$を優対角成分（superdiagonals），1つ下の成分$a_{ij} \ (j = i - 1)$を劣対角成分（subdiagonals）と呼ぶ．
なお，非対角成分の0を省略し，対角成分は*で表す表記もある．

`np.diag(A, k)`を使うと，$A$の対角成分や優対角成分，劣対角成分を抽出できる．引数`k`はオフセットを指定し，
`k=`で主対角成分（デフォルト），`k=1`で優対角成分，`k=-1`で劣対角成分を返す．


- https://numpy.org/doc/stable/reference/generated/numpy.diag.html
    > Extract a diagonal or construct a diagonal array.

In [ ]:
n = 5
A = rng.random(size=(n, n))
print("A\n", A)
print("main diagonals", diag(A))
print("superdiagonals", diag(A, k=1))
print("subdiagonals  ", diag(A, k=-1))


A
 [[0.17263214 0.87925698 0.47127358 0.86031395 0.68131375]
 [0.16013303 0.13769578 0.4836114  0.81299782 0.64987135]
 [0.28409181 0.19781878 0.22096947 0.20424359 0.00189013]
 [0.43444415 0.63203323 0.91721149 0.10835265 0.72125742]
 [0.08785106 0.90564646 0.81773732 0.81757165 0.92946235]]
main diagonals [0.17263214 0.13769578 0.22096947 0.10835265 0.92946235]
superdiagonals [0.87925698 0.4836114  0.20424359 0.72125742]
subdiagonals   [0.16013303 0.19781878 0.91721149 0.81757165]


また，`np.diag(d, k)`に$n$次元ベクトル`d`を設定すると，`k`の値によって得られる行列は次のように変わる．

- `k=0`なら主対角成分が`d`である$n$次対角行列
- `k=1`なら優対角成分が`d`でその他の成分が0である$n+1$次正方行列
- `k=-1`なら劣対角成分が`d`でその他の成分が0である$n+1$次正方行列


In [ ]:
d = np.array([1., 2., 3., 4.])
print("main diagonals\n", diag(d))
print("superdiagonals\n", diag(d, k=1))
print("subdiagonals\n", diag(d, k=-1))


main diagonals
 [[1. 0. 0. 0.]
 [0. 2. 0. 0.]
 [0. 0. 3. 0.]
 [0. 0. 0. 4.]]
superdiagonals
 [[0. 1. 0. 0. 0.]
 [0. 0. 2. 0. 0.]
 [0. 0. 0. 3. 0.]
 [0. 0. 0. 0. 4.]
 [0. 0. 0. 0. 0.]]
subdiagonals
 [[0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [0. 2. 0. 0. 0.]
 [0. 0. 3. 0. 0.]
 [0. 0. 0. 4. 0.]]



#### 1.9.7.2. <a id='toc1_9_7_2_'></a>[三重対角行列](#toc0_)

$n$次正方行列$A \in R^{n \times n}$のうち，対角・優対角・劣対角以外の成分$a_{ij} \ (j < i - 1, j > i + 1)$がすべて0である行列を，三重対角行列（tridiagonal matrix）と呼ぶ．

\begin{align*}
A &=
\begin{pmatrix}
a_{11} & a_{12} & 0 & 0 & \cdots & 0 & 0 & 0 \\
a_{21} & a_{22} & a_{23} & 0 & \cdots & 0 & 0 & 0 \\
0      & a_{32} & a_{33} & a_{34} & \cdots & 0 & 0 & 0 \\
0      & 0      & a_{43} & a_{44} & \cdots & 0 & 0 & 0 \\
\vdots & \vdots & \vdots & \vdots & \ddots & \vdots  & \vdots & \vdots \\
0 & 0 & 0 & 0 & \cdots & a_{n-2, n-2} & a_{n-2, n-1} & 0\\
0 & 0 & 0 & 0 & \cdots & a_{n-1, n-2} & a_{n-1, n-1} & a_{n-1, n} \\
0 & 0 & 0 & 0 & \cdots & 0 & a_{n, n-1} & a_{nn} \\
\end{pmatrix}
\end{align*}

<!-- \begin{align*}
&=
\begin{pmatrix}
* & * & \\
* & * & * & \\
  & * & * & * & \\
  &   & * & * & * \\
  &   &   & \ddots & \ddots & \ddots \\
  &&   &   &   &    * &  * &  * \\
  &&   &   &   &  & * &  *  &  * \\
  &&   &   &   &  &   &  *  &  * \\
\end{pmatrix}
\end{align*} -->

- https://en.wikipedia.org/wiki/Tridiagonal_matrix
    > In linear algebra, a tridiagonal matrix is a band matrix that has nonzero elements only on the main diagonal, the subdiagonal/lower diagonal (the first diagonal below this), and the supradiagonal/upper diagonal (the first diagonal above the main diagonal).

#### 1.9.7.3. <a id='toc1_9_7_3_'></a>[帯行列](#toc0_)

$n$次正方行列$A \in R^{n \times n}$の要素$a_{ij}$について，$a_{ij} \ (j < i - k_l, j > i + k_u)$以外が0である行列を帯行列（band matrix）と呼ぶ．ここで，$k_l$は下帯幅（lower bandwidth），$k_u$は上帯幅（upper bandwidth）と呼ばれる．

帯行列は以下のような特殊な場合を含む．

- $k_l = k_u = 0$の場合，これは対角行列に相当する．
- $k_l = k_u = 1$の場合，これは三重対角行列に相当する．

帯行列は，非ゼロ要素が主対角とその近傍の対角にのみ集中しており，さまざまな数値計算で用いられる．


- https://en.wikipedia.org/wiki/Band_matrix
    > In mathematics, particularly matrix theory, a band matrix or banded matrix is a sparse matrix whose non-zero entries are confined to a diagonal band, comprising the main diagonal and zero or more diagonals on either side.


#### 1.9.7.4. <a id='toc1_9_7_4_'></a>[上三角行列](#toc0_)

$n$次正方行列$A \in R^{n \times n}$のうち，下三角成分$a_{ij} \ (j < i)$がすべて0である行列を上三角行列（upper triangular matrix）と呼ぶ．


\begin{align*}
A &=
\begin{pmatrix}
a_{11}  & a_{12} & a_{11} & \cdots & a_{1, n-1}& a_{1, n} \\
0       & a_{22} & a_{23} & \cdots & a_{2, n-1}& a_{2, n} \\
0       & 0 & a_{33} & \cdots & a_{3, n-1}  & a_{3, n} \\
\vdots  & \vdots & \vdots & \ddots & \vdots & \vdots \\
0 & 0 & 0 & \cdots & a_{n-1, n-1}  & a_{n-1, n}    \\
0 & 0 & 0 & \cdots & 0 & a_{nn}
\end{pmatrix}
&=
\begin{pmatrix}
*  & * & * & \cdots & *& * \\
        & * & * & \cdots & * & * \\
        &   & * & \cdots & * & * \\
  &  &  & \ddots & \vdots & \vdots \\
  &   &   &  & *   & *     \\
  &   &   &  &  & *
\end{pmatrix}
\end{align*}

- https://en.wikipedia.org/wiki/Triangular_matrix#Description

##### 1.9.7.4.1. <a id='toc1_9_7_4_1_'></a>[単位上三角行列](#toc0_)

対角成分が1である場合，すなわち$a_{ii} = 1$の上三角行列を**単位上三角行列**（unit upper triangular matrix）と呼ぶ．


\begin{align*}
A &=
\begin{pmatrix}
1       & a_{12} & a_{13} & \cdots & a_{1, n-1}& a_{1, n} \\
0       & 1 & a_{23} & \cdots & a_{2, n-1}& a_{2, n} \\
0       & 0 & 1 & \cdots & a_{3, n-1}  & a_{3, n} \\
\vdots  & \vdots & \vdots & \ddots & \vdots & \vdots \\
0 & 0 & 0 & \cdots & 1  & a_{n-1, n}    \\
0 & 0 & 0 & \cdots & 0 & 1
\end{pmatrix}
&=
\begin{pmatrix}
1  & * & * & \cdots & *& * \\
        & 1 & * & \cdots & * & * \\
        &   & 1 & \cdots & * & * \\
  &  &  & \ddots & \vdots & \vdots \\
  &   &   &  & 1   & *     \\
  &   &   &  &  & 1
\end{pmatrix}
\end{align*}


##### 1.9.7.4.2. <a id='toc1_9_7_4_2_'></a>[狭義上三角行列](#toc0_)

対角成分も0である場合，すなわち$a_{ij} \ (j \leq i)$が0の行列を**狭義上三角行列**（strict upper triangular matrix）と呼ぶ．


\begin{align*}
A &=
\begin{pmatrix}
0       & a_{12} & a_{13} & \cdots & a_{1, n-1}& a_{1, n} \\
0       & 0 & a_{23} & \cdots & a_{2, n-1}& a_{2, n} \\
0       & 0 & 0 & \cdots & a_{3, n-1}  & a_{3, n} \\
\vdots  & \vdots & \vdots & \ddots & \vdots & \vdots \\
0 & 0 & 0 & \cdots & 0  & a_{n-1, n}    \\
0 & 0 & 0 & \cdots & 0 & 0
\end{pmatrix}
&=
\begin{pmatrix}
  & * & * & \cdots & *& * \\
        &  & * & \cdots & * & * \\
        &   &  & \cdots & * & * \\
  &  &  & & \vdots & \vdots \\
  &   &   &  &    & *     \\
  &   &   &  &  &
\end{pmatrix}
\end{align*}



##### 1.9.7.4.3. <a id='toc1_9_7_4_3_'></a>[上ヘッセンベルグ行列](#toc0_)


さらに，$a_{ij} \ (j < i - 1)$が0である行列を**上ヘッセンベルグ行列**（upper Hessenberg matrix）と呼ぶ．これは，対角の1つ下の成分も非ゼロとなる行列で，次のように表される．


\begin{align*}
A &=
\begin{pmatrix}
a_{11}  & a_{12} & a_{11} & \cdots & a_{1, n-1}& a_{1, n} \\
a_{12}  & a_{22} & a_{23} & \cdots & a_{2, n-1}& a_{2, n} \\
0       & a_{32} & a_{33} & \cdots & a_{3, n-1}  & a_{3, n} \\
\vdots  & \vdots & \vdots & \ddots & \vdots & \vdots \\
0 & 0 & 0 & \cdots & a_{n-1, n-1}  & a_{n-1, n}    \\
0 & 0 & 0 & \cdots & a_{n, n-1} & a_{nn}
\end{pmatrix}
&=
\begin{pmatrix}
*  & * & * & \cdots & *& * \\
*  & * & * & \cdots & * & * \\
   & * & * & \cdots & * & * \\
   &   & \ddots  &  & \vdots & \vdots \\
   &   &   & * & *   & *     \\
   &   &   &  & * & *
\end{pmatrix}
\end{align*}

- https://en.wikipedia.org/wiki/Hessenberg_matrix#Upper_Hessenberg_matrix


##### 1.9.7.4.4. <a id='toc1_9_7_4_4_'></a>[numpy を用いた上三角行列の生成](#toc0_)

`np.triu(A, k)`を用いると，行列`A`の下三角成分$a_{ij} \ (j < i + k)$を0にした上三角行列を生成できる．

- https://numpy.org/doc/stable/reference/generated/numpy.triu.html
    > Upper triangle of an array.


引数`k`の値によって以下のように行列が変わる．

- `k=0`は通常の上三角行列を返す（デフォルト）．
- `k=1`は狭義上三角行列を返す．
- `k=-1`は上ヘッセンベルグ行列を返す．




In [21]:
n = 5
A = rng.random(size=(n, n))
print("A\n", A)

A_tri_upper = np.triu(A)
print("upper triangular matrix\n", A_tri_upper)

A_strict_tri_upper = np.triu(A, k=1)
print("strict upper traiangular matrix\n", A_strict_tri_upper)

A_upper_hessenberg = np.triu(A, k=-1)
print("upper Hessenbrerg matrix\n", A_upper_hessenberg)


A
 [[0.16118005 0.27775967 0.0879739  0.83553047 0.63300962]
 [0.33746773 0.07117413 0.07987348 0.15778569 0.98617762]
 [0.28237512 0.82701853 0.03029276 0.54305252 0.04193919]
 [0.86044393 0.12222217 0.09386032 0.22453057 0.79053404]
 [0.83263062 0.68928352 0.85181574 0.32616017 0.61827099]]
upper triangular matrix
 [[0.16118005 0.27775967 0.0879739  0.83553047 0.63300962]
 [0.         0.07117413 0.07987348 0.15778569 0.98617762]
 [0.         0.         0.03029276 0.54305252 0.04193919]
 [0.         0.         0.         0.22453057 0.79053404]
 [0.         0.         0.         0.         0.61827099]]
strict upper traiangular matrix
 [[0.         0.27775967 0.0879739  0.83553047 0.63300962]
 [0.         0.         0.07987348 0.15778569 0.98617762]
 [0.         0.         0.         0.54305252 0.04193919]
 [0.         0.         0.         0.         0.79053404]
 [0.         0.         0.         0.         0.        ]]
upper Hessenbrerg matrix
 [[0.16118005 0.27775967 0.0879739  0.835


#### 1.9.7.5. <a id='toc1_9_7_5_'></a>[下三角行列](#toc0_)

$n$次正方行列$A \in R^{n \times n}$のうち，上三角成分$a_{ij} \ (j > i)$がすべて0である行列を下三角行列（lower triangular matrix）と呼ぶ．


\begin{align*}
A &=
\begin{pmatrix}
a_{11} & 0 & 0 & \cdots & 0 & 0 \\
a_{21} & a_{22} & 0 & \cdots & 0 & 0 \\
a_{31} & a_{32} & a_{33} & \cdots & 0 & 0 \\
\vdots & \vdots & \vdots & \ddots & \vdots & \vdots \\
a_{n-1, 1} & a_{n-1, 2} & a_{n-1, 3} & \cdots & a_{n-1, n-1} & 0 \\
a_{n, 1}   & a_{n, 2}   & a_{n, 3}   & \cdots & a_{n, n-1}   & a_{nn} \\
\end{pmatrix}
\end{align*}

<!-- &=
\begin{pmatrix}
* \\
* & * \\
* & * & * \\
\vdots & \vdots & \vdots & \ddots \\
* & * & * & \cdots & * \\
* & * & * & \cdots & * & * \\
\end{pmatrix} -->


- https://en.wikipedia.org/wiki/Triangular_matrix#Description


##### 1.9.7.5.1. <a id='toc1_9_7_5_1_'></a>[単位下三角行列](#toc0_)


対角成分が1である場合，すなわち$a_{ii} = 1$の下三角行列を**単位下三角行列**と呼ぶ．

\begin{align*}
A &=
\begin{pmatrix}
1 & 0 & 0 & \cdots & 0 & 0 \\
a_{21} & 1 & 0 & \cdots & 0 & 0 \\
a_{31} & a_{32} & 1 & \cdots & 0 & 0 \\
\vdots & \vdots & \vdots & \ddots & \vdots & \vdots \\
a_{n-1, 1} & a_{n-1, 2} & a_{n-1, 3} & \cdots & 1 & 0 \\
a_{n, 1}   & a_{n, 2}   & a_{n, 3}   & \cdots & a_{n, n-1}   & 1 \\
\end{pmatrix}
&=
\begin{pmatrix}
1 \\
* & 1 \\
* & * & 1 \\
\vdots & \vdots & \vdots & \ddots \\
* & * & * & \cdots & 1 \\
* & * & * & \cdots & * & 1 \\
\end{pmatrix}
\end{align*}


##### 1.9.7.5.2. <a id='toc1_9_7_5_2_'></a>[狭義下三角行列](#toc0_)


対角成分も0である場合，すなわち$a_{ij} \ (j \geq i)$が0の行列を**狭義下三角行列**と呼ぶ．

\begin{align*}
A &=
\begin{pmatrix}
0 & 0 & 0 & \cdots & 0 & 0 \\
a_{21} & 0 & 0 & \cdots & 0 & 0 \\
a_{31} & a_{32} & 0 & \cdots & 0 & 0 \\
\vdots & \vdots & \vdots & \ddots & \vdots & \vdots \\
a_{n-1, 1} & a_{n-1, 2} & a_{n-1, 3} & \cdots & 0 & 0 \\
a_{n, 1}   & a_{n, 2}   & a_{n, 3}   & \cdots & a_{n, n-1}   & 0 \\
\end{pmatrix}
&=
\begin{pmatrix}
 \\
* &  \\
* & * & \\
\vdots & \vdots & \vdots \\
* & * & * & \cdots &  \\
* & * & * & \cdots & * &  \\
\end{pmatrix}
\end{align*}


##### 1.9.7.5.3. <a id='toc1_9_7_5_3_'></a>[下ヘッセンベルグ行列](#toc0_)


さらに，$a_{ij} \ (j > i + 1)$が0である行列を**下ヘッセンベルグ行列**と呼ぶ．これは，対角よりも1つ上の成分まで非ゼロ要素を持つ行列であり，次のように表される．


\begin{align*}
A &=
\begin{pmatrix}
a_{11} & a_{12} & 0 & \cdots & 0 & 0 \\
a_{21} & a_{22} & a_{23} & \cdots & 0 & 0 \\
a_{31} & a_{32} & a_{33} & \cdots & 0 & 0 \\
\vdots & \vdots & \vdots & \ddots & \vdots & \vdots \\
a_{n-1, 1} & a_{n-1, 2} & a_{n-1, 3} & \cdots & a_{n-1, n-1} & a_{n-1, n} \\
a_{n, 1}   & a_{n, 2}   & a_{n, 3}   & \cdots & a_{n, n-1}   & a_{nn} \\
\end{pmatrix}
&=
\begin{pmatrix}
* & * \\
* & * & * \\
* & * & * & * \\
\vdots & \vdots & \vdots & & \ddots \\
* & * & * & \cdots & * & * \\
* & * & * & \cdots & * & * \\
\end{pmatrix}
\end{align*}


- https://en.wikipedia.org/wiki/Hessenberg_matrix#Lower_Hessenberg_matrix


##### 1.9.7.5.4. <a id='toc1_9_7_5_4_'></a>[numpyを用いた下三角行列の生成](#toc0_)

`np.tril(A, k)`を使うと，行列`A`の上三角成分$a_{ij} \ (j > i + k)$を0にした下三角行列を生成できる．


- https://numpy.org/doc/stable/reference/generated/numpy.tril.html
    > Lower triangle of an array.


引数`k`の値によって以下のように行列が変わる．

- `k=0`で通常の下三角行列を返す（デフォルト）．
- `k=-1`で狭義下三角行列を返す．
- `k=1`で下ヘッセンベルグ行列を返す．




In [ ]:
n = 5
A = rng.random(size=(n, n))
print("A\n", A)

A_tri_lower = np.tril(A)
print("lower triangular matrix\n", A_tri_lower)

A_strict_tri_lower = np.tril(A, k=-1)
print("strict lower traiangular matrix\n", A_strict_tri_lower)

A_lower_hessenberg = np.tril(A, k=1)
print("lower Hessenbrerg matrix\n", A_lower_hessenberg)


A
 [[0.96870612 0.09069376 0.14383047 0.25883038 0.53699385]
 [0.66546779 0.35559277 0.68356064 0.31821765 0.73865647]
 [0.45865127 0.82105924 0.35418865 0.20857661 0.08206227]
 [0.72748801 0.98161748 0.57121304 0.66151322 0.7561073 ]
 [0.47678579 0.56315663 0.51293182 0.73533104 0.38769583]]
lower triangular matrix
 [[0.96870612 0.         0.         0.         0.        ]
 [0.66546779 0.35559277 0.         0.         0.        ]
 [0.45865127 0.82105924 0.35418865 0.         0.        ]
 [0.72748801 0.98161748 0.57121304 0.66151322 0.        ]
 [0.47678579 0.56315663 0.51293182 0.73533104 0.38769583]]
strict lower traiangular matrix
 [[0.         0.         0.         0.         0.        ]
 [0.66546779 0.         0.         0.         0.        ]
 [0.45865127 0.82105924 0.         0.         0.        ]
 [0.72748801 0.98161748 0.57121304 0.         0.        ]
 [0.47678579 0.56315663 0.51293182 0.73533104 0.        ]]
lower Hessenbrerg matrix
 [[0.96870612 0.09069376 0.         0.   

#### 1.9.7.6. <a id='toc1_9_7_6_'></a>[三角行列の行列式](#toc0_)

三角行列の行列式は，対角行列と同様に，対角成分の積で計算される．

- **上三角行列**および**下三角行列**の行列式は，その対角成分の積と等しい．
- **狭義上三角行列**および**狭義下三角行列**（対角成分がすべて0である三角行列）の行列式は0である．

以下は，三角行列の行列式を確認する例である．


In [ ]:
n = 5
A = rng.random(size=(n, n))

A_tri_lower = np.tril(A)
print("lower triangular matrix\n", A_tri_lower)
print("det", det(A_tri_lower))
print("det", np.prod(diag(A_tri_lower)))

A_strict_tri_lower = np.tril(A, k=-1)
print("strict lower traiangular matrix\n", A_strict_tri_lower)
print("det", det(A_strict_tri_lower))


lower triangular matrix
 [[0.853625   0.         0.         0.         0.        ]
 [0.65035407 0.3970122  0.         0.         0.        ]
 [0.97835526 0.04835814 0.38146841 0.         0.        ]
 [0.48631036 0.7738005  0.98585647 0.54541288 0.        ]
 [0.70123782 0.03175853 0.87607842 0.86455809 0.48707701]]
det 0.034344134825252115
det 0.034344134825252115
strict lower traiangular matrix
 [[0.         0.         0.         0.         0.        ]
 [0.65035407 0.         0.         0.         0.        ]
 [0.97835526 0.04835814 0.         0.         0.        ]
 [0.48631036 0.7738005  0.98585647 0.         0.        ]
 [0.70123782 0.03175853 0.87607842 0.86455809 0.        ]]
det 0.0



##### 1.9.7.6.1. <a id='toc1_9_7_6_1_'></a>[特に対角成分が1である三角行列の行列式](#toc0_)

対角成分がすべて1である三角行列の行列式は，対角成分の積が1なので，その行列式も1になる．
次のコードでは，`np.fill_diagonal()`を使って，行列の対角成分に1を代入している．


- https://numpy.org/doc/stable/reference/generated/numpy.fill_diagonal.html
    > Fill the main diagonal of the given array of any dimensionality.


In [ ]:
n = 5
A = rng.random(size=(n, n))
np.fill_diagonal(A, 1.0)

A_tri_lower = np.tril(A)
print("lower triangular matrix\n", A_tri_lower)
print("det", det(A_tri_lower))


lower triangular matrix
 [[1.         0.         0.         0.         0.        ]
 [0.6657135  1.         0.         0.         0.        ]
 [0.52735005 0.34228608 1.         0.         0.        ]
 [0.8543165  0.40342686 0.0927158  1.         0.        ]
 [0.30669401 0.76959709 0.13329143 0.85067509 1.        ]]
det 1.0


## 1.10. <a id='toc1_10_'></a>[行列の変換](#toc0_)

数値計算における行列計算の基本は，直交行列を用いた変換である．

### 1.10.1. <a id='toc1_10_1_'></a>[直交行列](#toc0_)

$m$次正方行列$A \in R^{m \times m}$が**直交行列**（orthogonal matrix）である場合，次の性質が成り立つ．

- $A^T = A^{-1}$（転置行列は逆行列である）
- $A^T A = A A^T = I$（積が単位行列となる）
- $|A| = \pm 1$（行列式の絶対値が1である）
- 任意のベクトル$\boldsymbol{x} \in R^m$について，$\| A \boldsymbol{x} \| = \| \boldsymbol{x} \|$（ベクトルの長さが保存される）


#### 1.10.1.1. <a id='toc1_10_1_1_'></a>[列直交行列](#toc0_)

行列$B \in R^{m \times n}$が**列直交行列**（column orthogonal matrix）であるとは，次が成り立つ行列を指す．

- $B^T B = I \in R^{n \times n}$


直交行列$A \in R^{m \times m}$を列ベクトル$\boldsymbol{a}_i \in R^m$で表すと，

$$
A = (\boldsymbol{a}_1, \boldsymbol{a}_2, \ldots, \boldsymbol{a}_m) \in R^{m \times m}
$$

となり，$A^T A = I$は，列ベクトル同士が**正規直交**していることを意味する．すなわち，

\begin{align*}
\boldsymbol{a}_i^T \boldsymbol{a}_j =
\begin{cases}
1 & i = j \\
0 & i \neq j
\end{cases}
\end{align*}

この行列$A$の$m$個の列の一部を残した行列

$$
B = (\boldsymbol{a}_1, \boldsymbol{a}_2, \ldots, \boldsymbol{a}_n) \in R^{m \times n} \quad (n \le m)
$$

は，列ベクトル同士が正規直交しているため，列直交行列である．



### 1.10.2. <a id='toc1_10_2_'></a>[行列の基本変形](#toc0_)

行列の列や行を入れ替えたり，定数倍したりする操作は直交行列を用いて表現でき，これを**行列の基本変形**と呼ぶ．これらの変形は，行列に対する主な操作として今後重要な役割を果たす．


#### 1.10.2.1. <a id='toc1_10_2_1_'></a>[列・行の定数倍](#toc0_)

列や行を定数倍する操作は，単位行列の一部を修正した直交行列で実現できる．
たとえば，次の行列$I_{1c}$は，単位行列の1番目の要素を$c$に置き換えたものである．

\begin{align*}
I_{1c} &=
\begin{pmatrix}
c \\
 & 1  \\
 &  & \ddots \\
 &  &  & 1
\end{pmatrix}
\end{align*}

この$I_{1c}$を行列$A = (\boldsymbol{a}_1, \boldsymbol{a}_2, \ldots, \boldsymbol{a}_n) \in R^{n \times n}$に右から掛けると，次のように1列目が$c$倍される行列が得られる．

$$
A I_{1c} = (c\boldsymbol{a}_1, \boldsymbol{a}_2, \ldots, \boldsymbol{a}_n)
$$

一方，$A^T$に左から掛けると，1行目が$c$倍されることが分かる．

$$
I_{1c} A^T =
\begin{pmatrix}
c\boldsymbol{a}_1^T \\
\boldsymbol{a}_2^T \\
\vdots \\
\boldsymbol{a}_n^T
\end{pmatrix}
$$

つまり，単位行列を修正して$a_{ii} = c$とした行列$I_{ic}$は，行列に右から掛けると列$i$を$c$倍し，左から掛けると行$i$を$c$倍する行列である．


In [ ]:
n = 5
A = rng.integers(0, 10, size=(n, n)).astype(float)
print("A\n", A)

c = 10
i = 2
I_ic = np.eye(n)
I_ic[i, i] = c
print("I_ic\n", I_ic)

print("A I_ic\n", A @ I_ic)  # 列 i を c 倍
print("I_ic A\n", I_ic @ A)  # 行 i を c 倍

A
 [[2. 9. 9. 6. 9.]
 [1. 8. 2. 1. 9.]
 [9. 6. 9. 6. 8.]
 [0. 9. 0. 7. 0.]
 [9. 9. 7. 7. 6.]]
I_ic
 [[ 1.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.]
 [ 0.  0. 10.  0.  0.]
 [ 0.  0.  0.  1.  0.]
 [ 0.  0.  0.  0.  1.]]
A I_ic
 [[ 2.  9. 90.  6.  9.]
 [ 1.  8. 20.  1.  9.]
 [ 9.  6. 90.  6.  8.]
 [ 0.  9.  0.  7.  0.]
 [ 9.  9. 70.  7.  6.]]
I_ic A
 [[ 2.  9.  9.  6.  9.]
 [ 1.  8.  2.  1.  9.]
 [90. 60. 90. 60. 80.]
 [ 0.  9.  0.  7.  0.]
 [ 9.  9.  7.  7.  6.]]


#### 1.10.2.2. <a id='toc1_10_2_2_'></a>[列・行の置換](#toc0_)

単位行列を修正し，$a_{11} = a_{22} = 0$とし，$a_{12} = a_{21} = 1$とした行列$M_{12}$を考える．

\begin{align*}
M_{12} &=
\begin{pmatrix}
0 & 1  \\
1 & 0 \\
 &  & 1 \\
& & & \ddots \\
  & & & & 1
\end{pmatrix}
\end{align*}

この行列$M_{12}$を行列$A$に右から適用すると，$A$の列1と列2が置換される．

$$
A M_{12} = (\boldsymbol{a}_2, \boldsymbol{a}_1, \ldots, \boldsymbol{a}_n)
$$

一方，$A^T$に左から適用すると，$A^T$の行1と行2が置換される．

$$
M_{12} A^T =
\begin{pmatrix}
\boldsymbol{a}_2^T \\
\boldsymbol{a}_1^T \\
\vdots \\
\boldsymbol{a}_n^T
\end{pmatrix}
$$

したがって，単位行列を修正して$a_{pp} = a_{qq} = 0$とし，$a_{pq} = a_{qp} = 1$とした行列$M_{pq}$は，行列に右から適用すると列$p$と$q$が置換され，左から適用すると行$p$と$q$が置換される**置換行列**である．



\begin{align*}
M_{pq} =
\begin{pmatrix}
1 \\
& \ddots & \\
&& 1 \\
&&& 0 && 1\\
&&&& \ddots \\
&&& 1 && 0 \\
&&&&&&1 \\
&&&&&&& \ddots \\
&&&&&&&&1 \\
\end{pmatrix}
\end{align*}

なお，$M_{pq}$を2回適用すると元に戻るため，$M_{pq}^2 = I$であり，それ自身が逆行列$M_{pq}^{-1} = M_{pq}$である（ただし2つの行または列を置換する場合に限る）．

次のPythonコードは，行列に対して列や行を入れ替える操作を行う例を示している．


In [ ]:
n = 5
A = rng.integers(0, 10, size=(n, n)).astype(float)
print("A\n", A)

p = 2
q = 4
M = np.eye(n)
M[p, p] = 0
M[q, q] = 0
M[p, q] = 1
M[q, p] = 1
print("M\n", M)

print("A M\n", A @ M)  # 列 p と q の入れ替え
print("M A\n", M @ A)  # 行 p と q の入れ替え

print("A M M \n", A @ M @ M)  # 置換を2回適用して元に戻ることを確認


A
 [[0. 5. 8. 8. 0.]
 [0. 8. 1. 5. 7.]
 [3. 2. 7. 6. 5.]
 [0. 3. 3. 1. 6.]
 [8. 9. 1. 0. 6.]]
M
 [[1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 1. 0.]
 [0. 0. 1. 0. 0.]]
A M
 [[0. 5. 0. 8. 8.]
 [0. 8. 7. 5. 1.]
 [3. 2. 5. 6. 7.]
 [0. 3. 6. 1. 3.]
 [8. 9. 6. 0. 1.]]
M A
 [[0. 5. 8. 8. 0.]
 [0. 8. 1. 5. 7.]
 [8. 9. 1. 0. 6.]
 [0. 3. 3. 1. 6.]
 [3. 2. 7. 6. 5.]]
A M M 
 [[0. 5. 8. 8. 0.]
 [0. 8. 1. 5. 7.]
 [3. 2. 7. 6. 5.]
 [0. 3. 3. 1. 6.]
 [8. 9. 1. 0. 6.]]


#### 1.10.2.3. <a id='toc1_10_2_3_'></a>[列・行の線形和](#toc0_)

定数倍と列や行の交換の操作を基に，単位行列を修正した次の行列$P$を考える．


\begin{align*}
P &=
\begin{pmatrix}
\alpha & \beta  \\
\gamma & \delta \\
 &  & 1 \\
& & & \ddots \\
  & & & & 1
\end{pmatrix}
\end{align*}

この行列$P$を行列$A$に右から適用すると，1列目と2列目がそれらの定数倍による線形和に置き換えられる．

$$
A P = (\alpha \boldsymbol{a}_1 + \gamma \boldsymbol{a}_2, \beta \boldsymbol{a}_1 + \delta \boldsymbol{a}_2, \ldots, \boldsymbol{a}_n)
$$

一方，$A^T$に左から適用すると，行に対して同様に処理が行われる．

$$
P A^T =
\begin{pmatrix}
\alpha \boldsymbol{a}_1^T + \beta \boldsymbol{a}_2^T \\
\gamma \boldsymbol{a}_1^T + \delta \boldsymbol{a}_2^T \\
\vdots \\
\boldsymbol{a}_n
\end{pmatrix}
$$

この操作は，$p$列目と$q$列目（または行）に対しても同様に適用でき，その場合は次のような行列$P_{pq}$で表現できる．

\begin{align*}
P_{pq} =
\begin{pmatrix}
1 \\
& \ddots & \\
&& 1 \\
&&& \alpha && \beta \\
&&&& \ddots \\
&&& \gamma && \delta \\
&&&&&&1 \\
&&&&&&& \ddots \\
&&&&&&&&1 \\
\end{pmatrix}
\end{align*}


次のコードは，行列の列や行を定数倍して線形和を取る操作の例である．


In [ ]:
n = 5
A = rng.integers(0, 10, size=(n, n)).astype(float)
print("A\n", A)

p = 2
q = 4
P_pq = np.eye(n)
P_pq[p, p] = 2  # alpha
P_pq[q, q] = 3  # delta
P_pq[p, q] = -1  # beta
P_pq[q, p] = -10  # gamma
print("P_pq\n", P_pq)

print("A P_pq\n", A @ P_pq)
print("P_pq A\n", P_pq @ A)

A
 [[6. 1. 7. 6. 1.]
 [0. 7. 5. 1. 0.]
 [2. 6. 7. 6. 4.]
 [3. 0. 2. 7. 3.]
 [1. 5. 5. 6. 4.]]
P_pq
 [[  1.   0.   0.   0.   0.]
 [  0.   1.   0.   0.   0.]
 [  0.   0.   2.   0.  -1.]
 [  0.   0.   0.   1.   0.]
 [  0.   0. -10.   0.   3.]]
A P_pq
 [[  6.   1.   4.   6.  -4.]
 [  0.   7.  10.   1.  -5.]
 [  2.   6. -26.   6.   5.]
 [  3.   0. -26.   7.   7.]
 [  1.   5. -30.   6.   7.]]
P_pq A
 [[  6.   1.   7.   6.   1.]
 [  0.   7.   5.   1.   0.]
 [  3.   7.   9.   6.   4.]
 [  3.   0.   2.   7.   3.]
 [-17. -45. -55. -42. -28.]]


#### 1.10.2.4. <a id='toc1_10_2_4_'></a>[ndarayによる実装](#toc0_)

直交行列で表される行列の基本変形は，理論的には重要な操作だが，実際の実装において行列積を使用すると計算量が増加するため，計算効率の観点では不利である．


##### 1.10.2.4.1. <a id='toc1_10_2_4_1_'></a>[定数倍](#toc0_)

行列積を用いると，$A \in R^{n \times n}$と$I_{ic} \in R^{n \times n}$の積は計算量が$2n^3$ flopsになってしまうが，ndarrayのインデキシングを使って$i$列目に直接スカラーをかければ計算量は$n$ flopsですむ．

以下は，定数倍を効率よく行う実装例である．


In [ ]:
with np.printoptions(formatter={'float': '{:12.8f}'.format}):
    n = 5
    A = rng.random(size=(n, n))
    print("A\n", A)

    c = 10
    i = 2
    A[i] *= c
    print("A I_ic\n", A)

    c = 10
    i = 2
    A[:, i] *= c
    print("I_ic A\n", A)


A
 [[  0.75855204   0.01037082   0.19131655   0.67909869   0.15057504]
 [  0.27170061   0.80638500   0.09136109   0.25504647   0.03119116]
 [  0.21182315   0.68637998   0.28535488   0.67022936   0.61674339]
 [  0.92824822   0.03028555   0.03093735   0.00297987   0.15621315]
 [  0.44446931   0.06813625   0.78570950   0.31906418   0.15190990]]
A I_ic
 [[  0.75855204   0.01037082   0.19131655   0.67909869   0.15057504]
 [  0.27170061   0.80638500   0.09136109   0.25504647   0.03119116]
 [  2.11823145   6.86379980   2.85354877   6.70229364   6.16743391]
 [  0.92824822   0.03028555   0.03093735   0.00297987   0.15621315]
 [  0.44446931   0.06813625   0.78570950   0.31906418   0.15190990]]
I_ic A
 [[  0.75855204   0.01037082   1.91316551   0.67909869   0.15057504]
 [  0.27170061   0.80638500   0.91361093   0.25504647   0.03119116]
 [  2.11823145   6.86379980  28.53548766   6.70229364   6.16743391]
 [  0.92824822   0.03028555   0.30937355   0.00297987   0.15621315]
 [  0.44446931   0.06813625

なおここでは`with`文を使ってndarrayの表示を整形している．

- https://numpy.org/doc/stable/reference/generated/numpy.printoptions.html
    > Context manager for setting print options.

##### 1.10.2.4.2. <a id='toc1_10_2_4_2_'></a>[置換](#toc0_)

行列積を用いると，$A \in R^{n \times n}$と$M_{pq} \in R^{n \times n}$の積は$2n^3$ flopsになるが，ndarrayのインデキシングを使って$p$列目と$q$列目を入れ替えれば，浮動小数点演算を伴わないため，flopsは0で済む（コピー処理は発生するがflopsにはカウントされない）．

以下は列や行を置換する効率的な実装例である．

In [ ]:
with np.printoptions(formatter={'float': '{:12.8f}'.format}):
    n = 5
    A = rng.random(size=(n, n))
    print("A\n", A)

    p = 1
    q = 2
    A[p], A[q] = A[q].copy(), A[p].copy()
    print("M A\n", A)

    p = 0
    q = 4
    A[:, p], A[:, q] = A[:, q].copy(), A[:, p].copy()
    print("A M\n", A)


A
 [[  0.60654484   0.60196479   0.57040789   0.60609069   0.76858731]
 [  0.00085821   0.96799474   0.81486529   0.88592828   0.58990274]
 [  0.98927811   0.23607328   0.96859219   0.35487175   0.73275313]
 [  0.74163212   0.55258776   0.37366664   0.18253782   0.17083117]
 [  0.08642904   0.81191724   0.97662676   0.21880139   0.12596554]]
M A
 [[  0.60654484   0.60196479   0.57040789   0.60609069   0.76858731]
 [  0.98927811   0.23607328   0.96859219   0.35487175   0.73275313]
 [  0.00085821   0.96799474   0.81486529   0.88592828   0.58990274]
 [  0.74163212   0.55258776   0.37366664   0.18253782   0.17083117]
 [  0.08642904   0.81191724   0.97662676   0.21880139   0.12596554]]
A M
 [[  0.76858731   0.60196479   0.57040789   0.60609069   0.60654484]
 [  0.73275313   0.23607328   0.96859219   0.35487175   0.98927811]
 [  0.58990274   0.96799474   0.81486529   0.88592828   0.00085821]
 [  0.17083117   0.55258776   0.37366664   0.18253782   0.74163212]
 [  0.12596554   0.81191724   0.9

##### 1.10.2.4.3. <a id='toc1_10_2_4_3_'></a>[線形和](#toc0_)

行列積を使って線形和を実装すると計算量は$2n^3$ flopsになってしまうが，$\alpha \boldsymbol{a}_1^T + \gamma \boldsymbol{a}_2^T$のように2つの行にそれぞれスカラーを掛けて和を取るだけであれば，1行あたり$3n$ flops（2つのベクトルとスカラーとの積が$2n$，ベクトルの和が$n$）であり，合計$6n$ flopsですむ．

また，影響を与えるのは$p, q$行目（または列目）だけであるため，$2 \times 2$行列

$$
R =
\begin{pmatrix}
\alpha & \beta \\
\gamma & \delta
\end{pmatrix} \in R^{2 \times 2}
$$

を作成し，$A$から$p, q$行（または列）を抜き出した$2 \times n$部分行列に対してだけ行列積を適用するという実装も可能である．この場合の計算量は$8n$ flopsであり，最初の方法（$6n$ flops）とほぼ差がない．


以下のコードでは，2つの行や列に対して定数倍した線形和を効率的に計算する実装を示す．
なお，右辺の計算で一時オブジェクトが生成されるため，`.copy()`は不要である．


In [ ]:
a = 10
b = 0.1
c = 0.1
d = -10
R = np.array([
    [a, b],
    [c, d]
])

with np.printoptions(formatter={'float': '{:12.8f}'.format}):
    n = 5
    A = rng.random(size=(n, n))
    A_org = A.copy()
    print("A\n", A)

    # 行の線形和を取る
    p, q = 1, 2
    A[p], A[q] = a * A[p] + b * A[q], c * A[p] + d * A[q]
    print("P A\n", A)

    # 列の線形和を取る
    p, q = 0, 4
    A[:, p], A[:, q] = a * A[:, p] + b * A[:, q], c * A[:, p] + d * A[:, q]
    print("A P\n", A)


    A = A_org

    # 局所的な行列積による線形和（行）
    p, q = 1, 2
    A[[p, q]] = R @ A[[p, q]]
    print("P A\n", A)

    # 局所的な行列積による線形和（列）
    p, q = 0, 4
    A[:, [p, q]] = A[:, [p, q]] @ R
    print("A P\n", A)


A
 [[  0.80556557   0.09566179   0.15390683   0.60599717   0.11765205]
 [  0.49649903   0.98816569   0.39262669   0.19238206   0.40425573]
 [  0.82420672   0.57894468   0.22592259   0.36295978   0.88325783]
 [  0.34178678   0.59912514   0.64495726   0.13537668   0.40058326]
 [  0.65194149   0.82537345   0.44099039   0.58853199   0.77307944]]
P A
 [[  0.80556557   0.09566179   0.15390683   0.60599717   0.11765205]
 [  5.04741100   9.93955136   3.94885913   1.96011661   4.13088306]
 [ -8.19241734  -5.69063028  -2.21996320  -3.61035955  -8.79215269]
 [  0.34178678   0.59912514   0.64495726   0.13537668   0.40058326]
 [  0.65194149   0.82537345   0.44099039   0.58853199   0.77307944]]
A P
 [[  8.06742094   0.09566179   0.15390683   0.60599717  -1.09596399]
 [ 50.88719832   9.93955136   3.94885913   1.96011661 -40.80408951]
 [-82.80338870  -5.69063028  -2.21996320  -3.61035955  87.10228517]
 [  3.45792616   0.59912514   0.64495726   0.13537668  -3.97165396]
 [  6.59672282   0.82537345   0.4

## 1.11. <a id='toc1_11_'></a>[時間計算量と空間計算量](#toc0_)

これまで計算量をflops（浮動小数点演算の回数）で評価してきたが，実際の計算においてはflopsのような**時間計算量**（計算時間）だけでなく**空間計算量**（メモリ使用量）も重要な要素となる．flopsは理論的な計算量を評価するために使われるが，実際の計算ではさまざまな要因が計算時間やメモリ使用量に影響を与えるため，実際の時間やメモリ使用量を計測することも必要となる．

### 1.11.1. <a id='toc1_11_1_'></a>[notebook における計算時間の計測](#toc0_)


計算機のFLOPS（1秒間に処理できる浮動小数点演算の回数）と計算量（flops）を使って理論的な計算時間を見積もることができるが，実際の計算にはさまざまな要因が影響するため，計算時間を実際に計測することが望ましい．ここでは，notebookで計算時間を計測するための方法について説明する。


計算時間には以下の3つの時間が含まれる．
- **ユーザーCPU時間**: コードの実行にかかる時間．
- **システムCPU時間**: システムやOSが処理にかかる時間．
- **トータルの実行時間（wall time）**: ユーザーが処理を開始してから結果が得られるまでの，I/O処理も含めた，壁掛け時計（wall clock）で測った実際の経過時間．これはユーザーCPU時間とシステムCPU時間を含む．

### 1.11.2. <a id='toc1_11_2_'></a>[計算時間の簡単な計測方法](#toc0_)

notebookでの計算時間を簡単に計測するには、IPythonのマジックコマンド`%time`と`%%time`を用いる。

- `%time`
  - セル内の1行だけの実行時間を計測する．`%time`を行頭に書き，その後に1行のコードを記述する．
- `%%time`
  - セル全体の実行時間を計測する．`%%time`をセルの先頭に書く．

これらを用いると，システムCPU時間，ユーザーCPU時間，およびwall timeが表示される．

- https://ipython.readthedocs.io/en/stable/interactive/magics.html#magic-time
  > Time execution of a Python statement or expression.


In [ ]:
n = 4
%time A = rng.random(size=(n, n))
%time x = rng.random(size=(n))

%time b = A @ x


CPU times: user 29 µs, sys: 5 µs, total: 34 µs
Wall time: 36.7 µs
CPU times: user 18 µs, sys: 2 µs, total: 20 µs
Wall time: 23.6 µs
CPU times: user 30 µs, sys: 5 µs, total: 35 µs
Wall time: 37.9 µs


In [ ]:
%%time

n = 4
A = rng.random(size=(n, n))
x = rng.random(size=(n))

b = A @ x


CPU times: user 79 µs, sys: 0 ns, total: 79 µs
Wall time: 59.1 µs


### 1.11.3. <a id='toc1_11_3_'></a>[実行時間の詳細な計測方法](#toc0_)

システムCPU時間やユーザーCPU時間は様々な要因で時間は毎回変動する．
これらを複数回計測し，結果の平均と標準偏差を取りたい場合は，`%timeit`や`%%timeit`を用いる（itは反復のiterationを意味する）．

使い方は`time`と同じである．結果は平均$\pm$標準偏差で表示される．

- `%timeit`
  - セル内の1行の実行時間を複数回計測し，平均時間を表示する．`%timeit`を行頭に書き，その後に1行のコードを記述する．
- `%%timeit`
  - セル全体の実行時間を複数回計測し，平均時間を表示する．`%%timeit`をセルの先頭に書く．

- https://ipython.readthedocs.io/en/stable/interactive/magics.html#magic-timeit
  > Time execution of a Python statement or expression

オプション：
- `-r R`：計測を行う回数を指定（デフォルトは7回）．
- `-n N`：各回での反復回数を指定（デフォルトは自動設定）．
- `-t`：wall timeを表示する（デフォルト）．
- `-c`：ユーザーCPU時間を表示する．


In [ ]:
%%timeit

n = 4
A = rng.random(size=(n, n))
x = rng.random(size=(n))

b = A @ x


3.2 µs ± 167 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [ ]:
n = 4
A = rng.random(size=(n, n))
x = rng.random(size=(n))

%timeit b = A @ x


1.02 µs ± 20.4 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### 1.11.4. <a id='toc1_11_4_'></a>[メモリ使用量](#toc0_)

現在のpythonプロセス（つまりnotebook）全体が使用しているメモリ使用量を調べるためには，`psutil`モジュールを使用する．`psutil`は，プロセスやシステムのCPU使用率，メモリ使用量，ディスク，ネットワーク，センサーなどの情報を取得するためのライブラリである．

- https://github.com/giampaolo/psutil
    > psutil (process and system utilities) is a cross-platform library for retrieving information on running processes and system utilization (CPU, memory, disks, network, sensors) in Python.

以下のコードは，システム全体のメモリ使用量を調べる例である．


In [ ]:
import psutil

mem = psutil.virtual_memory()

print(f"Total memory    : {mem.total / 1024**3: .2f} GB")
print(f"Used memory     : {mem.used / 1024**3: .2f} GB")
print(f"Free memory     : {mem.free / 1024**3: .2f} GB")
print(f"Available memory: {mem.available / 1024**3: .2f} GB")
print(f"Now using {mem.percent: .2f} % of the memory.")


Total memory    :  31.01 GB
Used memory     :  5.82 GB
Free memory     :  11.37 GB
Available memory:  23.60 GB
Now using  23.90 % of the memory.


ndarray変数が使用しているメモリ量を取得するためには，`.nbytes`プロパティを確認する．このプロパティは，配列の要素が使用しているバイト数を返す．


- https://numpy.org/doc/stable/reference/generated/numpy.ndarray.nbytes.html
    > Total bytes consumed by the elements of the array.


以下のコードでは，ndarray `A`の要素がどれだけのメモリを使用しているかをバイト，KB，MBで表示している．

In [ ]:
N = 50
M = 300
A = rng.random((N, M))

print(f"{N}x{M}次元の行列Aは{N * M}個の要素を持ちます．")
print(f"各要素の型はfloat64なので，64ビットつまり8バイトを占めます．")
print(f"したがってAの要素に必要なメモリ使用量は{N * M * 8}バイトです．")
print(f"Aのメモリ使用量：{A.nbytes} bytes"
    f" = {A.nbytes / 1024:.4f} KB"
    f" = {A.nbytes / 1024**2:.4f} MB")


50x300次元の行列Aは15000個の要素を持ちます．
各要素の型はfloat64なので，64ビットつまり8バイトを占めます．
したがってAの要素に必要なメモリ使用量は120000バイトです．
Aのメモリ使用量：120000 bytes = 117.1875 KB = 0.1144 MB


`ndarray`には要素のデータ以外にも情報が含まれている．そのため，オブジェクト全体のメモリ使用量を調べるには，`sys.getsizeof()`を使用する．


- https://docs.python.org/ja/3/library/sys.html#sys.getsizeof
    > Return the size of an object in bytes.


以下の例では，ndarray `A`が全体で使用しているメモリ量を確認できる．


In [ ]:
import sys
A_nbytes = sys.getsizeof(A)

print(f"Aのメモリ使用量：{A.nbytes} bytes")
print(f"Aの実際のメモリ使用量：{A_nbytes} bytes")
print(f"Aの要素以外のデータに必要なメモリ使用量：{A_nbytes - A.nbytes} bytes")


Aのメモリ使用量：120000 bytes
Aの実際のメモリ使用量：120128 bytes
Aの要素以外のデータに必要なメモリ使用量：128 bytes


notebook内で使用しているすべての変数のメモリ使用量を確認するには，IPythonのマジックコマンド`%whos`を使用する．

- https://ipython.readthedocs.io/en/stable/interactive/magics.html#magic-whos
    > Like %who, but gives some extra information about each variable.


In [ ]:
%whos


Variable              Type         Data/Info
--------------------------------------------
A                     ndarray      50x300: 15000 elems, type `float64`, 120000 bytes (117.1875 kb)
A_col                 ndarray      5x3: 15 elems, type `float64`, 120 bytes
A_lower_hessenberg    ndarray      5x5: 25 elems, type `float64`, 200 bytes
A_nbytes              int          120128
A_org                 ndarray      5x5: 25 elems, type `float64`, 200 bytes
A_row                 ndarray      5x3: 15 elems, type `float64`, 120 bytes
A_strict_tri_lower    ndarray      5x5: 25 elems, type `float64`, 200 bytes
A_strict_tri_upper    ndarray      5x5: 25 elems, type `float64`, 200 bytes
A_tri_lower           ndarray      5x5: 25 elems, type `float64`, 200 bytes
A_tri_upper           ndarray      5x5: 25 elems, type `float64`, 200 bytes
A_upper_hessenberg    ndarray      5x5: 25 elems, type `float64`, 200 bytes
B                     ndarray      3x4: 12 elems, type `float64`, 96 bytes
C         

## 1.12. <a id='toc1_12_'></a>[密行列と疎行列](#toc0_)

行列は，その要素の多くが非ゼロか，ゼロかによって**密行列**と**疎行列**に分類される．

### 1.12.1. <a id='toc1_12_1_'></a>[密行列とは](#toc0_)

**密行列**（dense matrix）とは，行列の要素の大部分が非ゼロ（0ではない値）である行列を指す．数値計算において一般的な行列は密行列であり，通常はそのまま計算することが多い．ただし，密行列が大規模になると，メモリを大量に消費し，計算にも膨大な時間を要することがあり，そもそも行列をメモリに保持することも困難な場合がある．

### 1.12.2. <a id='toc1_12_2_'></a>[疎行列とは](#toc0_)

一方，**疎行列**（sparse matrix）は，行列の大部分の要素がゼロで，非ゼロ要素が少ない行列である．大規模な問題を扱うときには，疎行列の特性を生かして計算を効率化することが多い．

疎行列は専用のデータ構造で効率的にメモリに要素を格納する．さらに，疎行列に特化した計算方法を使うことで，大規模な計算を高速に行うことができる．

例えば，疎行列の逆行列は一般に密行列になるため，密行列用のアルゴリズムでは計算が困難になる場合が多い．そのため，疎行列には専用の解法やアルゴリズムが存在する．

- https://en.wikipedia.org/wiki/Sparse_matrix
    > In numerical analysis and scientific computing, a sparse matrix or sparse array is a matrix in which most of the elements are zero.

疎行列には，以下のような応用がある．

- **[ページランクアルゴリズム](https://ja.wikipedia.org/wiki/%E3%83%9A%E3%83%BC%E3%82%B8%E3%83%A9%E3%83%B3%E3%82%AF)** ：ウェブページのリンク構造を表現する隣接行列では，ページ間にリンクが存在する要素だけが非ゼロとなる．
- **[協調フィルタリング](https://ja.wikipedia.org/wiki/%E5%8D%94%E8%AA%BF%E3%83%95%E3%82%A3%E3%83%AB%E3%82%BF%E3%83%AA%E3%83%B3%E3%82%B0)** ：推薦アルゴリズムで用いられるユーザー・アイテム行列において，購入や評価の情報がある部分のみが非ゼロとなる．
- **[有限要素法（FEM）](https://ja.wikipedia.org/wiki/%E6%9C%89%E9%99%90%E8%A6%81%E7%B4%A0%E6%B3%95)** ：係数行列において，隣接する解析要素に対応する要素のみが非ゼロとなる．
- **コンピュータグラフィックス**：[ラジオシティ法](https://ja.wikipedia.org/wiki/%E3%83%A9%E3%82%B8%E3%82%AA%E3%82%B7%E3%83%86%E3%82%A3)による照明計算において，係数行列は間接光が届く微小表面に対応する要素だけが非ゼロとなる．

疎行列では，非ゼロ要素の数をnnz（number of non-zeros）と呼ぶ．

- **密行列**の場合，メモリ使用量は$O(n^2)$である．
- **疎行列**の場合，メモリ使用量は`nnz`に比例し，$O(n)$よりも少ないことが多い．

### 1.12.3. <a id='toc1_12_3_'></a>[疎行列の格納方法](#toc0_)

疎行列を効率的に格納する方法として，次の3つの代表的なフォーマットがある．

- **COO形式**（COOrdinate）：非ゼロ要素の行，列，値のリストを保持する．
- **CSR形式**（Compressed Sparse Row）：行のインデックスと非ゼロ要素の値を圧縮して格納する形式．
- **CSC形式**（Compressed Sparse Column）：CSR形式の列バージョンで，列ごとに非ゼロ要素を格納する．


以下ではscipyを使って，疎行列のCOO形式とCSR形式を扱う方法を説明する．


- https://en.wikipedia.org/wiki/Sparse_matrix#Coordinate_list_(COO)
    > COO stores a list of (row, column, value) tuples.
- https://en.wikipedia.org/wiki/Sparse_matrix#Compressed_sparse_row_(CSR,_CRS_or_Yale_format)
    > The compressed sparse row (CSR) or compressed row storage (CRS)
- https://en.wikipedia.org/wiki/Sparse_matrix#Compressed_sparse_column_(CSC_or_CCS)
    > CSC is similar to CSR except that values are read first by column, a row index is stored for each value, and column pointers are stored.




### 1.12.4. <a id='toc1_12_4_'></a>[密行列のメモリ使用量の概算](#toc0_)

まず，$n$次正方行列$A$を保持するために必要なメモリ使用量を計算する．行列の要素が`float64`型（64ビット，つまり8バイト）の場合，必要なメモリ量は`8 * n * n`バイトである．

具体的なメモリ使用量は、ndarrayの属性を使って確認できる．

- `.dtype`：要素の型
- `.itemsize`：要素1つあたりのバイト数
- `.size`：全要素数（列数×行数）

以下は具体的なコード例で，行列`A`のメモリ使用量を表示する．


In [ ]:
N = 5
M = 3
rng = np.random.default_rng()
A = rng.random((N, M))

print("A.dtype:", A.dtype)  # 要素の型を表示
print("A.itemsize:", A.itemsize)  # 1要素あたりのバイト数を表示
print("A.size:", A.size)  # 要素数（全要素数）を表示

# メモリ使用量を計算
print(f"A is using {A.itemsize * A.size} bytes")  # 要素ごとのメモリ使用量
print(f"A is using {A.nbytes} bytes")  # 全体のメモリ使用量
print(f"A is using {sys.getsizeof(A)} bytes including non-element information")  # 要素以外の情報も含めたメモリ使用量


A.dtype: float64
A.itemsize: 8
A.size: 15
A is using 120 bytes
A is using 120 bytes
A is using 248 bytes including non-element information


密行列のサイズが大きくなると，メモリ使用量も急激に増加する．
`float64`型（1要素あたり8バイト）を仮定し，例えば$30000 \times 30000$の行列ではメモリ使用量が約7GBになる．さらに，作業用メモリも必要になるため，通常のノートPCではこれ以上大きな行列を扱うのは困難である．
大型計算機サーバではTB単位のメモリを搭載しているものもあるが，$n$が更に大きくなると計算は現実的ではない．

以下のコードでは，行列サイズに応じたメモリ使用量をMB，GB，TB単位で表示している．


In [ ]:
itemsize = 8  # float64のサイズ
for n in [100, 1000, 10000, 30000, 100000, 1000000, 10000000]:
    nbytes = itemsize * n**2  # 行列サイズに応じたメモリ使用量を計算
    print(
        f"{n}x{n} matrix needs {nbytes / 1024**2:.02f} MB"  # メモリ使用量をMB単位で表示
        f" = {nbytes / 1024**3:.02f} GB"  # メモリ使用量をGB単位で表示
        f" = {nbytes / 1024**4:.02f} TB"  # メモリ使用量をTB単位で表示
    )


100x100 matrix needs 0.08 MB = 0.00 GB = 0.00 TB
1000x1000 matrix needs 7.63 MB = 0.01 GB = 0.00 TB
10000x10000 matrix needs 762.94 MB = 0.75 GB = 0.00 TB
30000x30000 matrix needs 6866.46 MB = 6.71 GB = 0.01 TB
100000x100000 matrix needs 76293.95 MB = 74.51 GB = 0.07 TB
1000000x1000000 matrix needs 7629394.53 MB = 7450.58 GB = 7.28 TB
10000000x10000000 matrix needs 762939453.12 MB = 745058.06 GB = 727.60 TB



このように，行列のサイズが大きくなると，メモリ使用量も指数的に増加するため，密行列を扱う場合には注意が必要である．特に，$n=100000$を超えるサイズの行列では，メモリ使用量が数十GBから数TBに達し，通常の計算機環境では扱えなくなる．このため，大規模な行列を扱う際には，疎行列（sparse matrix）の使用が推奨される場合が多い．

### 1.12.5. <a id='toc1_12_5_'></a>[scipyにおける疎行列](#toc0_)

`scipy.sparse`では、疎行列を効率的に扱うための複数のデータ格納形式がサポートされている．ここでは、**CSR形式**と**COO形式**を中心に説明する．

- https://docs.scipy.org/doc/scipy/reference/sparse.html
    > SciPy 2-D sparse array package for numeric data.
    - https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csr_array.html
        > Compressed Sparse Row array.
    - https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.coo_array.html
        > A sparse array in COOrdinate format.


#### 1.12.5.1. <a id='toc1_12_5_1_'></a>[CSR形式](#toc0_)

**CSR（Compressed Sparse Row）形式**は，行方向に非ゼロ要素を格納する形式であり，行列とベクトルの積を効率的に計算するために適している．CSR形式は，以下の3つの配列で表現される．

- **indptr**：各行の非ゼロ要素の開始位置を示す配列．
- **indices**：非ゼロ要素が存在する列のインデックスを示す配列．
- **data**：非ゼロ要素の実際の値を格納する配列．

以下のコードは、密行列をCSR形式に変換し、さらにCSR形式の疎行列を元の密行列に戻す例である．

In [ ]:
# 3x3密行列の例
dense_matrix = np.array(
    [[1, 0, 2],
     [0, 0, 3],
     [4, 5, 6]]
)

print("dense matrix A")
print(dense_matrix)
print()

# CSR形式に変換
A_csr = scipy.sparse.csr_array(dense_matrix)

print("CSR sparce matrix A")
print("indptr:", A_csr.indptr)
print("indices:", A_csr.indices)
print("data:", A_csr.data)
print("nnz:", A_csr.nnz)
print()

# CSR疎行列を元の密行列に戻す
A_dense = A_csr.toarray()
print("dense matrix A")
print(A_dense)


dense matrix A
[[1 0 2]
 [0 0 3]
 [4 5 6]]

CSR sparce matrix A
indptr: [0 2 3 6]
indices: [0 2 2 0 1 2]
data: [1 2 3 4 5 6]
nnz: 6

dense matrix A
[[1 0 2]
 [0 0 3]
 [4 5 6]]


このコードでは，以下の手順を行っている．

1. `scipy.sparse.csr_array()`を使用して，ndarrayCSR形式に変換する．
2. 変換されたCSR疎行列の情報（indptr, indices, data）を表示する．
3. `.nnz`で非ゼロ要素の数を確認する．
4. `.toarray()`を用いてCSR疎行列を元のndarrayに戻す．



##### 1.12.5.1.1. <a id='toc1_12_5_1_1_'></a>[CSR疎行列の情報：indptr, indices, data](#toc0_)

CSR疎行列の内部構造を理解するために，各配列の意味をさらに詳しく説明する．

- **indptr**（行へのポインタ）：各行の非ゼロ要素が`data`配列のどの位置にあるかを示す．
    - 例えば`[0, 2, 3, 6]`は，行0から行2までの非ゼロ要素の範囲を示している．
    - 行0は`data[0:2]`，行1は`data[2:3]`，行2は`data[3:6]`に非ゼロ要素が格納されている．
    - 一般的に，`data[indptr[i]:indptr[i+1]]`が行`i`の非ゼロ要素の値である．

- **indices**（列のインデックス）：各非ゼロ要素が属する列を示す．
    - 例えば`[0, 2, 2, 0, 1, 2]`は，非ゼロ要素がそれぞれどの列にあるかを示している．
    - 一般的に，`indices[indptr[i]:indptr[i+1]]`の範囲が行`i`の非ゼロ要素の列インデックスである．

- **data**（値）：各非ゼロ要素の値そのものを保持している．
    - 例えば`[1, 2, 3, 4, 5, 6]`は，各非ゼロ要素の実際の値を保持している．


<!-- 
上記のコードの結果を解釈すると，以下のようになる．

- `indptr`は`[0, 2, 3, 6]`なので，行列には3行ある．
    - これを`[0, 2], [2, 3], [3, 6]`とみて，3行それぞれを以下のように解釈する．
- 行0：`[indptr[0]:indptr[1]] = [0:2]`なので
    - 行0の非ゼロ要素の列インデックスは`indices[0:2] = [0, 2]`
    - 行0の非ゼロ要素の値は`data[0:2] = [1, 2]`
    - したがって，行0の非ゼロ要素は`[0, 0] = 1`と`[0, 2] = 2`
- 行1：`[indptr[1]:indptr[2]] = [2:3]`なので
    - 行1の非ゼロ要素の列インデックスは`indices[2:3] = [2]`
    - 行1の非ゼロ要素の値は`data[2:3] = [3]`
    - したがって，行0の非ゼロ要素は`[1, 2] = 3`
- 行2：`[indptr[2]:indptr[3]] = [3:6]`なので
    - 行1の非ゼロ要素の列インデックスは`indices[3:6] = [0, 1, 2]`
    - 行1の非ゼロ要素の値は`data[3:6] = [4, 5, 6]`
    - したがって，行0の非ゼロ要素は`[2, 0] = 4`と`[2, 1] = 5`と`[2, 2] = 6`
-->

つまり`indptr`が`[0, 2, 3, 6]`であれば，

- `data`の`[1, 2, 3, 4, 5, 6]`を`[1, 2], [3], [4, 5, 6]`に
- `indptr`の`[0, 2, 2, 0, 1, 2]`を`[0, 2], [2], [0, 1, 2]`に

分割して，それぞれの行の非ゼロ要素を表現する．
このように行ごとに非ゼロ要素を圧縮して格納するため，
CSR（Compressed Sparse Row）形式と呼ばれる．

一方，列方向の操作に特化した**CSC形式**（Compressed Sparse Column）は，CSR形式と似た構造であるが，列ごとに圧縮する形式である．


#### 1.12.5.2. <a id='toc1_12_5_2_'></a>[COO形式](#toc0_)

**COO形式**（COOrdinate）は，各非ゼロ要素をその行インデックス(`row_index`)，列インデックス(`col_index`)，値(`data`)の3つで表現するシンプルなフォーマットであり，要素の追加や削除が多い場合に便利な形式である．
COO形式に変換するには`scipy.sparse.coo_array()`を用いる．

- https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.coo_array.html
    > A sparse array in COOrdinate format.

次のコードは，密行列をCOO形式に変換し，また密行列に戻す例である．


In [ ]:
dense_matrix = np.array(
    [[1, 0, 2],
     [0, 0, 3],
     [4, 5, 6]]
)

print("dense matrix A")
print(dense_matrix)
print()

# COO形式に変換
A_coo = scipy.sparse.coo_array(dense_matrix)

# 非ゼロ要素の行インデックス、列インデックス、値を取得
# row_index, col_index = A_coo.coords  # scipy 1.13 or newer
row_index, col_index = A_coo.row, A_coo.col  # scipy 1.12 or older
data = A_coo.data

# 各インデックスとデータの表示
print("COO sparce matrix A")
print("col_index:", col_index)
print("row_index:", row_index)
print("data:", data)
print("nnz:", A_coo.nnz)
print()


# COO形式の疎行列を密行列に変換
A_dense = A_coo.toarray()
print("dense matrix A")
print(A_dense)

dense matrix A
[[1 0 2]
 [0 0 3]
 [4 5 6]]

COO sparce matrix A
col_index: [0 2 2 0 1 2]
row_index: [0 0 1 2 2 2]
data: [1 2 3 4 5 6]
nnz: 6

dense matrix A
[[1 0 2]
 [0 0 3]
 [4 5 6]]


このコードは，次の手順を行っている．

1. `scipy.sparse.coo_array()`を用いてndarrayからCOO形式の疎行列へ変換する．
2. COO疎行列の各情報（`.row`，`.col`，`.data`）を表示する．
3. `.nnz`を使って非ゼロ要素の個数を確認する．
4. `.toarray()`を使用してCOO疎行列を元のndarrayに戻す．


##### 1.12.5.2.1. <a id='toc1_12_5_2_1_'></a>[COO疎行列の情報](#toc0_)

COO形式の疎行列は，次の3つの配列で構成されている．

- **行インデックス**（`row_index`）：各非ゼロ要素の行位置を示す．
- **列インデックス**（`col_index`）：各非ゼロ要素の列位置を示す．
- **データ**（`data`）：非ゼロ要素の実際の値を保持する．

上記の例では，次のように表示される．

- 行インデックス：`[0, 0, 1, 2, 2, 2]`
- 列インデックス：`[0, 2, 2, 0, 1, 2]`
- データ（非ゼロ要素の値）：`[1, 2, 3, 4, 5, 6]`

この出力からわかるように，非ゼロ要素は次の位置に存在する．

- `[0, 0]`要素の値は`1`，
- `[0, 2]`要素の値は`2`，
- `[1, 2]`要素の値は`3`，
- `[2, 0]`要素の値は`4`，
- `[2, 1]`要素の値は`5`，
- `[2, 2]`要素の値は`6`，

つまり，行インデックス・列インデックス・値が対応する3つ組として，それぞれの非ゼロの位置と値を表している．


#### 1.12.5.3. <a id='toc1_12_5_3_'></a>[大規模疎行列の例](#toc0_)

[Googleページランク](https://ja.wikipedia.org/wiki/%E3%83%9A%E3%83%BC%E3%82%B8%E3%83%A9%E3%83%B3%E3%82%AF)で扱うインターネット上のwebサイト同士のグラフ隣接行列$A$は，
行と列のインデックスが1つのwebサイト（グラフの頂点，ノード）を表しており，
サイト$i$とサイト$j$にリンク（グラフの辺，エッジ）があれば$a_{ij}=1$，そうでなければ0である．


[Googleページランク](https://ja.wikipedia.org/wiki/%E3%83%9A%E3%83%BC%E3%82%B8%E3%83%A9%E3%83%B3%E3%82%AF)で扱われているインターネット上のwebサイト間のリンク構造は，グラフ隣接行列として表現される．この隣接行列$A$では，行と列のインデックスがwebサイト（ノード）を表し，サイト$i$がサイト$j$にリンクしている場合は$a_{ij}=1$，リンクがない場合は$a_{ij}=0$となる．

この隣接行列は非常に大規模ではあるが，リンクされているwebサイトは非常に少ないため，疎行列として扱うことができる．以下では，Google Web Graphのデータセットを使用して，そのような大規模な疎行列を扱う例を示す．このデータセットは，916,427個のノード（webサイト）と，5,105,039個のリンク（エッジ）を持っている．


- Google Web Graph https://www.kaggle.com/datasets/pappukrjha/google-web-graph
    - License CC0: Public Domain


データをダウンロードするためのコードは以下の通りである．このコードはデータを指定されたURLからダウンロードし、ファイルとして保存する．
なお上記のKaggleはログインが必要なため，[他のサイトのURL](https://github.com/yuhc/web-dataset/tree/master/web-Google)からダウンロードしている．



In [16]:
import requests
import os

def download(url: str, filename: str) -> None:
    """download a file from Internet

    Download a file from `url` and save to `filename` if `filename` does not exist.

    Args:
        url (str): URL (http://***)
        filename (str): local filename
    """
    if os.path.exists(filename):
        print("already downloaded.")
    else:
        headers={'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0.1 Safari/605.1.15'}
        print("start downloading....")
        with open(filename, 'wb') as saveFile:
            saveFile.write(requests.get(url, headers=headers).content)
        print("file downloaded.")


In [17]:
download(
    'https://github.com/yuhc/web-dataset/raw/master/web-Google/web-Google.txt',
    'web-Google.txt'
)


already downloaded.


##### 1.12.5.3.1. <a id='toc1_12_5_3_1_'></a>[ファイルの読み込みと疎行列への変換](#toc0_)

このファイルはcsvであり，`np.loadtxt`でndarrayとして読み込める．

このファイルはCSV形式であり，`np.loadtxt`を使って`ndarray`として読み込むことができる．`np.loadtxt`はテキストファイルからデータを読み込み，ndarrayに変換する関数である．

- https://numpy.org/doc/stable/reference/generated/numpy.loadtxt.html
    > Load data from a text file.

このデータは，インターネット上のウェブサイト同士のリンクを表す**グラフの隣接行列**であり，各列にはリンクがあるウェブサイト同士の番号（ノード番号）が記録されている．

次のコードでデータを読み込み，サイズなどを確認できる．

In [19]:
data = np.loadtxt("web-Google.txt", dtype=np.int64)
data, data.shape, data.max(), data.nbytes / 1024**2


(array([[     0,  11342],
        [     0, 824020],
        [     0, 867923],
        ...,
        [916425, 632916],
        [916425, 637936],
        [916425, 837379]], shape=(5105039, 2)),
 (5105039, 2),
 np.int64(916427),
 77.89671325683594)

ここで，`data`の0列目は各非ゼロ要素の**行インデックス**，1列目は**列インデックス**を表している．隣接行列において，非ゼロ要素はリンクの有無を0または1で表すため，非ゼロ要素の値はすべて1である．

このデータ構造はCOO形式に対応しているため，`scipy.sparse.csr_array()`または`scipy.sparse.coo_array()`を用いて
以下のように疎行列`A`に変換することができる．

これらの関数に与える引数`arg1`は，以下の2つをタプルにまとめたものである．

- 非ゼロ要素の値`val_data`（ここではすべて1）
- 非ゼロ要素の**行インデックス**と**列インデックス**を示すタプル`(row_index, col_index)`

In [20]:
row_index = data[:, 0]
col_index = data[:, 1]
val_data = np.ones(len(data))

A = scipy.sparse.csr_array(
    arg1=(
        val_data,
        (row_index, col_index)
    )
)

# A = scipy.sparse.coo_array(
#     arg1=(
#         val_data,
#         (row_index, col_index)
#     )
# )


以下のコードで確認できるように，
この時点で，疎行列`A`は916,428×916,428の巨大な行列であり，非ゼロ要素（リンクの数）は5,105,039個である．


In [ ]:
A.shape, A.nnz, A.ndim


((916428, 916428), 5105039, 2)

##### 1.12.5.3.2. <a id='toc1_12_5_3_2_'></a>[メモリ使用量](#toc0_)


疎行列`A`は，916,428×916,428のサイズを持つが，非ゼロ要素のみを格納するため，実際に必要なメモリは非常に少ない．もしこの行列を密行列として扱うと，膨大なメモリが必要になる．以下のコードで，密行列に変換した場合のメモリ使用量を計算してみる．


In [ ]:
itemsize = 8
m, n = A.shape
nbytes = itemsize * m * n
print(
    f"{n}x{n} matrix needs {nbytes / 1024**2:.02f} MB"
    f" = {nbytes / 1024**3:.02f} GB"
    f" = {nbytes / 1024**4:.02f} TB"
)


916428x916428 matrix needs 6407472.83 MB = 6257.30 GB = 6.11 TB


この結果，密行列として扱うと6TBものメモリが必要になることがわかる．

一方，疎行列として扱う場合，実際に必要なメモリは非ゼロ要素の数に依存するため，38MB程度で済むことが確認できる．


In [ ]:
print(
    f"A uses {A.dtype.itemsize} bytes * {A.nnz} nnz elements"
    f" = {A.dtype.itemsize * A.nnz// 1024**2}MB"
)


A uses 8 bytes * 5105039 nnz elements = 38MB


## 1.13. <a id='toc1_13_'></a>[数値計算ライブラリとの関係](#toc0_)

今後も演習の資料では，**科学技術計算のための数値計算アルゴリズム**について解説していくが，ここでまず覚えておくべき重要なポイントがある．それは次の考え方である．

- **数値計算のアルゴリズムを自分で実装して使ってはいけない**

もちろん，アルゴリズムを自分で実装して，その内部動作を理解することは非常に重要であり，この演習の資料もその理解を助けるために作成されている．実際に数値計算の仕組みを学ぶためには，自分でコードを書いて実験し，アルゴリズムの特性を把握することが有用である．

しかし，実際の応用においては，信頼性の高い数値計算ライブラリを使用することが推奨される．なぜなら，これらのライブラリは以下の点で優れているからである．

1. **数値的に安定**：多くのアルゴリズムには，計算誤差を最小限に抑える工夫がなされている．
2. **高速な動作**：長年にわたり最適化が進められ，非常に効率的に動作する．
3. **信頼性の高い実績**：世界中の研究者や技術者が長い間使用しており，その有用性が証明されている．

例えば，numpyやscipyは，pythonで数値計算を行うための代表的なライブラリであり，これらの関数を十分に活用することが非常に重要である．これらのライブラリに用意されている機能を使わずに，同じものを自分で一から作ることは，いわゆる「車輪の再発明」になりがちであり，結果として精度や計算速度において劣る可能性が高い．

さらに，これらのライブラリ（numpyやscipy）では，内部的に**BLAS**（Basic Linear Algebra Subprograms）や**LAPACK**（Linear Algebra PACKage）といった，密行列計算のために最適化された低レベルライブラリを利用している．これらのライブラリは，線形代数計算の標準として長年にわたり発展してきたものであり，効率的かつ正確に計算を行うことができる．

したがって，数値計算を行う際は，これらの信頼性のあるライブラリを積極的に活用することが推奨される．


### 1.13.1. <a id='toc1_13_1_'></a>[BLAS](#toc0_)

**BLAS**（Basic Linear Algebra Subprograms）は，基本的な行列計算を効率的に行うためのライブラリである．BLASは，数値線形代数の基礎的な演算を提供する標準的なAPIであり，ベクトルや行列に対する基本的な演算をサポートしている．


- https://www.netlib.org/blas/
    > The BLAS (Basic Linear Algebra Subprograms) are routines that provide standard building blocks for performing basic vector and matrix operations.
- https://ja.wikipedia.org/wiki/Basic_Linear_Algebra_Subprograms
    > Basic Linear Algebra Subprograms（BLAS）は数値線形代数の基礎的演算に必要な関数を定義するAPIである。

BLASは3つのレベルのAPIを定義しており，演算の種類によって次のように分類される．

- **Level 1**: ベクトルの積和計算などの基本的なベクトル演算．
- **Level 2**: 行列とベクトルの積．
- **Level 3**: 行列同士の積を含む高次の演算．

BLAS公式の参照実装はFORTRANおよびC言語で提供されている．
C言語の実装は**CBLAS**と呼ばれ，そのヘッダファイルは`cblas.h`である．


#### 1.13.1.1. <a id='toc1_13_1_1_'></a>[BLASの高速化](#toc0_)

BLASが提供する演算，例えば行列積やベクトル積和演算は，並列計算が可能であり，CPUやGPUの**SIMD拡張命令**を活用して大幅に高速化されている．
SIMD拡張命令には，
[Intel x86系のSSE](https://www.intel.co.jp/content/www/jp/ja/support/articles/000005779/processors.html)，
[x64系のAVX](https://www.intel.co.jp/content/www/jp/ja/architecture-and-technology/avx-512-overview.html)，
[ARMのNEON](https://www.arm.com/ja/technologies/neon)
などがあり，
[スーパーコンピュータ富岳](https://ja.wikipedia.org/wiki/%E5%AF%8C%E5%B2%B3_(%E3%82%B9%E3%83%BC%E3%83%91%E3%83%BC%E3%82%B3%E3%83%B3%E3%83%94%E3%83%A5%E3%83%BC%E3%82%BF)#%E3%83%97%E3%83%AD%E3%82%B0%E3%83%A9%E3%83%9F%E3%83%B3%E3%82%B0%E8%A8%80%E8%AA%9E%E3%81%A8%E3%83%A9%E3%82%A4%E3%83%96%E3%83%A9%E3%83%AA)
にもBLASがある．

以下のような各社のBLAS実装が存在し，特定のハードウェアで最適なパフォーマンスを引き出すことができるように最適化されている．


- [Intel Math Kernel Library (MKL)](https://www.intel.com/content/www/us/en/developer/tools/oneapi/onemkl.html): Intelベースのシステム向けに最適化された数学ライブラリ．
- [AMD AOCL-BLAS](https://www.amd.com/en/developer/aocl/dense.html): AMD向けに高性能なBLASの実装．
- [Apple vecLib](https://developer.apple.com/documentation/accelerate/veclib/): AppleのBLAS実装で，ベクトル計算に最適化．
- [NVIDIA cuBLAS](https://developer.nvidia.com/cublas): NVIDIAのGPUを利用したBLAS実装で，AIや高性能計算向けに最適化されている．
- [NEC Numeric Library Collection](https://sxauroratsubasa.sakura.ne.jp/documents/sdk/SDK_NLC/UsersGuide/blas/f/ja/index.html): NECのスーパーコンピュータ向けのBLAS実装を含むライブラリ．
- [FUJITSU Software Compiler Package](https://software.fujitsu.com/jp/manual/manualfiles/m200007/j2ul2595/02z000/j2ul-2595-02z0.pdf): 富士通のサーバ向けに最適化されたBLASライブラリ．
- [IBM Basic Linear Algebra Subprograms (BLAS) and C BLAS (CBLAS)](https://www.ibm.com/docs/en/essl/6.3?topic=egr-basic-linear-algebra-subprograms-blas-c-blas-cblas): IBMの高性能計算向けライブラリ．


#### 1.13.1.2. <a id='toc1_13_1_2_'></a>[オープンソース実装](#toc0_)

現在利用されているオープンソースのBLAS実装として，次の3つが挙げられる．

- [OpenBLAS](https://www.openblas.net/)：手作業で最適化された[GotoBLAS2](https://en.wikipedia.org/wiki/GotoBLAS)をベースに開発が始まり，現在は様々なプラットフォームで広く使われている．
- [ATLAS](https://en.wikipedia.org/wiki/Automatically_Tuned_Linear_Algebra_Software)（Automatically Tuned Linear Algebra Software）：OpenBLAS以前に広く利用されていた実装．CPUに応じて自動的に最適化されたコードを生成するため，様々なプラットフォームに対応することができる．
- [BLIS](https://github.com/flame/blis)（BLAS-like Library Instantiation Software）：近年利用が広まっている，BLASの上位互換として開発された，より柔軟かつ効率的なBLASライクな実装．


#### 1.13.1.3. <a id='toc1_13_1_3_'></a>[BLASの代表的なAPI](#toc0_)

BLASの代表的な演算には，密行列同士の積を行うGEMM（general matrix multiply）がある．GEMMは，行列同士の積を効率的に行うための基本的な操作であり，数値計算において非常に重要である．GEMMは，行列のデータ型（単精度，倍精度，実数，複素数）に応じて次のように分類されている．

- SGEMM：**単精度**実数用の行列積．
- DGEMM：**倍精度**実数用の行列積．
- CGEMM：単精度**複素数**用の行列積．
- ZGEMM：倍精度**複素数**用の行列積．

また，行列の形状や性質に応じて他にも様々なAPIが用意されている．例えば，対称行列や三角行列に特化したAPIがある．

- SYMM：**対称**行列用の行列積（symmetric matrix multiply）．
- TRMM：**三角**行列用の行列積（triangular matrix multiply）．

さらに，ベクトルと行列の積に特化したAPIも存在する．これには，一般行列や帯行列，対称帯行列などに対応したAPIが含まれる．

- GEMV：**一般**行列とベクトルの積（general matrix-vector multiply）．
- SYMV：**対称**行列とベクトルの積（symmetric matrix-vector multiply）．
- GBMV：一般**帯行列**とベクトルの積（general band matrix-vector multiply）．
- SBMV：**対称帯行列**とベクトルの積（symmetric band matrix-vector multiply）．

これらのAPIを使うことで，不要な計算を減らし，特定の行列やベクトルの性質に基づいた効率的な計算が可能になる．BLASを使用する際には，操作する行列の種類（対称行列，三角行列，帯行列など）に応じて最適なAPIを選ぶことで，計算の効率を最大限に引き出すことができる．





#### 1.13.1.4. <a id='toc1_13_1_4_'></a>[numpyとBLAS](#toc0_)


numpyで行列積を行う際，`@`演算子を使用するが，これは実際には`np.matmul()`関数が呼び出されている．この`np.matmul()`は内部でBLAS（Basic Linear Algebra Subprograms）を利用して，効率的に行列積を計算している．

- [numpy.matmul](https://numpy.org/doc/stable/reference/generated/numpy.matmul.html)
  > Matrix product of two arrays.

[numpyのコードを確認すると](https://github.com/numpy/numpy/blob/0aaa5d397f86d5b4aef9c83053f65296a790e501/numpy/core/src/umath/matmul.c.src#L157)，例えば自分自身との積（$A A^T$）では`SYRK`（symmetric rank-k update）を使用し，一般的な行列積では`GEMM`を呼び出している．

BLASでは，三角行列や帯行列のように0が多い行列との積に特化した最適化が用意されている．しかし，numpyで行列積を計算する場合，こうした最適化は基本的には適用されない．そのため，0との不要な計算が行われてしまうことがある．

BLASのような低レベルライブラリは柔軟で効率的だが，扱いが難しい場合がある．一方，numpyのような高レベルのラッパーは使いやすい反面，最適化の自由度が低くなることがある．この自由度と使いやすさのトレードオフは，numpyだけでなく多くの数値計算ライブラリに共通するものである．

この資料では，numpyの高レベルAPIを利用しながら，その利便性を最大限に活かす方針を取る．例えば，理論上はflops（演算量）は少ないという説明をするが，実際の計算ではflopsが多くなることがあるという点は理解しておいた方が良い．



### 1.13.2. <a id='toc1_13_2_'></a>[LAPACK](#toc0_)

**LAPACK**（Linear Algebra PACKage）は，BLASを基盤として高度な線形代数の問題（連立方程式や固有値計算，特異値分解）を解くためのソルバーを提供するライブラリである．

- https://www.netlib.org/lapack/
  > LAPACK is written in Fortran 90 and provides routines for solving systems of simultaneous linear equations, least-squares solutions of linear systems of equations, eigenvalue problems, and singular value problems. (...)
  > LAPACK routines are written so that as much as possible of the computation is performed by calls to the Basic Linear Algebra Subprograms (BLAS).
- https://ja.wikipedia.org/wiki/LAPACK
  > LAPACK (Linear Algebra PACKage)は数値線形代数のための数値解析ソフトウェアライブラリで、線型方程式や線型最小二乗問題、固有値問題、特異値問題等を数値的に解くために利用される。

BLASの実装にはIntel MKLやOpenBLASがあるが，これらはLAPACKの実装も提供しており，特定のハードウェアにおいて最適化された線形代数のソルバーを使うことができる．



#### 1.13.2.1. <a id='toc1_13_2_1_'></a>[LAPACKのAPI](#toc0_)

LAPACKもBLASと同様に，行列の種類（`GE`（一般行列）, `SY`（対称行列）, `TR`（三角行列）など）に応じてAPIが提供されている．また，LAPACKには同じ問題を解くために複数のアルゴリズム（ドライバ）が実装されているため，計算内容に応じて最適なルーチンを選択することができる．

LAPACKのユーザーガイドには，さまざまなドライバルーチンについての説明がある．

- LAPACK Users' Guide Third Edition, Driver Routines, https://www.netlib.org/lapack/lug/node25.html
  > This section describes the driver routines in LAPACK.





### 1.13.3. <a id='toc1_13_3_'></a>[numpyのBLAS・LAPACK](#toc0_)


numpyやscipyを使って連立方程式を解いたり，固有値の計算を行う場合，内部では適切なLAPACKのルーチンが呼び出されている．

numpyによる処理を本当に高速に行うためには，BLASおよびLAPACKのライブラリが，使用しているCPUに適した高速な実装を利用していることが重要である．例えば，IntelのCPUを使用している場合はIntel MKL，AMDのCPUの場合はAOCL，または汎用的なOpenBLASなどが最適である．

現在の環境で，numpyが内部的に利用しているBLASおよびLAPACKの情報を確認するためには，`np.show_config()`関数を使用する．これにより，現在のnumpyがどのBLASおよびLAPACKを使用しているか，またどのSIMD拡張命令（例えばSSEやAVXなど）に対応しているかを確認できる．

- https://numpy.org/doc/stable/reference/generated/numpy.show_config.html
  > Show libraries and system information on which NumPy was built and is being used


In [ ]:
np.show_config()


blas_armpl_info:
  NOT AVAILABLE
blas_mkl_info:
    libraries = ['mkl_rt', 'pthread']
    library_dirs = ['/usr/local/anaconda3/envs/python311/lib']
    define_macros = [('SCIPY_MKL_H', None), ('HAVE_CBLAS', None)]
    include_dirs = ['/usr/local/anaconda3/envs/python311/include']
blas_opt_info:
    libraries = ['mkl_rt', 'pthread']
    library_dirs = ['/usr/local/anaconda3/envs/python311/lib']
    define_macros = [('SCIPY_MKL_H', None), ('HAVE_CBLAS', None)]
    include_dirs = ['/usr/local/anaconda3/envs/python311/include']
lapack_armpl_info:
  NOT AVAILABLE
lapack_mkl_info:
    libraries = ['mkl_rt', 'pthread']
    library_dirs = ['/usr/local/anaconda3/envs/python311/lib']
    define_macros = [('SCIPY_MKL_H', None), ('HAVE_CBLAS', None)]
    include_dirs = ['/usr/local/anaconda3/envs/python311/include']
lapack_opt_info:
    libraries = ['mkl_rt', 'pthread']
    library_dirs = ['/usr/local/anaconda3/envs/python311/lib']
    define_macros = [('SCIPY_MKL_H', None), ('HAVE_CBLAS', None)]

これにより，以下のような情報が表示される．

- BLASやLAPACKライブラリが何であるか（例えば`openblas`や`mkl`など）
- 使用しているSIMD拡張命令（例えばSSE, AVX）

例えば，出力の一例として，以下のような結果が得られることがある．

```plaintext
blas_opt_info:
    libraries = ['cblas', 'blas']
    library_dirs = ['/usr/lib']
    include_dirs = ['/usr/include']
lapack_opt_info:
    libraries = ['lapack', 'blas']
    library_dirs = ['/usr/lib']
    include_dirs = ['/usr/include']
```

この場合，`libraries`に`['cblas', 'blas']`とあるため，BLASおよびLAPACKの参照実装が使われていることがわかる．参照実装は最適化が少なく，特定のCPUに特化していないため，動作が遅い可能性がある．

一方，次のような結果が表示されることもある．

```plaintext
openblas_info:
    libraries = ['openblas64_']
    library_dirs = ['/usr/lib']
    include_dirs = ['/usr/include']
```

このような場合，64ビット対応の**OpenBLAS**が使われていることが確認できる．OpenBLASは高速なBLAS実装の1つであり，CPUに応じて最適な演算を行うことができるため，計算のパフォーマンスが向上する．

### 1.13.4. <a id='toc1_13_4_'></a>[scipyのBLAS・LAPACK](#toc0_)


**scipy**も，内部的にBLASやLAPACKを利用して高速に線形代数の計算を行っている．scipyがどのBLASやLAPACKを利用しているかを確認するためには，`scipy.show_config()`関数を使用する．この関数は，現在の環境でscipyがどのライブラリを使っているかを表示する．着目すべき点は，numpyでの`np.show_config()`と同様で，どのBLASやLAPACKが利用されているか，最適化されているかなどである．


- https://docs.scipy.org/doc/scipy/reference/generated/scipy.show_config.html
    > Show libraries and system information on which SciPy was built and is being used

次のように，`scipy.show_config()`を実行することで，情報を確認できる．


In [ ]:
scipy.show_config()


Build Dependencies:
  blas:
    detection method: pkgconfig
    found: true
    include directory: /usr/local/anaconda3/envs/python311/include
    lib directory: /usr/local/anaconda3/envs/python311/lib
    name: mkl-sdl
    openblas configuration: unknown
    pc file directory: /usr/local/anaconda3/envs/python311/lib/pkgconfig
    version: '2023.1'
  lapack:
    detection method: pkgconfig
    found: true
    include directory: /usr/local/anaconda3/envs/python311/include
    lib directory: /usr/local/anaconda3/envs/python311/lib
    name: mkl-sdl
    openblas configuration: unknown
    pc file directory: /usr/local/anaconda3/envs/python311/lib/pkgconfig
    version: '2023.1'
  pybind11:
    detection method: pkgconfig
    include directory: /usr/local/anaconda3/envs/python311/include
    name: pybind11
    version: 2.10.4
Compilers:
  c:
    commands: /croot/scipy_1696543286448/_build_env/bin/x86_64-conda-linux-gnu-cc
    linker: ld.bfd
    name: gcc
    version: 11.2.0
  c++:
    comm

例えば，次のような結果が得られることがある．

```plaintext
blas_opt_info:
    libraries = ['openblas']
    library_dirs = ['/usr/lib']
    include_dirs = ['/usr/include']
lapack_opt_info:
    libraries = ['openblas']
    library_dirs = ['/usr/lib']
    include_dirs = ['/usr/include']
```

この場合，`libraries`に`openblas`とあるため，scipyがOpenBLASを利用していることがわかる．
また，場合によっては，scipyが使用しているBLAS/LAPACKの情報が，`np.show_config()`で表示される情報とほぼ同じである場合もある．これは，numpyとscipyが同じBLAS/LAPACKライブラリを共有しているためである．

### 1.13.5. <a id='toc1_13_5_'></a>[疎行列の実装とライブラリ](#toc0_)

Pythonで疎行列を扱う際には，前述のとおりscipyの`scipy.sparse`を使用すればよい．

疎行列を用いた線形代数の操作（連立方程式の解法や固有値計算など）は，`scipy.sparse.linalg`に含まれている．このモジュールでは，外部の疎行列ソルバー（例えば[ARPACK](https://en.wikipedia.org/wiki/ARPACK)や[PROPACK](http://sun.stanford.edu/~rmunk/PROPACK/)）のラッパーや，scipy独自の実装を利用して計算を行うようである．
疎行列用のBLASライブラリである[Sparse BLAS](https://math.nist.gov/spblas/)も存在するが，scipyでは独自にPythonで疎行列の処理を実装しているようである．

なおpython以外の言語でも，疎行列を用いた実用的な計算を実行するためには，適切なライブラリを選択することが重要である．以下は，メジャーな疎行列用ライブラリの例である．

- [SuiteSparse](https://github.com/DrTimothyAldenDavis/SuiteSparse)：疎行列に関するさまざまなパッケージ
- [cuSPARSE](https://docs.nvidia.com/cuda/cusparse/)：NVIDIAが提供するCUDA用の疎行列計算ライブラリ
